# Capstone Project
**Beibarys Nyussupov, Joseph Tadros, Luke Ducker**


This project assesses university professors using a large, publicly available dataset scraped from RateMyProfessor.com. The objective is to integrate core concepts from the course into a single applied analysis and to extract actionable insights about teaching quality and student perceptions. The dataset contains aggregated student ratings and related attributes for a broad sample of professors, with low individual response rates but substantial overall scale. Prior research reports a correlation of approximately 0.7 between RateMyProfessor ratings and official end-of-course teaching evaluations, which supports the analytical value of this source despite known response bias. All data collection and basic structuring steps were completed in advance, while data science-relevant preprocessing, including the identification and handling of missing data, was performed in this project. To address the research questions, appropriate statistical methods were selected based on underlying assumptions, with explicit justification provided throughout the report. Visualizations support interpretation and highlight key patterns in the data. All hypothesis testing uses α = 0.005 to reduce false positive findings (Habibzadeh, 2025). 


## Dataset description

### First dataset: `rmpCapstoneNum.csv`
Dataset contains 89893 records. Each of these
records (rows) corresponds to information about one professor. 

Columns:

`1: Average Rating (the arithmetic mean of all individual quality ratings of this professor)`

`2: Average Difficulty (the arithmetic mean of all individual difficulty ratings of this professor)`

`3: Number of ratings (simply the total number of ratings these averages are based on)`

`4: Received a “pepper”? (Boolean - was this professor judged as “hot” by the students?)`

`5: The proportion of students that said they would take the class again`

`6: The number of ratings coming from online classes`

`7: Male gender (Boolean – 1: determined with high confidence that professor is male)`

`8: Female (Boolean – 1: determined with high confidence that professor is female`


### Second dataset: ` rmpCapstoneQual.csv`
Dataset contains same 89893 records. 

Columns: 

`1: Major/Field`

`2: University`

`3: US State (2 letter abbreviation)`

### Third dataset: `rmpCapstoneTags.csv`
Dataset contains same 89893 records. 

The numbers in these columns correspond to the raw number of “tags” a professor has received. A student can award up to 3 such tags, but doesn’t have to award any. These tags are supposed to characterize the teaching style of the professor qualitatively, beyond ratings. 

Columns: 

`1: “Tough grader”`

`2: “Good feedback”`

`3: “Respected”`

`4: “Lots to read”`

`5: “Participation matters”`

`6: “Don’t skip class or you will not pass”`

`7: “Lots of homework”`

`8: “Inspirational”`

`9: “Pop quizzes!”`

`10: “Accessible”`

`11: “So many papers”`

`12: “Clear grading”`

`13: “Hilarious”`

`14: “Test heavy”`

`15: “Graded by few things”`

`16: “Amazing lectures”`

`17: “Caring”`

`18: “Extra credit”`

`19: “Group projects”`

`20: “Lecture heavy"`

**How to Run This Notebook:**

**1. Run all cells before Q1.**
- This initializes imports, helper functions, data loading, preprocessing, and global configuration.

**2. After that, each question section is independent.**

**3. For any question QX:**
- First run the cell that defines qX_df.
- Then run the remaining cells in that section in order.
- Skipping the qX_df cell will cause errors in downstream cells.

In [1]:
# libraries 
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# statistical tests
from scipy import stats
from scipy.stats import randint, uniform

from cliffs_delta import cliffs_delta as cd

# machine learning 
# train test split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import (KFold, StratifiedKFold, 
                                     train_test_split,
                                     cross_val_score, cross_validate, 
                                     GridSearchCV, RandomizedSearchCV)
import optuna
from optuna.samplers import TPESampler
from optuna.visualization import (plot_optimization_history, 
                                  plot_param_importances, 
                                  plot_slice, 
                                  plot_parallel_coordinate)

# preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer

# evaluation
from sklearn.metrics import (r2_score, root_mean_squared_error, 
                             mean_squared_error, recall_score, precision_score, 
                             f1_score, accuracy_score, roc_auc_score, confusion_matrix, 
                             ConfusionMatrixDisplay, classification_report, roc_curve, auc)
import shap

# models 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV, RidgeCV, LassoCV, ElasticNetCV
from sklearn import svm

# cloning model 
from sklearn.base import clone

# reproducibility 
import random

# specify seed 
n_number = 12250697
random.seed(n_number)
np.random.seed(n_number)

In [2]:
# useful functions
# simple significance test interpretation
def significance(alpha, p_value):
    """
    Decide based on alpha. 
    Note: p is the probability, under H0, of observing a statistic at least this extreme.

    """
    print(f"p-value: {p_value:.6g}")
    if p_value < alpha:
        print(f"p = {p_value:.6g} < α = {alpha} = Reject H0 (statistically significant).")
    else:
        print(f"p = {p_value:.6g} ≥ α = {alpha} = Fail to reject H0 (not statistically significant).")

# short-cut for assessing regression model
def score_model(model, x_train, x_test, y_train, y_test):

    # fit the model 
    model.fit(x_train, y_train)

    y_train_hat = model.predict(x_train)
    y_test_hat = model.predict(x_test)

    #r2_score
    train_r2 = r2_score(y_train, y_train_hat)
    test_r2 = r2_score(y_test, y_test_hat)

    #mse
    train_rmse = root_mean_squared_error(y_train, y_train_hat)
    test_rmse = root_mean_squared_error(y_test, y_test_hat)

    out = {
        "Training RMSE": train_rmse,
        "Training R2": train_r2,
        "Testing RMSE": test_rmse,
        "Testing R2": test_r2
    }

    return out

# fine-tuning function for models (optuna)
def start_study_optuna(objective, n_trials, sampler_seed, direction="minimize"):
    # sampler for reproducibility
    sampler = TPESampler(seed=sampler_seed)

    # start the study for fine-tuning
    study = optuna.create_study(direction=direction, sampler=sampler)
    study.optimize(objective, n_trials=n_trials)

    # print best parameters 
    print(f"Best Params: {study.best_params}")
    return study

# short-cut for assessing classification model
def score_classifier(model, x_train, x_test, y_train, y_test, threshold=0.5):

    # fit model
    model.fit(x_train, y_train)

    # predicted probabilities
    y_train_proba = model.predict_proba(x_train)[:, 1]
    y_test_proba = model.predict_proba(x_test)[:, 1]

    # predicted classes
    y_train_hat = (y_train_proba >= threshold).astype(int)
    y_test_hat = (y_test_proba >= threshold).astype(int)

    # confusion matrix on test
    tn, fp, fn, tp = confusion_matrix(y_test, y_test_hat).ravel()

    # specificity
    specificity = tn / (tn + fp)

    out = {
        # probability-based metrics
        "Training ROC-AUC": roc_auc_score(y_train, y_train_proba),
        "Testing ROC-AUC": roc_auc_score(y_test, y_test_proba),

        # threshold-based metrics
        "Training Accuracy": accuracy_score(y_train, y_train_hat),
        "Testing Accuracy": accuracy_score(y_test, y_test_hat),

        "Training Precision": precision_score(y_train, y_train_hat),
        "Testing Precision": precision_score(y_test, y_test_hat),

        "Training Recall": recall_score(y_train, y_train_hat),  # sensitivity
        "Testing Recall": recall_score(y_test, y_test_hat),

        "Testing Specificity": specificity,

        "Training F1": f1_score(y_train, y_train_hat),
        "Testing F1": f1_score(y_test, y_test_hat),

        # diagnostics
        "Confusion Matrix (test)": [[tn, fp], [fn, tp]]
    }

    return out

# function for ROC-AUC plot 
def add_roc(ax, y_true, y_prob, label, lw=2):
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, linewidth=lw, label=f"{label} (AUC={roc_auc:.4f})")
    return roc_auc

## Pre-processing

In [ ]:
# read and write locations
data_folder = os.path.join("..", "data")
raw_folder = os.path.join(data_folder, "raw")

# file names
rmp_num_filename = "rmpCapstoneNum.csv"
rmp_qual_filename = "rmpCapstoneQual.csv"
rmp_tags_filename = "rmpCapstoneTags.csv"

In [ ]:
# import the data 
df_num = pd.read_csv(os.path.join(raw_folder, rmp_num_filename), header=None)
df_qual = pd.read_csv(os.path.join(raw_folder, rmp_qual_filename), header=None)
df_tags = pd.read_csv(os.path.join(raw_folder, rmp_tags_filename), header=None)

In [ ]:
# headers for first dataset 
df_num_column_names = [
     "average_rating",
     "average_difficulty",
     "number_of_ratings",
     "received_a_pepper",
     "would_take_again",
     "number_of_ratings_online",
     "male_gender",
     "female_gender",
]

# headers for second dataset
df_qual_column_names = [
     "major",
     "university",
     "state",
]

# headers for third dataset 
df_tags_column_names = [
    "tough_grader",
    "good_feedback",
    "respected",
    "lots_to_read",
    "participation_matters",
    "dont_skip_class_or_you_will_not_pass",
    "lots_of_homework",
    "inspirational",
    "pop_quizzes",
    "accessible",
    "so_many_papers",
    "clear_grading",
    "hilarious",
    "test_heavy",
    "graded_by_few_things",
    "amazing_lectures",
    "caring",
    "extra_credit",
    "group_projects",
    "lecture_heavy"
]

# assign headers to dataframes using .columns attribute 
df_num.columns = df_num_column_names
df_qual.columns = df_qual_column_names
df_tags.columns = df_tags_column_names

In [ ]:
# join the data for the easier manipulation 
df = df_num.join(df_qual).join(df_tags)

In [ ]:
# check the data set (first five rows)
df.head()

In [ ]:
# information about the dataset 
df.info()

### Null Values

In [ ]:
# check null values of the dataset by each column. 
df.isna().sum()

A large block of missing data appears in the dataset. Exactly 19,889 records share missing values across all core numerical, categorical, and qualitative variables, with the exception of the “would take again” column. These missing values occur in the same row positions, which indicates a systematic issue rather than random missingness. In addition, all tag variables equal zero for these same records, which implies an absence of meaningful rating information rather than true zero-valued responses. Together, these patterns suggest these rows do not represent valid professor evaluations. Removing all 19,889 records is therefore justified, as retaining them would introduce noise and bias.

In [ ]:
# drop null values from the dataset 
df = df.dropna(subset="average_rating")
# check the results
df.isna().sum()

In [ ]:
# check information about the dataset after removal 
df.info()

After removing the invalid records, the dataset contains 70,004 observations. The only remaining missing data is in the would_take_again variable, where a large proportion of values is missing. This variable is a bounded proportion, and the extent of missingness is too high for null values removal or imputation to be appropriate. Imputing values would impose artificial structure on a substantial share of the data and bias both estimates and inference. For this reason, we do not modify this column unless explicitly required. 

### Number of ratings threshold 
This section addresses the reliability of average professor ratings. An average rating based on very few student evaluations is unstable and can produce extreme values that do not reflect typical teaching quality. Professors with only one or two ratings are therefore not comparable to those evaluated by many students. To reduce noise and improve interpretability, an optimal number of ratings threshold is chosen and applied so that average ratings are computed and analyzed only when they are supported by a sufficient volume of observations. This choice reduces the influence of outliers driven by sparse data and ensures that comparisons across professors rely on more reliable estimates.

In [ ]:
# distribution of number of ratings 
df["number_of_ratings"].describe()


In [ ]:
# plot for distribution of number of ratings 
plt.figure(figsize = (12, 10))

# histogram 
plt.hist(df["number_of_ratings"], 
         bins = 100, 
         alpha = 0.7)

# show median and mean value on the plot 
# median
plt.axvline(df["number_of_ratings"].median(), 
           color = "green", 
           linestyle = "dashed", 
           linewidth = 2, 
           label = f"median: {df['number_of_ratings'].median():.1f}")

# mean
plt.axvline(df["number_of_ratings"].mean(), 
           color = "red", 
           linestyle = "dashed", 
           linewidth = 2, 
           label= f"mean: {df['number_of_ratings'].mean():.1f}")

# aesthetics 
# title
plt.title("Distribution of number of ratings", 
          fontweight = "bold", 
          fontsize = 18)

# x axis 
plt.xlabel("Number of ratings", 
           fontsize = 16)
# limit
plt.xlim(0, 50)

# y axis
plt.ylabel("Frequency", 
           fontsize = 16)
# size for both x and y ticks 
plt.tick_params(axis = "both", 
                labelsize = 14)

# legend 
plt.legend(fontsize = 16)

# show the plot
plt.show()

In [ ]:
# filter out the professors with less than 5 ratings (mean)
df_filtered = df[df["number_of_ratings"] >= 5].copy()

In [ ]:
# check the distribution and information about the data after filter 
df_filtered["number_of_ratings"].describe()

The distribution of the number of ratings is highly right skewed, with a median of 3 and a mean of 5.4. Most professors receive only a small number of ratings, which leads to unstable and potentially extreme average values. To address this, we set a minimum threshold of five ratings and excluded professors with fewer than five evaluations. This cutoff is close to the mean and removes cases where average ratings are driven by very sparse data, while preserving the majority of the sample and improving the reliability of comparisons across professors. After applying this filter, 25,368 professors remain in the analysis dataset.


In [ ]:
# info about the dataset 
df_filtered.info()

### Categorical variables 
In this section, we address the handling of categorical variables. Teaching style tags were normalized by dividing each tag count by the total number of tags received by a professor, converting raw counts into proportions. This normalization controls for differences in the total number of ratings and ensures that tag values are comparable across professors. Cases with no assigned tags resulted in undefined values after normalization and were set to zero, reflecting the absence of tag information rather than meaningful signal.

In addition, we examine the gender indicators provided in the dataset. The male and female variables are treated as categorical indicators.

In [ ]:
# distribution of all tags before normalization 
# starting from 11th variable 
# .describe 
df_filtered.iloc[:, 11:].describe()

In [ ]:
# calculate total # of tags for each professor
total_tag_counts = df_filtered[df_tags_column_names].sum(axis = 1)

# normalize tag columns to be between 0 and 1 by dividing tag counts by total tag counts
for col in df_tags_column_names:
    df_filtered[col] = df_filtered[col] / total_tag_counts

# fill na values with 0 that resulted from divide by zero issues
df_filtered[df_tags_column_names] = df_filtered[df_tags_column_names].fillna(0)

In [ ]:
# look at the distribution of tags after the normalization 
df_filtered.iloc[:, 11:].describe()

Now, we are looking at gender variables. 

In [ ]:
# check male and female gender error, where both are 1, or both 0
both_11 = df_filtered[(df_filtered["male_gender"] == 1) & (df_filtered["female_gender"] == 1)].shape[0]
both_00 = df_filtered[(df_filtered["male_gender"] == 0) & (df_filtered["female_gender"] == 0)].shape[0]

# print the results 
print(f"Number of professors with both male and female gender as 1: {both_11}")
print(f"Number of professors with both male and female gender as 0: {both_00}")

In [ ]:
df_filtered.info()

This is a clear data error and should be removed. 

In [ ]:
# drop rows where both male and female gender are 1 or 0
df_filtered_final = df_filtered[~((df_filtered["male_gender"] == 1) & (df_filtered["female_gender"] == 1)) & 
                          (~((df_filtered["male_gender"] == 0) & (df_filtered["female_gender"] == 0)))].copy()

# check the results
df_filtered_final.info()


After data pre-processing, we are left with ~18k rows. Now we will proceed answering research questions. 

## Q1. Activists have asserted that there is a strong gender bias in student evaluations of professors, with male professors enjoying a boost in rating from this bias...We would like you to answer the question whether there is evidence of a pro-male gender bias in this dataset.


In [ ]:
# separate the average ratings into two groups based on gender
# Male = 1, Male = 0
df1_male = df_filtered_final[df_filtered_final["male_gender"] == 1].copy()
df1_female = df_filtered_final[df_filtered_final["male_gender"] == 0].copy()

In [ ]:
# count number of professors in each group
print(f"Number of male professors: {df1_male.shape[0]}")
print(f"Number of female professors: {df1_female.shape[0]}")

A difference is noticeable, but not very large. We can proceed with the analysis. 

In [ ]:
# compare distributions of average rating between male and female professors
plt.figure(figsize=(12, 10))

# box plot for male and female professors 
sns.boxplot(data=df_filtered_final, 
            x='average_rating', 
            y='male_gender', 
            palette = "Set2", 
            showfliers = False, 
            medianprops = {"color": "red", "linewidth": 2}, 
            orient= "horizontal")

# aesthetics 
plt.title('Average Rating Distribution by Gender', 
          fontweight = "bold",
          fontsize = 18)

# xlabel
plt.xlabel('Average Rating', 
           fontsize = 16)
# ylabel
plt.ylabel('Gender (1 = Male, 0 = Female)', 
           fontsize = 16)

# legend
plt.legend(['0 - Female', 
            '1 - Male'], 
            fontsize = 16,
           loc = "upper left")
# ticks 
plt.tick_params(axis = "both", 
                labelsize = 14)

plt.show()

The boxplot suggests a small difference in average ratings by gender. The distributions for male and female professors overlap substantially, but the male distribution appears slightly shifted to the right, with a higher median average rating. The spread of ratings is similar across genders, indicating comparable variability. To further test the difference, we have to apply a significance test. 

### Mann-Whitney U Test (one-tailed) 

`Rationale for the test:` For this question, we test whether male professors receive higher average ratings than female professors. Average ratings are bounded, derived from ordinal Likert-scale responses, and do not reliably satisfy normality assumptions at the individual professor level. Population parameters such as the true mean and variance are unknown.

We therefore use a one-sided Mann-Whitney U test, which is appropriate for comparing central tendency between two independent groups without assuming normality. This test evaluates whether ratings for one group tend to be systematically higher than those for the other, consistent with the directional hypothesis of pro-male bias.

`Null hypothesis (H0):` The median average rating for male professors is equal to the median average rating for female professors.

`Alternative hypothesis (Ha):` The median average rating for male professors is higher than the median average rating for female professors.

`Significance level:` 0.005

`Interpretation:` The p-value represents the probability of observing a difference in average ratings at least as extreme as the one observed, assuming the null hypothesis is true.
- If the p-value is smaller than 0.005, we reject H0 and conclude that there is statistically significant evidence of a pro-male bias in average ratings.
- If the p-value is larger than 0.005, we fail to reject H0 and conclude that the observed difference is consistent with random variation and does not provide sufficient evidence of gender bias.

In [ ]:
# conduct mann-whitney u test for average rating using our function
u_stat, p_value = stats.mannwhitneyu(df1_male['average_rating'], 
                                     df1_female['average_rating'], 
                                     alternative='greater')
# interpretation 
significance(0.005, p_value)


Our p-value (0.00024523) is smaller than the significance threshold of 0.005, so we reject the null hypothesis and conclude that the median average rating for male professors is higher than that for female professors.

This result does not establish causal evidence of pro-male rating bias, as the data are observational, not randomized, and we do not control for all potential confounders such as course type, institution, teaching experience, or selection effects in who chooses to rate professors.

However, we do find statistical evidence of a systematic difference in ratings by gender, with male professors receiving higher typical ratings in this dataset. This pattern is consistent with, but does not by itself prove, the presence of pro-male bias in average professor ratings.

## Q2. Is there a gender difference in the spread (variance/dispersion) of the ratings distribution? Again, it is advisable to consider the statistical significance of any observed gender differences in this spread. 

In [ ]:
# again_separate the average ratings into two groups based on gender
# Male = 1, Male = 0
df2_male = df_filtered_final[df_filtered_final["male_gender"] == 1].copy()
df2_female = df_filtered_final[df_filtered_final["male_gender"] == 0].copy()

In [ ]:
# count number of professors in each group
print(f"Number of male professors: {df2_male.shape[0]}")
print(f"Number of female professors: {df2_female.shape[0]}")


In [ ]:
# plot the variance of average rating dipersion between male and female professors
# kernel density plot 
plt.figure(figsize=(12, 10))

# plot
sns.kdeplot(df2_male['average_rating'], 
           label='Male', 
           color='blue', 
           fill=True, 
           legend= True, 
           alpha=0.15
           )
sns.kdeplot(df2_female['average_rating'],
              label='Female', 
              color='green', 
              fill=False, 
              legend= True)

# aesthetics
plt.title('Average Rating Dispersion by Gender', 
          fontweight = "bold",
          fontsize = 18)
# xlabel
plt.xlabel('Average Rating',
              fontsize = 16)
# ylabel
plt.ylabel('Density',
                fontsize = 16)

# axis 
plt.tick_params(axis = "both", 
                labelsize = 14)
# legend 
plt.legend()
# ticks
plt.show()

The density curves for average ratings by gender are highly similar, indicating comparable dispersion for male and female professors. Both distributions cover a similar range of ratings and exhibit similar tail behavior, suggesting no substantial difference in variability. While the male distribution appears slightly shifted to the right, consistent with higher typical ratings observed in Q1, the overall spread and shape of the distributions closely align.

From a visual inspection, we do not observe meaningful differences in variance or dispersion between the two groups. Any differences in spread appear minor relative to the overall similarity of the distributions and are unlikely to reflect a practically important gender difference in rating variability.

This visual evidence suggests that gender differences in ratings are primarily related to central tendency rather than dispersion, motivating a formal statistical test of variance in the subsequent analysis.

### Levene's test 

`Rationale for the test:` This question asks whether male and female professors differ in the spread of their average rating distributions. We need a test of dispersion, not a test of location. Average ratings are bounded (1 to 5) and do not satisfy normality assumptions, so classical variance tests that rely on normality, such as an F-test, are not appropriate. We therefore use Levene's test because it tests equality of variance while remaining robust under non-normal data. We use the median-centered version to further reduce sensitivity to skewness and outliers. The test is two-tailed because the question asks whether dispersion differs in either direction, not whether one gender has higher variance.

`Null hypothesis (H0):` The dispersion (variance) of average ratings for male professors is equal to the dispersion of average ratings for female professors.

`Alternative hypothesis (Ha):` The dispersion of average ratings significantly differs between male and female professors.

`Significance level:` 0.005

If the p-value is smaller than 0.005, we reject H0 and conclude that the dispersion of average ratings differs significantly by gender.

If the p-value is larger than 0.005, we fail to reject H0, indicating that the observed difference in dispersion is consistent with random variation and that the data do not provide sufficient statistical evidence of a gender difference in rating variability.


In [ ]:
# dispersion of average ratings 
# levene's test for equal variances
levene_stat, levene_p_value = stats.levene(df2_male['average_rating'], 
                                      df2_female['average_rating'], 
                                      center='median')

# check the result 
significance(0.005, levene_p_value)

Since the p-value (0.00000598) is smaller than the significance level α = 0.005, we reject the null hypothesis. This provides statistically significant evidence that the dispersion of average ratings differs between male and female professors. The observed difference in spread is unlikely to be due to random variation alone.

## Q3. What is the likely size of both of these effects (gender bias in average rating, gender bias in spread of average rating), as estimated from this dataset? Please use 95% confidence and make sure to report each/both. 

### Effect size of gender bias in average rating

Because the ratings data are derived from ordinal Likert-scale responses and violate normality assumptions, we do not use parametric effect size measures such as Cohen's d or Hedges' g, which rely on normality and equal variances. Instead, we require a non-parametric effect size that is interpretable and appropriate for rank-based comparisons.

We therefore use Cliff's Delta (δ), a non-parametric measure of effect size suitable for ordinal and non-normal data. Cliff's delta quantifies the degree of distributional non-overlap between two groups by comparing all pairwise observations. It ranges from −1 to +1, where 0 indicates complete overlap and larger absolute values indicate stronger separation between groups.

Following the recommendations in [Meissel and Yao (2024)](https://github.com/NBeibarys/Project-Hypothesis-Testing-of-MovieRatings-Using-Python/blob/main/project_guidelines/Using%20Cliff%E2%80%99s%20Delta%20as%20a%20Non-Parametric%20Effect%20Size%20Measure.pdf), effect sizes are interpreted as:

|δ| < 0.15: negligible effect

0.15 ≤ |δ| < 0.33: small effect

0.33 ≤ |δ| < 0.47: medium effect

|δ| ≥ 0.47: large effect

The sign of δ indicates the direction of the effect relative to the reference group.



To estimate the uncertainty around the effect sizes, we compute 95% confidence intervals using bootstrapping. Bootstrapping is appropriate in this setting because it does not assume normality of the underlying data and is well suited for ordinal outcomes and non-parametric statistics.

For each effect, we repeatedly resample the observed data with replacement and recompute the corresponding effect size on each bootstrap sample. The empirical distribution of these bootstrap estimates is then used to construct percentile-based 95% confidence intervals.

This approach allows us to quantify the likely range of both:
- the location effect (gender difference in average ratings, measured via Cliff's delta), and
- the dispersion effect (gender difference in rating spread, measured via robust dispersion statistics), without relying on parametric assumptions. As a result, the reported confidence intervals reflect the variability inherent in the observed data rather than model-based assumptions.

In [ ]:
 # divide average ratings by groups of gender and convert to arrays (bootstrap expects that)
df3_male = df_filtered_final[df_filtered_final["male_gender"] == 1].copy()
df3_female = df_filtered_final[df_filtered_final["male_gender"] == 0].copy()

# define function for effect size 
def cliffs_delta_trap(sample1, sample2, axis = 0):
    return cd(sample1, sample2)[0]

# Perform the bootstrap
result = stats.bootstrap(
    (df3_male['average_rating'].to_numpy(), 
    df3_female['average_rating'].to_numpy()), 
    statistic=cliffs_delta_trap,
    confidence_level=0.95, 
    random_state= n_number, 
    vectorized= False,
    method='BCa', 
    n_resamples=10000)

# compute the confidence interval
ci_lower, ci_upper = result.confidence_interval.low, result.confidence_interval.high

# print the results 
# point estimate
print(f"Effect size (Cliffs delta): {cd(df3_male['average_rating'], 
                                        df3_female['average_rating'])[0]:.4f}")

# confidence interval
print(f"95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")




In [ ]:
# plot boostrap distribution of cliffs delta
# figure size 
plt.figure(figsize=(12, 10))

# plot
plt.hist(result.bootstrap_distribution, 
         bins=40, 
         density=True, 
         color = "skyblue",
         edgecolor = "black",
         fill = True)

# point estimate line
plt.axvline(cd(df3_male['average_rating'].to_numpy(), 
                df3_female['average_rating'].to_numpy())[0], 
            color = "orange", 
            linestyle = "dashed", 
            label = f"Effect size: {cd(df3_male['average_rating'].to_numpy(), 
                                            df3_female['average_rating'].to_numpy())[0]:.4f}"
            )

# confidence interval lines
# lower
plt.axvline(ci_lower, 
            color = "red", 
            linestyle = "dashed", 
            label = f"95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]"
            )
# upper
plt.axvline(ci_upper, 
            color = "red", 
            linestyle = "dashed")

# aesthetics
# title 
plt.title("Bootstrap Distribution of Cliff's Delta for Average Rating by Gender", 
          fontweight = "bold",
          fontsize = 18)

# x and y axis labels
plt.xlabel("Cliff's Delta", 
           fontsize = 16)
plt.ylabel("Density", 
           fontsize = 16)

# ticks 
plt.tick_params(axis = "both", 
                labelsize = 14)

# legend 
plt.legend(fontsize = 16)

# show the plot 
plt.show()
         

A 95 percent confidence interval means that, over repeated samples, 95 percent of such intervals would contain the true Cliff’s delta. In this case, the estimated Cliff’s delta lies between 0.0127 and 0.0466, with a point estimate of 0.0297. This corresponds to a negligible effect size, meaning that the distributions of average ratings for male and female professors are almost entirely overlapping. Although this difference is statistically detectable given the large sample size, it does not represent a practically meaningful difference in central tendency and provides little evidence of substantive gender bias in average ratings.

Now we will compute a point estimate and 95% confidence interval for dispersion difference effect size. To measure the effect size of dispersion, we substract the female variance from male variance. 

### Effect size of gender bias in dispersion of average rating 

In [ ]:
# variance difference in average rating for male and female professors
def variance_diff(sample1, sample2, axis = 0):
    # compute variance for each sample
    variance1 = np.var(sample1, ddof = 1)
    variance2 = np.var(sample2, ddof = 1)
    return variance1 - variance2   # male minus female

# compute the bootstrap
result_variance = stats.bootstrap(
    data=(df3_male['average_rating'].to_numpy(), 
    df3_female['average_rating'].to_numpy()),
    statistic=variance_diff,
    confidence_level=0.95,
    n_resamples=10000,
    method='BCa',
    random_state=n_number,
    vectorized=False
)

# compute the confidence interval
ci_lower_variance, ci_upper_variance = result_variance.confidence_interval.low, result_variance.confidence_interval.high

# print the results 
# point estimate
print(f"Effect size (variance difference): {variance_diff(df3_male['average_rating'], df3_female['average_rating'])}")
# confidence interval
print(f"95% CI: [{ci_lower_variance:.4f}, {ci_upper_variance:.4f}]")



In [ ]:
# plot boostrap distribution of variance difference
# figure size 
plt.figure(figsize=(12, 10))

# plot
plt.hist(result_variance.bootstrap_distribution, 
         bins=40, 
         density=True, 
         color = "orange",
         edgecolor = "black",
         fill = True)

# point estimate line
plt.axvline(variance_diff(df3_male['average_rating'], 
                df3_female['average_rating']), 
            color = "blue", 
            linestyle = "dashed", 
            label = f"Effect size: {variance_diff(df3_male['average_rating'], df3_female['average_rating']):.4f}"
            )

# confidence interval lines
# lower
plt.axvline(ci_lower_variance, 
            color = "red", 
            linestyle = "dashed", 
            label = f"95% CI: [{ci_lower_variance:.4f}, {ci_upper_variance:.4f}]"
            )
# upper
plt.axvline(ci_upper_variance, 
            color = "red", 
            linestyle = "dashed")

# aesthetics
# title 
plt.title("Bootstrap Distribution of Variance Difference for Average Rating by Gender", 
          fontweight = "bold",
          fontsize = 18)

# x and y axis labels
plt.xlabel("Variance Difference", 
           fontsize = 16)
plt.ylabel("Density", 
           fontsize = 16)

# ticks 
plt.tick_params(axis = "both", 
                labelsize = 14)

# legend 
plt.legend(fontsize = 16)

# show the plot 
plt.show()

# show the plot 
plt.show()
         


The estimated variance difference in average ratings between male and female professors is −0.0776, with a 95 percent CI from −0.1128 to −0.0411. The interval lies entirely below zero, indicating that average ratings for female professors exhibit slightly greater dispersion than those for male professors. However, the magnitude of this difference is very small in absolute terms. This suggests that, although the difference in spread is statistically detectable due to the large sample size, it is negligible in practice and provides little evidence of meaningful gender bias in rating variability.

## Q4.  Is there a gender difference in the tags awarded by students? Make sure to test each of the 20 tags for a potential gender difference and report which of them exhibit a statistically significant difference. Comment on the 3 most gendered (lowest p-value) and least gendered (highest p-value) tags for a potential gender difference and report which of them exhibit a statistically significant difference. Comment on the 3 most gendered (lowest p-value) and least gendered (highest p-value) tags.

`Rationale for the test:` Each teaching tag is converted into a proportion of all tags awarded to a professor in order to account for differences in the total number of ratings. These tag proportions are bounded between 0 and 1 and exhibit skewness, violating normality and equal variance assumptions. Parametric tests are therefore not appropriate.

To compare tag distributions between male and female professors, we use a two-tailed Mann-Whitney U test. This non-parametric test is suitable for comparing two independent groups on ordinal or non-normal data and evaluates whether one group tends to receive systematically higher or lower tag proportions than the other.

`Null hypothesis (H0):` The distribution of tag proportions is the same for male and female professors.

`Alternative hypothesis (Ha):` The distribution of tag proportions significantly differs between male and female professors.

`Significance level:` 0.005

The p-value represents the probability of observing a difference in tag proportions, or a more extreme one, assuming the null hypothesis is true.

If the p-value is smaller than 0.005, we reject H0 and conclude that the tag is awarded in significantly different proportions to male and female professors.
If the p-value is larger than 0.005, we fail to reject H0 and conclude that any observed difference is consistent with random variation.

This test is applied separately to each of the 20 teaching tags in the dataset in order to identify which tags exhibit statistically significant gender differences.

In [ ]:
# filter our the data for this question 
df4_male = df_filtered_final[df_filtered_final["male_gender"] == 1].copy()
df4_female = df_filtered_final[df_filtered_final["male_gender"] == 0].copy()

# filter mask for all tags columns
tag_columns = df_tags_column_names
alpha = 0.005

# list for all tags 
results_q4 = []

# for loop for separating each movie subset into two groups and conducting the non-parametric test
for tag in tag_columns: 
    # separating into two groups
    males = df4_male[tag]
    females = df4_female[tag]

    # conducting the test 
    u_statistic, p_value = stats.mannwhitneyu(males, females, alternative= 'two-sided')
    
    # save data as well
    results_q4.append({
    "tag": tag,
    "n_female": len(females),
    "n_male": len(males),
    "U": u_statistic,
    "p_value": p_value,
    "significant": p_value < alpha
    })
        
    # # use the significance function created 
    # print(f"{tag} Result:")
    # print(f"{significance(0.005, p_value)}\n")

# check the results
results_q4_df = pd.DataFrame(results_q4).sort_values(by = 'p_value', ascending= False)
styled = results_q4_df.style.set_properties(**{
    "background-color": "white",
    "color": "black", 
     "border": "1px solid black"
}).format({"p_value": "{:.6e}"})

# sort by p-value
styled

We tested all 20 tag proportions for gender differences using two tailed Mann-Whitney U tests at a significance level of 0.005. Most tags show statistically detectable gender differences, while a small number do not. The least gendered tags are pop_quizzes and accessible, which have the highest p values and show no evidence of a gender difference, and tough_grader, which shows only weak evidence. The most gendered tags are hilarious, amazing_lectures, and a participation_matters, which have the lowest p values and the strongest statistical evidence of a difference in tag proportions by gender. 

## Q5. Is there a gender difference in terms of average difficulty? Again, a significance test is indicated.

In [ ]:
# separate into two groups based on gender
# Male = 1, Male = 0
df5_male = df_filtered_final[df_filtered_final["male_gender"] == 1].copy()
df5_female = df_filtered_final[df_filtered_final["male_gender"] == 0].copy()

In [ ]:
# check quantity 
# it is the same number of rows, so this code is just in case
print(f"Number of male professors: {df5_male.shape[0]}")
print(f"Number of female professors: {df5_female.shape[0]}")

In [ ]:
# compare distributions of average rating between male and female professors
plt.figure(figsize=(12, 10))

# box plot for male and female professors 
sns.boxplot(data=df_filtered_final, 
            x='average_difficulty', 
            y='male_gender', 
            palette = "Set3", 
            showfliers = False, 
            medianprops = {"color": "red", "linewidth": 2}, 
            orient= "horizontal")

# aesthetics 
plt.title('Average Difficulty Rating Distribution by Gender', 
          fontweight = "bold",
          fontsize = 18)

# xlabel
plt.xlabel('Average Difficulty Rating', 
           fontsize = 16)
# ylabel
plt.ylabel('Gender (1 = Male, 0 = Female)', 
           fontsize = 16)

# legend
plt.legend(['0 - Female', 
            '1 - Male'], 
            fontsize = 16,
           loc = "upper left")
# ticks 
plt.tick_params(axis = "both", 
                labelsize = 14)

plt.show()

The box plots of average difficulty ratings for male and female professors show substantial overlap and do not suggest a clear difference in typical difficulty. Visually, there is no strong evidence of a difference between the two groups. To formally assess whether any difference exists, we apply a two-tailed Mann-Whitney U test.

### Mann-Whitney U Test (two-tailed)

`Rationale for the test`: We use a two-tailed Mann-Whitney U test to compare average difficulty ratings by gender. This test is appropriate because average ratings are ordinal and non-normal.

`Null hypothesis (H0):` Average difficulty ratings for male professors are not different than average difficulty ratings of female professors.

`Alternative hypothesis (Ha):` Average difficulty ratings for male professors are significantly different from average difficulty ratings of female professors.

`Significance level:` 0.005

`Interpretation:` If the p-value is smaller than 0.005, it means such a result would be very unlikely by chance under the null hypothesis, so we reject the H0 and conclude that average difficulty ratings for male professors are significantly different from average difficulty ratings of female professors.

If the p-value is larger than 0.005, then the observed difference could easily occur by random variation, so we fail to reject the H0, meaning the data do not provide enough statistical evidence to conclude that there is a significant difference in average difficulty ratings by gender.

In [ ]:
# conduct mann-whitney u test for average rating using our function
u_stat, p_value = stats.mannwhitneyu(df5_male['average_difficulty'], 
                                     df5_female['average_difficulty'], 
                                     alternative='two-sided')
# interpretation 
significance(0.005, p_value)

The Mann-Whitney U test yielded a p-value of 0.828753, which is substantially larger than the significance level of 0.005. Therefore, we fail to reject the null hypothesis.

This result indicates that the observed differences in average difficulty ratings between male and female professors are consistent with random variation. The data provide no statistical evidence of a gender difference in perceived course difficulty.

## Q6. Please quantify the likely size of this effect at 95% confidence

Because average difficulty ratings are ordinal and non-normal, parametric effect size measures such as Cohen's d and Hedges' g are not appropriate. As introduced earlier in this report, we therefore use Cliff's delta (δ) as a non-parametric measure of effect size suitable for ordinal data (Meissel & Yao, 2024).

Cliff's delta is interpreted using the same thresholds defined previously: negligible (|δ| < 0.15), small (0.15 ≤ |δ| < 0.33), medium (0.33 ≤ |δ| < 0.47), and large (|δ| ≥ 0.47).

We apply this measure here to quantify the magnitude of any gender difference in average difficulty ratings, complementing the hypothesis test results.

In [ ]:
 # divide average difficulty ratings by groups of gender and convert to arrays (bootstrap expects that)
df6_male = df_filtered_final[df_filtered_final["male_gender"] == 1].copy()
df6_female = df_filtered_final[df_filtered_final["male_gender"] == 0].copy()

# define function for effect size 
def cliffs_delta_trap(sample1, sample2, axis = 0):
    return cd(sample1, sample2)[0]

# Perform the bootstrap
result_q6 = stats.bootstrap(
    (df6_male['average_difficulty'].to_numpy(), 
    df6_female['average_difficulty'].to_numpy()), 
    statistic=cliffs_delta_trap,
    confidence_level=0.95, 
    random_state= n_number, 
    vectorized= False,
    method='BCa', 
    n_resamples=10000)

# compute the confidence interval
ci_lower_q6, ci_upper_q6 = result_q6.confidence_interval.low, result_q6.confidence_interval.high

# print the results 
# point estimate
print(f"Effect size (Cliffs delta): {cd(df6_male['average_difficulty'], 
                                        df6_female['average_difficulty'])[0]:.4f}")

# confidence interval
print(f"95% CI: [{ci_lower_q6:.4f}, {ci_upper_q6:.4f}]")


In [ ]:
# plot boostrap distribution of cliffs delta
# figure size 
plt.figure(figsize=(12, 10))

# plot
plt.hist(result_q6.bootstrap_distribution, 
         bins=40, 
         density=True, 
         color = "darkgreen",
         edgecolor = "black",
         fill = True)

# point estimate line
plt.axvline(cd(df6_male['average_difficulty'].to_numpy(), 
                df6_female['average_difficulty'].to_numpy())[0], 
            color = "orange", 
            linestyle = "dashed", 
            label = f"Effect size: {cd(df6_male['average_difficulty'].to_numpy(), 
                                            df6_female['average_difficulty'].to_numpy())[0]:.4f}"
            )

# confidence interval lines
# lower
plt.axvline(ci_lower_q6, 
            color = "red", 
            linestyle = "dashed", 
            label = f"95% CI: [{ci_lower_q6:.4f}, {ci_upper_q6:.4f}]"
            )
# upper
plt.axvline(ci_upper_q6, 
            color = "red", 
            linestyle = "dashed")

# aesthetics
# title 
plt.title("Bootstrap Distribution of Cliff's Delta for Average Difficulty Rating by Gender", 
          fontweight = "bold",
          fontsize = 18)

# x and y axis labels
plt.xlabel("Cliff's Delta", 
           fontsize = 16)
plt.ylabel("Density", 
           fontsize = 16)

# ticks 
plt.tick_params(axis = "both", 
                labelsize = 14)

# legend 
plt.legend(fontsize = 16)

# show the plot 
plt.show()

After computing a 95% confidence interval using bootstrap resampling, the estimated Cliff's delta lies between -0.0152 and 0.0182, with a point estimate of 0.0018. This corresponds to a negligible effect size, indicating that the distributions of average difficulty ratings for male and female professors almost completely overlap.

Because the confidence interval includes zero and remains well within the negligible range, there is no practical evidence of a meaningful gender difference in perceived course difficulty. This result reinforces the hypothesis test findings and suggests that any statistically detectable differences are not substantively important at the user-experience level.

## Q7 - Build a regression model predicting average rating from all numerical predictors (the ones in the rmpCapstoneNum.csv) file. Make sure to include the R2 and RMSE of this model. Which of these factors is most strongly predictive of average rating? 

### Linear Regression

In [ ]:
# copy the data for the question 
df_7 = df_filtered_final[df_num_column_names].copy()

# drop female_gender as it is redundant (we have male_gender binary variable already)
df_7 = df_7.drop(columns = ['female_gender'])
# check 
df_7.info()

In [ ]:
# split the data into x and y 
# x 
x_7 = df_7.drop(columns = "average_rating")

# y 
y_7 = df_7["average_rating"]

# split the data into train and test data 
x_train_7, x_test_7, y_train_7, y_test_7 = train_test_split(x_7, y_7, test_size=0.2, random_state=n_number)

# check distributions of the data 
print(f'Number of rows for train data:{y_train_7.shape[0]}\n')
print(f'Number of rows for test data:{y_test_7.shape[0]}\n')

In [ ]:
# correlation matrix for x and y 
corr7_yx = x_train_7.corrwith(y_train_7, 
                              method = 'pearson').sort_values(ascending= False)
# check 
corr7_yx

The correlations indicate that average ratings are driven primarily by student satisfaction and course rigor rather than demographics. Would take again shows an extremely strong positive association with ratings, while average difficulty exhibits a strong negative relationship, suggesting that harder courses tend to receive lower evaluations. Received a pepper has a moderate positive association, whereas gender, number of ratings, and online share show near-zero correlations, indicating minimal influence. These results suggest that instructional experience and perceived difficulty dominate rating outcomes, while demographic and exposure-related variables contribute little explanatory value.

In [ ]:
# correlation of features with each other 
corr7_x = x_train_7.corr()

# Plot the heatmap using Seaborn
plt.figure(figsize=(12, 10)) # Adjust the figure size as needed

# plot itself
sns.heatmap(corr7_x, annot=True, cmap='coolwarm', fmt=".2f")

# aesthetics 
# title 
plt.title('Correlation Matrix of Numerical Features', fontweight = "bold", fontsize = 18)

# ticks
plt.tick_params(labelsize = 16)

# show the plot
plt.show()

The correlation matrix shows no strong pairwise correlations among the numerical predictors, indicating no evidence of multicollinearity. Most correlations are close to zero, including those involving gender, number of ratings, and online ratings. The largest association appears between average difficulty and “would take again” (≈ ~0.52), which reflects a meaningful but expected relationship rather than redundancy. Overall, the predictors capture distinct aspects of professor evaluation and can be included jointly in downstream models without collinearity concerns.

#### KNN Imputation 

Only the variable `would_take_again` contains missing values. Dropping these observations would remove a substantial amount of information, especially because this variable shows a strong association with average rating.

We therefore apply KNN imputation to recover missing values in `would_take_again` by leveraging similarity across the remaining numerical predictors. This method preserves local structure in the data and avoids the strong distributional assumptions imposed by mean or regression based imputation, which is appropriate given the bounded and ordinal nature of the ratings.

The number of neighbors governs the bias variance tradeoff of the imputation. To avoid introducing noise, we tune this parameter within a cross validated pipeline, ensuring that the imputed values improve out of sample performance.

We deliberately do not include a missing indicator for `would_take_again`. Although missingness exhibits statistical correlation with ratings, it reflects survey response behavior rather than any interpretable course characteristic. Including such indicators risks capturing artifacts of the data collection process rather than substantive effects and reduces the interpretability and external validity of the model. For this reason, missingness is handled solely through imputation.

In [ ]:
# fixed cross-validation
cv = KFold(n_splits=5, 
           shuffle=True, 
           random_state=n_number
           )

# pipeline for tuning 
knn_pipe = Pipeline([
    ("imputer", KNNImputer()),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

# define objective function for optuna 
def objective(trial):
    # parameters to tune
    params = {
        "imputer__n_neighbors": trial.suggest_int("imputer__n_neighbors", 2, 25),
        "imputer__weights": trial.suggest_categorical("imputer__weights", ["uniform", "distance"]),
        "imputer__add_indicator": trial.suggest_categorical("imputer__add_indicator", [False])
    }

    # clone our model and set parameters 
    pipe = clone(knn_pipe)
    pipe.set_params(**params)

    # cross-validation score with tuned parameters 
    scores = cross_val_score(
        pipe,
        x_train_7,
        y_train_7,
        cv=cv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )

    # return average RMSE 
    return -scores.mean()

# start the study for knn imputer tuning
study_knn = start_study_optuna(
    objective=objective,
    n_trials=25,
    sampler_seed=n_number,
    direction="minimize"
)

# set the best parameters to the pipeline
knn_pipe.set_params(**study_knn.best_params)

# check the model performance with the best parameters 
score_model(
    knn_pipe,
    x_train_7,
    x_test_7,
    y_train_7,
    y_test_7
)


The tuned preprocessing pipeline shows stable cross-validation performance. The best Optuna trial (trial 15) achieves a mean CV RMSE of ~0.584. The train and test results are also consistent, RMSE is 0.5832 on training and ~0.584 on testing, R² is ~0.6019 on training and ~0.62 on testing. This alignment suggests good generalization and no clear overfitting.

We use this tuned preprocessing plus linear regression setup as our baseline. It gives a fair reference point because the preprocessing is selected using cross-validation, so the baseline reflects expected out-of-sample behavior.

Next, we compare the tuned KNN-based imputation strategy against simpler imputation methods. In particular, we evaluate whether replacing KNN imputation with less complex approaches, such as mean or median imputation, leads to comparable out-of-sample performance. This comparison allows us to assess whether the additional complexity of KNN imputation is justified, or whether simpler imputers are sufficient for this problem.

In [ ]:
# fixed CV
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipelines for comparison
pipelines = {
    "KNNImputer (tuned)": Pipeline([
        # imputer 
        ("imputer", KNNImputer(n_neighbors=20, 
                               weights="uniform", 
                               add_indicator=False)),

        # scaler 
        ("scaler", StandardScaler()),

        # model 
        ("model", LinearRegression())
    ]),

    "SimpleImputer (mean)": Pipeline([
        ("imputer", SimpleImputer(strategy="mean", 
                                  add_indicator=False)),
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),
    "SimpleImputer (median)": Pipeline([
        ("imputer", SimpleImputer(strategy="median", 
                                  add_indicator=False)),
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),
}

# compare the pipelines
rows = []
for name, pipe in pipelines.items():
    results = cross_validate(
        pipe,
        x_train_7,
        y_train_7,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        return_train_score=False,
        n_jobs=-1
    )
    rows.append({
        "model": name,
        "R2_mean": results["test_r2"].mean(),
        "RMSE_mean": (-results["test_neg_root_mean_squared_error"]).mean()
    })

# create a dataframe for comparison
compare_imputers_df = (
    pd.DataFrame(rows)
      .sort_values("RMSE_mean", 
                   ascending= True)
      .reset_index(drop=True)
)

# check the results
compare_imputers_df

The comparison confirms that KNN imputation provides a clear advantage over simpler imputation strategies. The tuned KNN imputer achieves the best cross-validation performance, with an RMSE of ~0.584 and an R² of 0.601. In contrast, both mean and median imputation lead to noticeably worse results, with higher RMSE and lower R².

This gap indicates that missing values in `would_take_again` are not random and that borrowing information from similar observations improves predictive accuracy. Simpler imputers fail to capture this structure and introduce additional noise, resulting in inferior out-of-sample performance.

Given these results, we retain the tuned KNN imputation strategy as part of the preprocessing pipeline.

Next, we move to regularized modeling by fitting a Lasso regression on top of this optimized preprocessing setup. The goal is to assess whether introducing sparsity can simplify the model and improve interpretability without sacrificing out-of-sample performance.

#### Lasso Regression

In [ ]:
# model with all variables 
# pipeline for Lasso model
pipeline_q7 = Pipeline([
    ("imputer", KNNImputer(n_neighbors=20, 
                           weights= 'uniform', 
                           add_indicator= False
                           )),
    ("scaler", StandardScaler()),
    ("model", LassoCV(
        cv = 5, 
        random_state = n_number,
        alphas= 100, 
        max_iter= 10000
    ))
])

# cross-validation 
cv = KFold(n_splits=5, 
           shuffle=True, 
           random_state=n_number)

# run cross-validation
cv_results_raw_7 = cross_validate(
    pipeline_q7,
    x_train_7,
    y_train_7,
    cv=cv,
    scoring=("r2", "neg_root_mean_squared_error"),
    return_train_score=False, 
    return_estimator = True
)

# results 
cv_results_7 = [{
    "model": "LassoCV (all features)", 
    "R2_mean": cv_results_raw_7["test_r2"].mean(),
    "RMSE_mean": (-cv_results_raw_7["test_neg_root_mean_squared_error"]).mean()
}]

# save to dataframe for comparison later 
compare_models_q7_df = pd.DataFrame(cv_results_7).sort_values("RMSE_mean").reset_index(drop = True)

# check 
compare_models_q7_df

In [ ]:
# check the model on test data 
score_model(pipeline_q7, 
            x_train_7, 
            x_test_7, 
            y_train_7, 
            y_test_7)

The Lasso model achieves nearly identical cross-validated performance compared to the baseline. The change in RMSE is negligible and within expected sampling variability. This suggests that the baseline model already captures the dominant linear relationships in the data, and additional regularization does not provide a meaningful predictive advantage.

The value of Lasso in this setting is therefore interpretability rather than accuracy. By shrinking weaker coefficients toward zero, it confirms that only a small subset of predictors contributes materially to the model, while maintaining essentially the same out-of-sample performance.

In [ ]:
# extract feature weights from the model
model = pipeline_q7.named_steps["model"]

# feature names
feature_names = x_train_7.columns.to_list()
# coefficients
coef = model.coef_

# build dataframe
weights_df_7 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_7["abs_weight"] = weights_df_7["weight"].abs()
weights_df_7 = weights_df_7.sort_values("abs_weight", ascending=False)

# check 
weights_df_7

In [ ]:
# plot weights of the multiple linear regression model 
# figure size 
plt.figure(figsize = (12, 10))

# bar plot 
sns.barplot(y = weights_df_7["tag"], 
        x = weights_df_7["weight"])

# aesthetics
# title 
plt.title("Lasso Regression Coefficients for Predicting Average Rating", 
          fontweight="bold", 
          fontsize=18)

# axis labels
plt.xlabel("Coefficient value", fontsize=16)
plt.ylabel("Predictor", fontsize=16)

# ticks
plt.tick_params(axis="both", labelsize=14)

# show plot
plt.show()

To focus the model on meaningful predictors, we retain variables whose absolute standardized coefficients exceed 5 percent. This cutoff removes predictors with negligible contribution while keeping variables that materially affect predicted ratings.

Based on the Lasso results, three variables meet this criterion.

`would_take_again (0.456)`
- This is the strongest positive predictor. Professors whom students are willing to take again receive substantially higher ratings. This variable captures core teaching quality and overall student satisfaction.

`average_difficulty (-0.271)`
- This shows a strong negative relationship. Higher perceived difficulty is associated with lower ratings, even after controlling for other factors. This suggests that difficulty directly shapes student evaluations.

`received_a_pepper (0.119)`
- This has a moderate positive effect. Perceived attractiveness correlates with higher ratings, but the magnitude is much smaller than the effect of teaching related variables.

All remaining variables, including gender, number of ratings, and online rating volume, have coefficients close to zero and are excluded. Their limited weights indicate minimal incremental predictive value once the main drivers are accounted for.

This selection step yields a simpler and more interpretable model driven primarily by student experience and course perception rather than demographic or exposure related noise. We now evaluate the performance of this reduced model.

In [ ]:
# select features with |weight| >= 0.05
selected_features = (
weights_df_7
.loc[weights_df_7["abs_weight"] >= 0.05, "tag"]
.tolist()
)

# reduce training data 
x_train_reduced_7 = x_train_7[selected_features]
x_test_reduced_7 = x_test_7[selected_features]

# cross-validation 
cv = KFold(n_splits=5, 
           shuffle=True, 
           random_state=n_number)

# pipeline for model 
pipeline_q7_reduced = Pipeline([
    ("imputer", KNNImputer(n_neighbors=20, 
                           weights= 'uniform', 
                           add_indicator= False)),
    ("scaler", StandardScaler()),
    ("model", LassoCV(
        cv = cv, 
        random_state = n_number,
        alphas = 100, 
        max_iter= 10000
    ))
])

# run cross-validation
cv_results_raw_7_reduced = cross_validate(
    pipeline_q7_reduced,
    x_train_reduced_7,
    y_train_7,
    cv=cv,
    scoring=("r2", "neg_root_mean_squared_error"),
    return_train_score=False, 
    return_estimator = True
)

# save results 
cv_results_7.append({
    "model": f"LassoCV | {len(x_train_reduced_7.columns)} features",
    "R2_mean": cv_results_raw_7_reduced["test_r2"].mean(),
    "RMSE_mean": (-cv_results_raw_7_reduced["test_neg_root_mean_squared_error"]).mean()}
)

# create dataframe 
compare_models_q7_df = pd.DataFrame(cv_results_7).sort_values(by = "RMSE_mean").reset_index(drop = True)

# check results 
compare_models_q7_df

In [ ]:
print("Model performance with all features:\n")
# check the model on test data 
score_model(pipeline_q7, 
            x_train_7, 
            x_test_7, 
            y_train_7, 
            y_test_7)

In [ ]:
print("Model performance with the most important features:\n")
# check on test data again 
score_model(pipeline_q7_reduced, 
            x_train_reduced_7, 
            x_test_reduced_7, 
            y_train_7, 
            y_test_7)

In [ ]:
# final model standardized weights
# extract feature weights from the model
final_model_q7 = pipeline_q7_reduced.named_steps["model"]

# feature names
feature_names_final = x_train_reduced_7.columns.to_list()
# coefficients
coef_final = final_model_q7.coef_

# build dataframe
weights_df_7_final = pd.DataFrame({
    "tag": feature_names_final,
    "weight": coef_final
})

# sort by absolute weight
weights_df_7_final["abs_weight"] = weights_df_7_final["weight"].abs()
weights_df_7_final = weights_df_7_final.sort_values("abs_weight", ascending=False)

# check 
weights_df_7_final

The reduced Lasso model consistently outperforms the full-feature baseline across all evaluation settings. In cross-validation, RMSE decreases from ~0.584 to ~0.57 and R² increases from ~0.601 to ~0.619. 

The same pattern holds on the hold-out set. Test RMSE drops from ~0.584 to ~0.569, while test R² increases from ~0.616 to ~0.635. Training and testing metrics remain closely aligned, suggesting that the performance gain is not driven by overfitting.

These results indicate that several predictors in the full model contribute noise rather than signal. Removing them improves generalization and yields a simpler model with better predictive accuracy.

Overall, the reduced model achieves a better tradeoff between performance and complexity. It shows that ratings are primarily driven by teaching experience and course perception, while demographic and exposure-related variables add little once the core predictors are included.

### Extra: Feature Engineering
#### Do ratings change linearly with difficulty and volume?

To test for nonlinear relationships, we introduced polynomial and logarithmic transformations of selected predictors. Because these transformations induce multicollinearity, we employed ElasticNetCV, which combines L1 and L2 regularization to stabilize coefficient estimates while controlling overfitting.

In [ ]:
# transform average difficulty and number of ratings 
# average difficulty in a square 
x_train_7["average_difficulty^2"] = np.square(x_train_7["average_difficulty"])
x_test_7["average_difficulty^2"] = np.square(x_test_7["average_difficulty"])

# log number of ratings 
x_train_7["number_of_ratings_log"] = np.log(1 + x_train_7["number_of_ratings"])
x_test_7["number_of_ratings_log"] = np.log(1 + x_test_7["number_of_ratings"])
    
x_train_7["number_of_ratings_online_log"] = np.log(1 + x_train_7["number_of_ratings_online"])

x_test_7["number_of_ratings_online_log"] = np.log(1 + x_test_7["number_of_ratings_online"])

In [ ]:
# fixed cv
cv = KFold(n_splits = 5, 
           shuffle = True, 
           random_state = n_number)

# pipeline to test transformations
pipeline_q7_transform = Pipeline([
    # null values imputer 
    ("imputer", KNNImputer(n_neighbors= 20, 
                           weights = "uniform", 
                           add_indicator= False)),
    
    # data standardization (scaler)
    ("scaler", StandardScaler()), 

    # model 
    ("model", ElasticNetCV(
        cv = cv, 
        alphas = 100, 
        max_iter= 10000, 
        random_state = n_number
    ))
])

# run cross_validation
cv_results_transformation_raw_7 = cross_validate(
    pipeline_q7_transform, 
    x_train_7, 
    y_train_7, 
    cv = cv, 
    scoring = ("r2", "neg_root_mean_squared_error"), 
    return_train_score = False, 
    return_estimator = True
)

# save results 
cv_results_transformation_7 = {
    "R2_mean": cv_results_transformation_raw_7["test_r2"].mean(),
    "RMSE_mean": -cv_results_transformation_raw_7["test_neg_root_mean_squared_error"].mean()

}

# save results 
cv_results_7.append({
    "model": "ElasticNetCV (with transformations)",
    "R2_mean": cv_results_transformation_raw_7["test_r2"].mean(),
    "RMSE_mean": (-cv_results_transformation_raw_7["test_neg_root_mean_squared_error"]).mean()}
)

# create dataframe 
compare_models_q7_df = pd.DataFrame(cv_results_7).sort_values(by = "RMSE_mean").reset_index(drop = True)

# check results 
compare_models_q7_df



In [ ]:
# check on test data
print("Reduced LassoCV performance on test data:\n")
score_model(pipeline_q7_transform, 
            x_train_7, 
            x_test_7, 
            y_train_7, 
            y_test_7)

In [ ]:
# results of reduced model on test data 
print("ElasticNetCV with transformations performance on test data:\n")
score_model(pipeline_q7_reduced, 
            x_train_reduced_7, 
            x_test_reduced_7, 
            y_train_7, 
            y_test_7)

The ElasticNet model with nonlinear transformations does not meaningfully outperform the reduced Lasso model on either cross-validation or test data. Performance differences are within noise, suggesting that most predictive power is already captured by the reduced feature set.

Next, we examine coefficient magnitudes to identify and remove insignificant variables. We also test whether retaining only the squared difficulty term improves model parsimony and interpretability relative to the linear difficulty term.

In [ ]:
# extract feature weights from the model
model = pipeline_q7_transform.named_steps["model"]

# feature names
feature_names = x_train_7.columns.to_list()
# coefficients
coef = model.coef_

# build dataframe
weights_df_7 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_7["abs_weight"] = weights_df_7["weight"].abs()
weights_df_7 = weights_df_7.sort_values("abs_weight", ascending=False)

# cleanup
weights_df_7 = weights_df_7.drop(columns="abs_weight").reset_index(drop=True)

# check 
weights_df_7

The coefficient magnitudes from the ElasticNet model confirm the expected structure:
- `would_take_again` remains the dominant predictor.
- `average_difficulty²` has a strong negative coefficient, indicating a nonlinear relationship between difficulty and rating.
- The linear `average_difficulty` term is substantially weaker once the squared term is included.

Other variables have negligible coefficients and contribute little to predictive performance. Given the clear nonlinear effect of difficulty, we remove the `linear average_difficulty` term and retain only its squared transformation. This results in a parsimonious model with three primary predictors.

Next, we assess whether introducing the squared term materially increases multicollinearity. If multicollinearity remains controlled, we refit the model using LassoCV rather than ElasticNetCV, since Lasso provides comparable regularization with lower computational cost.

In [ ]:
# select features with |weight| >= 0.05
selected_features = ["would_take_again", 
                     "average_difficulty^2", 
                     "received_a_pepper"]

# reduce training data 
x_train_transformation_reduced_7 = x_train_7[selected_features]
x_test_transformation_reduced_7 = x_test_7[selected_features]


# correlations between predictors 
corr_pred_transformation_reduced_7 = x_train_transformation_reduced_7.corr()

# Plot the heatmap using Seaborn
plt.figure(figsize=(12, 10)) # Adjust the figure size as needed

# plot itself
sns.heatmap(corr_pred_transformation_reduced_7, annot=True, cmap='coolwarm', fmt=".2f")

# aesthetics 
# title 
plt.title('Correlation Matrix of Numerical Features', fontweight = "bold", fontsize = 18)

# ticks
plt.tick_params(labelsize = 16)

# show the plot
plt.show()


The correlation matrix for the reduced feature set shows no evidence of severe multicollinearity:
- Correlations are all well below common concern thresholds of |0.7-0.8|.
- The strongest relationship is between `would_take_again` and `average_difficulty²` at -0.53, which reflects a meaningful behavioral association rather than redundancy.
- `received_a_pepper` shows only moderate correlation with the other predictors.

These results indicate that introducing the squared difficulty term does not materially inflate multicollinearity once the linear difficulty term is removed.
Given this, regularization via ElasticNet is no longer strictly necessary, and we proceed by refitting the model using LassoCV for improved simplicity and computational efficiency.

In [ ]:
# cross-validation 
cv = KFold(n_splits=5, 
           shuffle=True, 
           random_state=n_number)

# pipeline for model 
pipeline_q7_transformation_reduced = Pipeline([
    ("imputer", KNNImputer(n_neighbors=20, 
                           weights= 'uniform', 
                           add_indicator= False)),
    ("scaler", StandardScaler()),
    ("model", LassoCV(
        cv = cv, 
        random_state = n_number,
        alphas = 100, 
        max_iter= 10000
    ))
])

# run cross-validation
cv_results_raw_7_transformation_reduced = cross_validate(
    pipeline_q7_transformation_reduced,
    x_train_transformation_reduced_7,
    y_train_7,
    cv=cv,
    scoring=("r2", "neg_root_mean_squared_error"),
    return_train_score=False, 
    return_estimator = True
)

# save results 
cv_results_7.append({
    "model": "LassoCV | transformation + 3 features",
    "R2_mean": cv_results_raw_7_transformation_reduced["test_r2"].mean(),
    "RMSE_mean": (-cv_results_raw_7_transformation_reduced["test_neg_root_mean_squared_error"]).mean()
    })

# show as a dataframe
compare_models_q7_df = pd.DataFrame(cv_results_7).sort_values(by = "RMSE_mean").reset_index(drop = True)
compare_models_q7_df


In [ ]:
# LassoCV (reduced features) performance on the test data
print("LassoCV (reduced features) performance on the test data\n")
score_model(pipeline_q7_reduced, 
            x_train_reduced_7, 
            x_test_reduced_7, 
            y_train_7,
            y_test_7)

In [ ]:
# LassoCV (transformation + reduced features) performance on the test data
print("LassoCV (transformation + reduced features) performance on the test data:\n")
score_model(pipeline_q7_transformation_reduced, 
            x_train_transformation_reduced_7, 
            x_test_transformation_reduced_7, 
            y_train_7, 
            y_test_7)

In [ ]:
# final model weights 
# extract feature weights from the model
model_poly = pipeline_q7_transformation_reduced.named_steps["model"]

# feature names
feature_names_poly = x_train_transformation_reduced_7.columns.to_list()
# coefficients
coef_poly = model_poly.coef_

# build dataframe
weights_df_7_poly = pd.DataFrame({
    "tag": feature_names_poly,
    "weight": coef_poly
})

# sort by absolute weight
weights_df_7_poly["abs_weight"] = weights_df_7_poly["weight"].abs()
weights_df_7_poly = weights_df_7_poly.sort_values("abs_weight", ascending=False)

# check 
weights_df_7_poly

Cross-validation results show that introducing nonlinear transformations improves model performance. The best cross-validated specification, LassoCV with transformations and reduced features, achieves an R² of approximately ~0.622 and an RMSE of about ~0.568, outperforming both the linear reduced model with an RMSE of roughly ~0.570 and the full linear model with an RMSE near ~0.584. ElasticNetCV produces nearly identical cross-validation metrics, indicating that multicollinearity is adequately controlled and that Lasso regularization is sufficient.

Evaluation on the held-out test set confirms this pattern. The reduced Lasso model attains a testing RMSE of about ~0.567 and an R² of approximately ~0.635, while the transformed reduced Lasso model further improves performance with a testing RMSE of roughly ~0.561 and an R² near 0.644. These results motivate a final assessment of interaction effects to determine whether any additional structure remains unexploited.

#### Interactions

In [ ]:
# create interaction variables 
# would take gain and difficulty 
x_train_7["wta_x_difficulty"] = (
x_train_7["would_take_again"] * x_train_7["average_difficulty"]
)

x_test_7["wta_x_difficulty"] = (
x_test_7["would_take_again"] * x_test_7["average_difficulty"]
)

x_train_7["wta_x_difficulty2"] = (
x_train_7["would_take_again"] * x_train_7["average_difficulty^2"]
)

x_test_7["wta_x_difficulty2"] = (
x_test_7["would_take_again"] * x_test_7["average_difficulty^2"]
)

# pepper and would take again 
x_train_7["pepper_x_wta"] = (
x_train_7["received_a_pepper"] * x_train_7["would_take_again"]
)

x_test_7["pepper_x_wta"] = (
x_test_7["received_a_pepper"] * x_test_7["would_take_again"]
)

# would take again and log_number of ratings 
x_train_7["wta_x_log_number_of_ratings"] = (x_train_7["would_take_again"] * x_train_7["number_of_ratings_online_log"])
x_train_7["wta_x_log_number_of_ratings"] = (x_train_7["would_take_again"] * x_train_7["number_of_ratings_online_log"])

x_train_7["wta_x_log_number_of_ratings_online"] = (x_train_7["would_take_again"] * x_train_7["number_of_ratings_online_log"])
x_train_7["wta_x_log_number_of_ratings_online"] = (x_train_7["would_take_again"] * x_train_7["number_of_ratings_online_log"])

x_test_7["wta_x_log_number_of_ratings"] = (x_test_7["would_take_again"] * x_test_7["number_of_ratings_online_log"])
x_test_7["wta_x_log_number_of_ratings"] = (x_test_7["would_take_again"] * x_test_7["number_of_ratings_online_log"])

x_test_7["wta_x_log_number_of_ratings_online"] = (x_test_7["would_take_again"] * x_test_7["number_of_ratings_online_log"])
x_test_7["wta_x_log_number_of_ratings_online"] = (x_test_7["would_take_again"] * x_test_7["number_of_ratings_online_log"])


In [ ]:
# fixed cv
cv = KFold(n_splits = 5, 
           shuffle = True, 
           random_state = n_number)

# pipeline to test interactions
pipeline_q7_interactions = Pipeline([
    # null values imputer 
    ("imputer", KNNImputer(n_neighbors= 20, 
                           weights = "uniform", 
                           add_indicator= False)),
    
    # data standardization (scaler)
    ("scaler", StandardScaler()), 

    # model 
    ("model", ElasticNetCV(
        cv = cv, 
        alphas = 100, 
        max_iter= 10000, 
        random_state = n_number
    ))
])

# run cross_validation
cv_results_interactions_raw_7 = cross_validate(
    pipeline_q7_interactions, 
    x_train_7, 
    y_train_7, 
    cv = cv, 
    scoring = ("r2", "neg_root_mean_squared_error"), 
    return_train_score = False, 
    return_estimator = True
)

# save results 
cv_results_7.append({
    "model": "ElasticNet CV (interactions)",
    "R2_mean": cv_results_interactions_raw_7["test_r2"].mean(),
    "RMSE_mean": (-cv_results_interactions_raw_7["test_neg_root_mean_squared_error"]).mean()}
)

# create dataframe 
compare_models_q7_df = pd.DataFrame(cv_results_7).sort_values(by = "RMSE_mean").reset_index(drop = True)

# check results 
compare_models_q7_df




In [ ]:
# ElasticNet CV (interactions) performance on the test data:
print("ElasticNet CV (interactions) performance on the test data:")
score_model(pipeline_q7_interactions, 
            x_train_7, 
            x_test_7, 
            y_train_7, 
            y_test_7)

In [ ]:
# Lasso CV (transformation + reduced variables) performance on the test data:
print("Lasso CV (transformation + reduced variables) performance on the test data:")
score_model(pipeline_q7_transformation_reduced,
            x_train_transformation_reduced_7,
            x_test_transformation_reduced_7,
            y_train_7, 
            y_test_7)

Cross-validation results show that the full interaction-based Elastic Net model performs no better than a much simpler Lasso model with nonlinear transformations and only three features. Both achieve nearly identical cross-validated RMSE (~ 0.568) and R² (~ 0.622), despite the interaction model being substantially more complex.

This is already a warning signal. When a full interaction model fails to outperform a reduced transformed model during cross - validation, it suggests that interaction terms are not introducing new signal, but instead re-expressing information already captured by simpler nonlinear transformations.

This conclusion is reinforced on the test set. The interaction-based Elastic Net model generalizes worse, with a higher testing RMSE (~ 0.566) and lower R² (~ 0.638), compared to the transformed reduced Lasso model, which achieves a lower testing RMSE (~ 0.561) and higher R² (~ 0.644).

Importantly, the interaction-based model increases complexity along several dimensions simultaneously:
- Explicit interaction terms inflate the feature space
- Elastic Net regularization adds tuning complexity
- The model retains more predictors than the reduced alternative
- Multicollinearity is reintroduced by construction

Yet this added complexity yields neither improved cross-validated performance nor better generalization.

In [ ]:
# extract feature weights from the model
model = pipeline_q7_interactions.named_steps["model"]

# feature names
feature_names = x_train_7.columns.to_list()
# coefficients
coef = model.coef_

# build dataframe
weights_df_7 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_7["abs_weight"] = weights_df_7["weight"].abs()
weights_df_7 = weights_df_7.sort_values("abs_weight", ascending=False)

# check 
weights_df_7

In [ ]:
# cv
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# define threshold
initial_threshold = [0.05]
elasticnetcv_q7_comparisons_interactions = []

# filter-out variables 
keep_features_interactions = weights_df_7[
    weights_df_7["abs_weight"] >= 0.05
].sort_values("abs_weight", ascending=False)["tag"].tolist()

# create train and test samples 
x_train_reduced_7_interactions = x_train_7[keep_features_interactions]

x_test_reduced_10_interactions = x_train_7[keep_features_interactions]

# loop to drop each variable 1 by one 
for k in range(len(keep_features_interactions), 0, -1):
    keep_pol_interactions = keep_features_interactions[:k]

    x_train_reduced_7_interactions = x_train_reduced_7_interactions[keep_pol_interactions]

    # pipeline to test interactions
    pipeline = Pipeline([
    # null values imputer 
    ("imputer", KNNImputer(n_neighbors= 20, 
                           weights = "uniform", 
                           add_indicator= False)),
    
    # data standardization (scaler)
    ("scaler", StandardScaler()), 

    # model 
    ("model", ElasticNetCV(
        cv = cv, 
        alphas = 100, 
        max_iter= 10000, 
        random_state = n_number
    ))
])
    
    results_reduced_7_interactions = cross_validate(
        pipeline,
        x_train_reduced_7_interactions,
        y_train_7,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

    elasticnetcv_q7_comparisons_interactions.append({
    "model": f"ElasticNetCV | top-{k} by |w|",
    "R2_mean": results_reduced_7_interactions["test_r2"].mean(),
    "RMSE_mean": (-results_reduced_7_interactions["test_neg_root_mean_squared_error"]).mean()}
)
    
# dataframe 
drop_df_7_interactions = (
            pd.DataFrame(elasticnetcv_q7_comparisons_interactions)
            .sort_values("RMSE_mean", ascending=False)
            .reset_index(drop=True))
drop_df_7_interactions

In [ ]:
# best models dataframe
compare_models_q7_df


No alternative specification provides a meaningful improvement over the transformed Lasso model with three features. All competing models either show equivalent cross-validated performance within noise or introduce additional complexity without improving generalization. As a result, this model is selected as the final specification under a balanced objective that prioritizes both predictive performance and parsimony.

If the modeling objective were instead shifted toward pure performance maximization, interpretability and sparsity would no longer be binding constraints. In that case, more flexible nonlinear models such as gradient-boosted trees could be considered to assess the upper bound of achievable performance. However, under the current objective, the transformed three-feature Lasso model represents the most appropriate and defensible choice.

### Extra: Prediction Accuracy Optimization

##### LightGBM

We first evaluate whether LightGBM benefits from our tuned KNN imputation compared to simpler missing-value strategies. Specifically, we compare four preprocessing options, tuned KNN imputation, mean imputation, median imputation, and no imputation. For each option, we measure cross-validated RMSE and R² using the same 5-fold split to ensure a fair comparison.

After selecting the imputation strategy that yields the best cross-validated RMSE, we fix this preprocessing choice and tune the LightGBM hyperparameters with Optuna. This two-stage approach separates preprocessing selection from model tuning and ensures that hyperparameter optimization is performed on the strongest preprocessing setup.

In [ ]:
# fixed cross-validation 
cv = KFold(n_splits = 5, 
           shuffle = True, 
           random_state = n_number)

# pipelines for comparison
pipelines_q7_lgbm = {
    "LGBMRegressor (no imputation)": Pipeline([
        # scaler 
        ("scaler", StandardScaler()), 

        # model 
        ("model", LGBMRegressor(random_state = n_number, 
                                objective = "regression", 
                                metric = "rmse",
                                n_jobs = 1, 
                                verbosity = -1))
    ]), 

    "LGBMRegreessor (KNNImputer)": Pipeline([
        # imputer
        ("imputer", KNNImputer(n_neighbors=20, 
                               weights="uniform", 
                               add_indicator=False)),

        # scaler 
        ("scaler", StandardScaler()),   

        # model 
        ("model", LGBMRegressor(random_state = n_number, 
                                objective = "regression", 
                                metric = "rmse",
                                n_jobs = 1, 
                                verbosity = -1)) 
    ]),

    "LGBMRegressor (SimpleImputer - mean)": Pipeline([
        # imputer
        ("imputer", SimpleImputer(strategy="mean", 
                                  add_indicator=False)),

        # scaler 
        ("scaler", StandardScaler()),   

        # model 
        ("model", LGBMRegressor(random_state = n_number, 
                                objective = "regression", 
                                metric = "rmse",
                                n_jobs = 1, 
                                verbosity = -1)) 
    ]), 

    "LGBMRegressor (SimpleImputer - median)": Pipeline([
        # imputer
        ("imputer", SimpleImputer(strategy="median", 
                                  add_indicator=False)),

        # scaler 
        ("scaler", StandardScaler()),   

        # model 
        ("model", LGBMRegressor(random_state = n_number, 
                                objective = "regression", 
                                metric = "rmse",
                                n_jobs = 1, 
                                verbosity = -1)) 
    ]),
}

# compare and cross_validate the pipelines 
rows_lgbm = []
for name, pipe in pipelines_q7_lgbm.items():
    results_lgbm = cross_validate(
        pipe,
        x_train_7,
        y_train_7,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        return_train_score=False,
        n_jobs=-1
    )
    rows_lgbm.append({
        "model": name,
        "R2_mean": results_lgbm["test_r2"].mean(),
        "RMSE_mean": (-results_lgbm["test_neg_root_mean_squared_error"]).mean()
    })

# create a dataframe for comparison
compare_lgbm_df = (
    pd.DataFrame(rows_lgbm)
      .sort_values("RMSE_mean", 
                   ascending= True)
      .reset_index(drop=True)
)

# check the results
compare_lgbm_df

Cross-validation results indicate that LightGBM performs best without explicit imputation. Models using no imputation achieve an RMSE of approximately ~0.557 with an R² around ~0.636, which is statistically indistinguishable from mean and median imputation variants (RMSE  ~ 0.556 - 0.557).

While mean and median imputation yield marginally lower RMSE values, the differences are negligible and fall well within cross-validation noise. In contrast, KNN imputation leads to a noticeable degradation in performance, increasing RMSE to approximately ~0.563.

Because LightGBM natively handles missing values by learning optimal split directions, explicit imputation does not introduce additional signal and may remove informative missingness patterns. Given the lack of meaningful performance gains and the added complexity introduced by imputation, we proceed without explicit imputation for LightGBM.

This choice preserves model simplicity while maintaining optimal predictive performance.

In [ ]:
# fixed cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline for LightGBM 
lgbm_pipe_q7 = Pipeline([
    # scaler 
    ("scaler", StandardScaler()),
    
    # model itself 
    ("model", LGBMRegressor(
        objective = "regression", 
        metric = "rmse", 
        random_state=n_number,
        n_jobs=1,
        verbosity=-1
    ))
])

# optuna objective
def objective_lgbm(trial):
    # parameters to tune
    params = {
        # core tree shape
        "model__num_leaves": trial.suggest_int("model__num_leaves", 15, 255),
        "model__max_depth": trial.suggest_int("model__max_depth", -1, 16),
        "model__min_child_samples": trial.suggest_int("model__min_child_samples", 5, 200),

        # learning
        "model__n_estimators": trial.suggest_int("model__n_estimators", 200, 3000),
        "model__learning_rate": trial.suggest_float("model__learning_rate", 0.005, 0.2, log=True),

        # subsampling
        "model__subsample": trial.suggest_float("model__subsample", 0.6, 1.0),
        "model__colsample_bytree": trial.suggest_float("model__colsample_bytree", 0.6, 1.0),

        # regularization
        "model__reg_alpha": trial.suggest_float("model__reg_alpha", 1e-8, 10.0, log=True),
        "model__reg_lambda": trial.suggest_float("model__reg_lambda", 1e-8, 10.0, log=True),

        # split control
        "model__min_split_gain": trial.suggest_float("model__min_split_gain", 0.0, 0.5),
    }

    # clone pipeline and set parameters
    pipe = clone(lgbm_pipe_q7)
    pipe.set_params(**params)

    # cross-validation score with tuned parameters
    scores = cross_val_score(
        pipe,
        x_train_7,
        y_train_7,
        cv=cv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )

    return -scores.mean()

# run optuna
study_lgbm = start_study_optuna(
    objective=objective_lgbm,
    n_trials=50,    # number of trials 
    sampler_seed=n_number,
    direction="minimize"
)

In [ ]:
# cross-validate the model with best parameters
# set the best parameters to the pipeline
lgbm_pipe_q7.set_params(**study_lgbm.best_params)

cv_results_raw_lgbm = cross_validate(
    lgbm_pipe_q7,
    x_train_7,
    y_train_7,
    cv=cv,
    scoring=("r2", "neg_root_mean_squared_error"),
    return_train_score=False, 
    return_estimator = True
)

# results
cv_results_7.append({
    "model": "LightGBM (no imputation)",
    "R2_mean": cv_results_raw_lgbm["test_r2"].mean(),
    "RMSE_mean": -cv_results_raw_lgbm["test_neg_root_mean_squared_error"].mean()
})

# add to comparison dataframe 
compare_models_q7_df = pd.DataFrame(cv_results_7).sort_values(by = "RMSE_mean").reset_index(drop = True)

# check results 
compare_models_q7_df


In [ ]:
# performance of LightGBM on test data:
print("LightGBM model performance on test data:\n")
score_model(
    lgbm_pipe_q7,
    x_train_7,
    x_test_7,
    y_train_7,
    y_test_7
)

In [ ]:
# performance of LassoCV (transformation + reduced variables) on test data:
print("performance of LassoCV (transformation + reduced variables) on test data:\n")
score_model(
    pipeline_q7_transformation_reduced,
    x_train_transformation_reduced_7,
    x_test_transformation_reduced_7,
    y_train_7,
    y_test_7
)

Cross-validation results show a clear and non-marginal improvement from LightGBM relative to all linear and regularized models. The LightGBM model without imputation achieves a CV RMSE of approximately ~0.554, improving substantially over the best linear specification, whose CV RMSE is around ~0.568. This gain carries over to the test set, where LightGBM attains a testing RMSE of about ~0.552 and an R² of roughly ~0.656, compared to a testing RMSE of approximately ~0.561 and an R² near ~0.644 for the transformed reduced Lasso model. Unlike earlier stages, this improvement is large enough to be considered meaningful and cannot be attributed to noise or minor functional-form adjustments.

Given this clear performance gap, we conclude that LightGBM is able to capture nonlinearities and higher-order interactions that linear models fail to exploit, even after extensive feature engineering. Before proceeding to XGBoost for further performance maximization, we first examine LightGBM feature importance to understand which variables drive these gains and whether the model relies on sensible, interpretable structure rather than spurious interactions.

In [ ]:
# shapley values
explainer = shap.TreeExplainer(lgbm_pipe_q7.named_steps["model"])
shap_values = explainer.shap_values(x_train_7)   

# plot 
shap.summary_plot(shap_values, x_train_7)

`average_difficulty` is the single strongest driver of predictions. Low difficulty values consistently increase predicted ratings, while high difficulty values decrease them. The wide spread of SHAP values shows a strong nonlinear effect, which LightGBM captures directly through splits rather than relying on engineered polynomial terms.

`would_take_again` is the second most influential feature and has a strong, monotonic positive impact. Higher values reliably push predictions upward across almost all observations, confirming that student willingness to retake a professor is the most stable sentiment signal in the data.

`received_a_pepper`  has a positive but smaller effect. Its SHAP values are more dispersed, indicating that the pepper signal matters, but only conditionally and less consistently than the two dominant variables.

All remaining features, including engineered interactions, squared terms, counts, and demographic indicators, have SHAP values concentrated near zero. They contribute little to predictions once difficulty and willingness to retake are accounted for.

In short, LightGBM’s performance gains are driven almost entirely by three variables: average difficulty, willingness to take the professor again, and pepper status. Everything else is secondary or redundant. Now we will assess Xgboost. 

#### Xgboost 

We now again are going to evaluate Whether Xgboost benefits from our tuned KNN imputation compared to simpler missing-value strategies. As with Lightgbm, we compare four preprocessing options, tuned KNN imputation, mean imputation, median imputation, and no imputation. For each option, we measure cross-validated RMSE and R² using the same 5-fold split to ensure a fair comparison.

After selecting the imputation strategy that yields the best cross-validated RMSE, we fix this preprocessing choice and tune the Xgboost hyperparameters with Optuna. 

In [ ]:
# fixed cross-validation 
cv = KFold(n_splits = 5, 
           shuffle = True, 
           random_state = n_number)

# pipelines for comparison
pipelines_q7_xgboost = {
    "XGBRegressor (no imputation)": Pipeline([
        # model 
        ("model", XGBRegressor(
            objective="reg:squarederror",
            eval_metric="rmse",
            tree_method="hist",
            random_state=n_number,
            verbosity=0,
            n_jobs=1
        ))
    ]), 

    "XGBRegressor (KNNImputer)": Pipeline([
        # imputer
        ("imputer", KNNImputer(
            n_neighbors=20, 
            weights="uniform", 
            add_indicator=False
        )),

        # model 
        ("model", XGBRegressor(
            objective="reg:squarederror",
            eval_metric="rmse",
            tree_method="hist",
            random_state=n_number,
            verbosity = 0,
            n_jobs=1
        )) 
    ]),

    "XGBRegressor (SimpleImputer - mean)": Pipeline([
        # imputer
        ("imputer", SimpleImputer(
            strategy="mean", 
            add_indicator=False
        )),

        # model 
        ("model", XGBRegressor(
            objective="reg:squarederror",
            eval_metric="rmse",
            tree_method="hist",
            random_state=n_number,
            verbosity = 0,
            n_jobs=1
        )) 
    ]), 

    "XGBRegressor (SimpleImputer - median)": Pipeline([
        # imputer
        ("imputer", SimpleImputer(
            strategy="median", 
            add_indicator=False
        )),

        # model 
        ("model", XGBRegressor(
            objective="reg:squarederror",
            eval_metric="rmse",
            tree_method="hist",
            random_state=n_number,
            verbosity = 0,
            n_jobs=1
        )) 
    ]),
}

# compare and cross_validate the pipelines 
rows_xgboost = []
for name, pipe in pipelines_q7_xgboost.items():
    results_xgboost = cross_validate(
        pipe,
        x_train_7,
        y_train_7,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        return_train_score=False,
        n_jobs=-1
    )
    rows_xgboost.append({
        "model": name,
        "R2_mean": results_xgboost["test_r2"].mean(),
        "RMSE_mean": (-results_xgboost["test_neg_root_mean_squared_error"]).mean()
    })

# create a dataframe for comparison
compare_xgboost_df = (
    pd.DataFrame(rows_xgboost)
      .sort_values("RMSE_mean", ascending=True)
      .reset_index(drop=True)
)

# check the results
compare_xgboost_df


The best XGBoost specification without imputation achieves a CV RMSE of approximately ~0.567 and an R² of about ~0.623, which is comparable to but slightly worse than LightGBM's CV RMSE of roughly ~0.554. Mean and median imputation lead to nearly identical results, while KNN imputation substantially degrades performance, increasing RMSE to approximately 0.~579.

These results suggest that, similar to LightGBM, XGBoost does not benefit from explicit imputation and may leverage missingness implicitly. 

In [ ]:
# fixed cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline for XGBoost
xgb_pipe = Pipeline([
    # scaler 
    ("scaler", StandardScaler()), 

    # model itself 
    ("model", XGBRegressor(
        objective="reg:squarederror",
        random_state=n_number,
        n_jobs=1,              
        tree_method="hist",
        eval_metric="rmse"
    ))
])

# optuna objective
def objective_xgb(trial):
    params = {
        # core tree shape
        "model__max_depth": trial.suggest_int("model__max_depth", 2, 12),
        "model__min_child_weight": trial.suggest_float("model__min_child_weight", 1.0, 50.0, log=True),
        "model__gamma": trial.suggest_float("model__gamma", 0.0, 5.0),

        # learning
        "model__n_estimators": trial.suggest_int("model__n_estimators", 200, 3000),
        "model__learning_rate": trial.suggest_float("model__learning_rate", 0.005, 0.2, log=True),

        # subsampling
        "model__subsample": trial.suggest_float("model__subsample", 0.6, 1.0),
        "model__colsample_bytree": trial.suggest_float("model__colsample_bytree", 0.6, 1.0),
        "model__colsample_bylevel": trial.suggest_float("model__colsample_bylevel", 0.6, 1.0),

        # regularization
        "model__reg_alpha": trial.suggest_float("model__reg_alpha", 1e-8, 10.0, log=True),
        "model__reg_lambda": trial.suggest_float("model__reg_lambda", 1e-8, 50.0, log=True),

        # split control
        "model__max_delta_step": trial.suggest_int("model__max_delta_step", 0, 10),
    }

    pipe = clone(xgb_pipe)
    pipe.set_params(**params)

    scores = cross_val_score(
        pipe,
        x_train_7,
        y_train_7,
        cv=cv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )

    return -scores.mean()

# run optuna
study_xgb = start_study_optuna(
    objective=objective_xgb,
    n_trials=50,
    sampler_seed=n_number,
    direction="minimize"
)

# after tuning, set best params and cross-validate for R2 + RMSE
xgb_pipe.set_params(**study_xgb.best_params)

# raw results 
cv_results_raw_xgb = cross_validate(
    xgb_pipe,
    x_train_7,
    y_train_7,
    cv=cv,
    scoring=("r2", "neg_root_mean_squared_error"),
    return_train_score=False,
    return_estimator=True
)

# append the results 
cv_results_7.append({
    "model": "XGBoost",
    "R2_mean": cv_results_raw_xgb["test_r2"].mean(),
    "RMSE_mean": -cv_results_raw_xgb["test_neg_root_mean_squared_error"].mean()
})


# show and save as pdf 
compare_models_q7_df = (pd.DataFrame(cv_results_7).sort_values("RMSE_mean").reset_index(drop=True)
)
compare_models_q7_df


Based on cross-validation performance alone, there is no justification to pursue XGBoost further. Although XGBoost marginally outperforms LightGBM in R², the difference in RMSE is negligible, and both models fall well within the same performance range. Given the additional tuning complexity and reduced interpretability of XGBoost, further analysis of its test-set performance or feature importance is unwarranted.

Among nonlinear models, LightGBM offers the best balance of predictive accuracy and practical interpretability, achieving strong performance while still allowing partial explanation through feature importance and SHAP values. When performance is the sole objective, LightGBM is therefore the preferred model.

From a perspective that balances explainability and performance, the LassoCV model with nonlinear transformations and a reduced feature set remains the most appropriate choice. It delivers competitive predictive accuracy relative to other linear specifications while maintaining transparent and stable coefficient interpretations.

## Q8 - Build a regression model predicting average ratings from all tags (the ones in the rmpCapstoneTags.csv) file. Make sure to include the R2 and RMSE of this model. Which of these tags is most strongly predictive of average rating?



In [ ]:
# copy the data for the question
df_8 = df_filtered_final[df_tags_column_names + ["average_rating"]].copy()

In [ ]:
# information about the dataset
df_8.info()

In [ ]:
# split the data into x and y 
# x 
x_8 = df_8.drop(columns = "average_rating")

# y 
y_8 = df_8["average_rating"]

# split the data into train and test data 
x_train_8, x_test_8, y_train_8, y_test_8 = train_test_split(x_8, y_8, test_size=0.2, random_state=n_number)

# check distributions of the data 
print(f'Number of rows for train data:{y_train_8.shape[0]}\n')
print(f'Number of rows for test data:{y_test_8.shape[0]}\n')

In [ ]:
# correlation of predictors with the target 
corr_8_y = x_train_8.corrwith(y_train_8, 
                              method = 'pearson').sort_values(ascending= False)

# check 
corr_8_y

These correlations reveal a strong and structured relationship between course characteristics and average ratings. On the positive side, teaching quality and instructor behavior dominate. Being respected shows the highest positive correlation with ratings (0.46), followed closely by caring (0.43) and good feedback (0.42). Engaging delivery also matters. Amazing lectures (0.38), inspirational teaching (0.36), and being humorous (0.30) all show moderate positive associations. Clear grading (0.22) and accessibility (0.14) contribute positively but with smaller magnitudes, while extra credit has only a minor association (0.10).

Negative correlations are even more pronounced and largely reflect course rigor and grading strictness. Tough grading exhibits the strongest relationship in absolute value (-0.67), making it the single most influential correlate in the list. Lecture-heavy formats (-0.37), lots of homework (-0.34), lots to read (-0.30), and strict attendance or assessment rules such as dont skip class (-0.28) and test heavy (-0.27) are all associated with substantially lower ratings. Smaller but still negative effects appear for so many papers (-0.23), graded by few things (-0.17), pop quizzes (-0.14), and group projects (-0.13).

At the same time, these magnitudes suggest substantial overlap across predictors. Many workload-related tags cluster tightly in the range from about -0.27 to -0.37, while instructor-quality tags cluster between roughly 0.30 and 0.46. This raises concerns that individual coefficients in a multivariate model may reflect shared signal rather than distinct effects.

For this reason, the next step focuses on assessing multicollinearity among these variables, to determine how much redundancy exists and how it may affect coefficient stability and interpretation.

In [ ]:
# plot the heatmap of correlations between predictors 
# figure size 
plt.figure(figsize = (12, 10))

# plot itself 
sns.heatmap(x_train_8.corr(), 
            annot = True, 
            cmap = "coolwarm", 
            fmt = ".2f")

# aesthetics 
# title 
plt.title("Correlation of Tags", 
          fontweight = "bold", 
          fontsize = 18)

# ticks
plt.tick_params(labelsize = 16)

# show the plot
plt.show()


The correlation matrix shows limited multicollinearity among the tag variables. Most pairwise correlations fall between -0.30 and 0.30, well below common concern thresholds. The strongest relationships appear where expected but remain moderate. For example, tough grader correlates negatively with caring (-0.37) and respected (-0.35), while respected correlates positively with caring (0.22) and inspirational (0.27). Workload-related tags such as lots of homework, lots to read, lecture heavy, and test heavy show correlations mostly in the -0.20 to -0.30 range.

There are no clusters with correlations approaching 0.7 or higher. While several tags capture related aspects of teaching quality or course rigor, the overlap is not strong enough to raise serious multicollinearity concerns.

Given this, we proceed to fit several regression models using the tag variables jointly, with coefficient estimates expected to remain stable and interpretable.

### Linear Regression

In [ ]:
# fixed CV
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipelines for linear model comparison (no imputation)
pipelines_linear = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),

    "RidgeCV": Pipeline([
        ("scaler", StandardScaler()),
        ("model", RidgeCV(
            alphas=np.logspace(-4, 4, 50),
            cv=cv
        ))
    ]),

    "LassoCV": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LassoCV(
            alphas=100,
            cv=cv,
            random_state=n_number,
            max_iter=10000
        ))
    ]),

    "ElasticNetCV": Pipeline([
        ("scaler", StandardScaler()),
        ("model", ElasticNetCV(
            l1_ratio=[0.1, 0.5, 0.9],
            alphas=100,
            cv=cv,
            random_state=n_number,
            max_iter=10000
        ))
    ])
}

# compare models
rows = []
for name, pipe in pipelines_linear.items():
    results = cross_validate(
        pipe,
        x_train_8,
        y_train_8,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

    rows.append({
        "model": name,
        "R2_mean": results["test_r2"].mean(),
        "RMSE_mean": (-results["test_neg_root_mean_squared_error"]).mean()
    })

compare_models_q8_df = (
    pd.DataFrame(rows)
      .sort_values("RMSE_mean", ascending=True)
      .reset_index(drop=True)
)

# check results 
compare_models_q8_df

In [ ]:
# check models on test data
for name, pipe in pipelines_linear.items():
    print(f"\nModel: {name}")
    print(score_model(
        pipe,
        x_train_8,
        x_test_8,
        y_train_8,
        y_test_8
    ))


We fitted several linear models using the tag variables, including ordinary least squares, Ridge, Lasso, and Elastic Net, under an identical cross-validation and preprocessing setup. All four models deliver virtually identical performance. Cross-validated R² is approximately 0.708 across all models, and RMSE is tightly clustered around 0.50. Differences in error are extremely small, on the order of 0.0003 to 0.0004, indicating that regularization does not materially affect predictive accuracy in this setting.

Hold-out test results confirm the same pattern. Training and testing metrics are closely aligned for all models, with test RMSE around 0.504 and test R² around 0.713. This consistency indicates stable generalization and reinforces the conclusion that plain linear regression already captures most of the available signal.

These findings are consistent with the earlier multicollinearity analysis. Since the tag variables exhibit only moderate pairwise correlations, coefficient instability is limited, and shrinkage provides little benefit in terms of prediction. Ridge and Elastic Net therefore offer no practical advantage over ordinary least squares.

Despite this, Lasso remains a reasonable choice for the final linear model. Although its predictive performance is nearly indistinguishable from the other models, Lasso introduces coefficient shrinkage and implicit feature selection. This improves interpretability and provides a safeguard against overfitting if additional or noisier predictors are introduced.

We therefore proceed with the Lasso model and examine its estimated coefficients. The next step is to assess whether removing weak predictors identified by Lasso improves predictive performance or simply yields a more parsimonious model with comparable accuracy.

### Lasso Regression

In [ ]:
# fixed CV 
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline for Lasso Regression model 
pipeline_q8 = Pipeline([
    # scaler 
    ("scaler", StandardScaler()), 

    # model 
    ("model", LassoCV(
        cv=cv,
        random_state=n_number,
        alphas=100,
        max_iter=10000
    ))
])

# fit 
pipeline_q8.fit(x_train_8, 
                y_train_8)

# extract feature weights from the model
model = pipeline_q8.named_steps["model"]

# feature names
feature_names = x_train_8.columns.to_list()
# coefficients
coef = model.coef_

# build dataframe
weights_df_8 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_8["abs_weight"] = weights_df_8["weight"].abs()
weights_df_8 = weights_df_8.sort_values("abs_weight", ascending=False).reset_index(drop = True)

# check 
weights_df_8

In [ ]:
# plot weights of the multiple linear regression model 
# figure size 
plt.figure(figsize = (12, 10))

# bar plot 
sns.barplot(x=weights_df_8["weight"],
        y = weights_df_8["tag"])

# aesthetics
# title 
plt.title("Lasso Regression Coefficients for Predicting Average Rating", 
          fontweight="bold", 
          fontsize=18)

# axis labels
plt.xlabel("Predictor", fontsize=16)
plt.ylabel("Coefficient value", fontsize=16)

# ticks
plt.tick_params(axis="both", labelsize=14)

# show plot
plt.show()

We examined the standardized coefficients from the Lasso model to assess the relative importance of each numerical predictor. As expected based on the earlier correlation analysis, variables related to perceived teaching quality and classroom experience carry the largest coefficients, while many workload and structural indicators have coefficients close to zero once stronger predictors are included. This confirms that much of the predictive signal is concentrated in a relatively small subset of tags.

Rather than treating a single cutoff as definitive, we use the coefficient magnitudes as a diagnostic tool to guide systematic feature reduction. Predictors with very small absolute coefficients contribute little marginal information after accounting for collinearity and shared variance among tags. To balance interpretability and performance, we therefore evaluate multiple coefficient thresholds and refit the model under each reduced subset. By comparing cross-validated R² and RMSE across these thresholds, we assess how aggressively the feature set can be reduced before predictive performance begins to deteriorate. This approach allows us to identify a parsimonious model that retains the dominant explanatory factors without sacrificing accuracy.

In [ ]:
# cv
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# define threshold
initial_threshold = [0.05]
lassocv_q8_comparisons = []

# filter-out variables 
keep_features_q8 = weights_df_8[
    weights_df_8["abs_weight"] >= 0.05
].sort_values("abs_weight", ascending=False)["tag"].tolist()

# create train and test samples 
x_train_reduced_8 = x_train_8[keep_features_q8]

x_test_reduced_8 = x_test_8[keep_features_q8]

# loop to drop each variable 1 by one 
for k in range(len(keep_features_q8), 0, -1):
    keep_pol_q8 = keep_features_q8[:k]

    x_train_reduced_8 = x_train_reduced_8[keep_pol_q8]

    # pipeline to test
    pipeline = Pipeline([
    
    # data standardization (scaler)
    ("scaler", StandardScaler()), 

    # model 
    ("model", LassoCV(
        cv = cv, 
        alphas = 100, 
        max_iter= 10000, 
        random_state = n_number
    ))
])
    
    results_reduced_8 = cross_validate(
        pipeline,
        x_train_reduced_8, 
        y_train_8, 
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

    lassocv_q8_comparisons.append({
    "model": f"LassoCV | top-{k} by |w|",
    "R2_mean": results_reduced_8["test_r2"].mean(),
    "RMSE_mean": (-results_reduced_8["test_neg_root_mean_squared_error"]).mean()}
)
    
# dataframe 
drop_df_8_comparisons = (
            pd.DataFrame(lassocv_q8_comparisons)
            .sort_values("RMSE_mean", ascending=True)
            .reset_index(drop=True))
drop_df_8_comparisons

The results from systematic feature pruning indicate that only modest reduction in the number of predictors is possible without incurring a meaningful loss in predictive performance. Performance remains essentially stable when reducing the model from the full specification to approximately 11 - 12 predictors, with only marginal changes in R² and RMSE.

However, increasing the pruning threshold beyond this point leads to clear and monotonic degradation in both metrics. When the model is restricted to 10 or fewer predictors, predictive accuracy declines steadily, and more aggressive reduction to 7 or fewer variables results in a substantial loss of explanatory power.

This behavior suggests that, while many predictors have relatively small individual coefficients, they collectively contribute meaningful information to the model. Removing too many of these weak but informative predictors eliminates shared signal rather than noise.

The practical limit of parsimony for the linear specification lies at approximately 11 - 12 predictors. Further feature removal materially harms model performance and is therefore not justified. Given that the full model achieves R² ~ 0.71 and RMSE ~ 0.50, pruning to 11 predictors preserves nearly all predictive power while reducing complexity in a controlled and defensible manner.

In [ ]:
# sort features by absolute weight
weights_df_8_sorted = weights_df_8.copy()
weights_df_8_sorted = weights_df_8_sorted.sort_values("abs_weight", ascending=False)

# keep top 11 features
selected = weights_df_8_sorted.head(11)["tag"].tolist()

# reduce data
x_train_reduced_8 = x_train_8[selected]
x_test_reduced_8  = x_test_8[selected]

# pipeline
pipeline_q8_reduced = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LassoCV(
        cv=cv,
        random_state=n_number,
        alphas=100,
        max_iter=10000
    ))
])

In [ ]:
# use cross-validation to save results 
results_reduced_8 = cross_validate(
        pipeline,
        x_train_reduced_8, 
        y_train_8, 
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

rows.append({
    "model": f"LassoCV | top-{len(x_train_reduced_8.columns)} by |w|",
    "R2_mean": results_reduced_8["test_r2"].mean(),
    "RMSE_mean": (-results_reduced_8["test_neg_root_mean_squared_error"]).mean()}
)
    
# dataframe 
compare_models_q8_df = (
            pd.DataFrame(rows)
            .sort_values("RMSE_mean", ascending=True)
            .reset_index(drop=True))
compare_models_q8_df

In [ ]:
# LassoCV with reduced number variables on test data
print("LassoCV with 11 variables:\n")
score_model(pipeline_q8_reduced,
            x_train_reduced_8, 
            x_test_reduced_8, 
            y_train_8, 
            y_test_8)

In [ ]:
# LassoCV on test data 
print("LassoCV with 20 variables:\n")
score_model(pipeline_q8,
            x_train_8, 
            x_test_8, 
            y_train_8, 
            y_test_8)

We evaluated how far the linear model can be reduced while preserving predictive performance by progressively pruning features based on Lasso coefficient magnitude. With all 20 numerical predictors, the LassoCV model achieves a test RMSE of approximately 0.504 and a test R² of about 0.714, which serves as the linear performance benchmark.

When the model is restricted to the top 11 predictors ranked by absolute standardized coefficient, performance remains strong. On the test set, the reduced model attains an RMSE of approximately 0.516 and an R² of about 0.700. Although this represents a measurable decline relative to the full specification, the degradation is modest and does not indicate a loss of generalization stability.

Importantly, the reduced model substantially improves interpretability and simplicity while maintaining competitive predictive accuracy. More aggressive pruning beyond this point leads to increasingly rapid deterioration in both RMSE and R², indicating that additional predictors, while individually weak, collectively contribute meaningful signal.

In summary, while the full 20-variable Lasso model provides the strongest linear performance, the 11-variable specification represents the practical limit of parsimony. Beyond this point, further feature removal yields disproportionate losses in predictive accuracy relative to the gains in simplicity.

In [ ]:
# extract feature weights from the model
model = pipeline_q8_reduced.named_steps["model"]

# feature names
feature_names = x_train_reduced_8.columns.to_list()
# coefficients
coef = model.coef_

# build dataframe
weights_df_8 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_8["abs_weight"] = weights_df_8["weight"].abs()
weights_df_8 = weights_df_8.sort_values("abs_weight", ascending=False)

# cleanup
weights_df_8 = weights_df_8.drop(columns="abs_weight").reset_index(drop=True)
# check 
weights_df_8

These coefficients represent standardized effects from the reduced Lasso model and therefore allow direct comparison of relative importance across predictors. As expected, variables capturing teaching quality and instructor - student interaction dominate the model.

The strongest positive predictors of average rating are `good_feedback`, `amazing_lectures`, `respected`, and `caring`, indicating that clarity, engagement, and perceived respect are the primary drivers of student evaluations. `Hilarious` also exhibits a sizable positive coefficient, suggesting that an engaging and enjoyable classroom atmosphere contributes meaningfully to overall ratings.

In contrast, `tough_grader` is the only predictor with a large negative coefficient, reflecting the well-known trade-off between grading strictness and student satisfaction. Structural and behavioral indicators such as `clear_grading`, `extra_credit`, `participation_matters`, `inspirational`, and accessible contribute positively but with smaller magnitudes, indicating secondary effects once core teaching quality is accounted for.

Overall, the reduced model aligns closely with domain intuition. Student ratings are primarily driven by perceived teaching quality, respect, clarity, and engagement, while grading strictness acts as the main negative factor. At the same time, the model remains parsimonious, relying on a compact and interpretable set of predictors while retaining most of the predictive performance of the full specification.

### Extra: Feature Engineering

#### Index Features 

In [ ]:
x_train_8.info()

In [ ]:
# correlation of predictors with the target 
corr_8_y = x_train_8.corrwith(y_train_8, 
                              method = 'pearson').sort_values(ascending= False)

# check 
corr_8_y

In [ ]:
# teaching quality group
teaching_quality = [
    "good_feedback", "respected", "inspirational", "hilarious", 
    "amazing_lectures", "caring", "accessible", "clear_grading", 
    "accessible"]

# structure group
structure_positive = ["extra_credit", 
             "participation_matters"]

structure_negative = ["group_projects", 
                      "pop_quizzes", 
                      "graded_by_few_things", 
                      "dont_skip_class_or_you_will_not_pass"
                      ]

# workload group
workload = ["so_many_papers", 
            "test_heavy",
            "lots_to_read", 
            "lots_of_homework", 
            "lecture_heavy", 
            "tough_grader"]



# create new index features 
# train data 
x_train_8["teaching_quality_index"] = x_train_8[teaching_quality].mean(axis=1)
x_train_8["workload_index"] = x_train_8[workload].mean(axis=1)
x_train_8["structure_index_positive"] = x_train_8[structure_positive].mean(axis=1)
x_train_8["structure_index_negative"] = x_train_8[structure_negative].mean(axis=1)

# test data 
x_test_8["teaching_quality_index"] = x_test_8[teaching_quality].mean(axis=1)
x_test_8["workload_index"] = x_test_8[workload].mean(axis=1)
x_test_8["structure_index_positive"] = x_test_8[structure_positive].mean(axis=1)
x_test_8["structure_index_negative"] = x_test_8[structure_negative].mean(axis=1)


We introduced index features to capture latent dimensions underlying the individual tag proportions. Many of the original tags are conceptually related, as they describe overlapping aspects of the student experience. Modeling them separately forces the regression to distribute weight across multiple noisy proxies for the same underlying construct, which can reduce stability and interpretability.

To address this, we grouped tags into three conceptually coherent indices. The `teaching_quality_index` aggregates indicators related to instructional clarity, engagement, and instructor support. The `workload_index` summarizes perceived course rigor and assessment intensity. The `structure_index` captures organizational and grading-related aspects of the course. Each index is computed as the mean proportion of its constituent tags, preserving scale while reducing dimensionality.

These indices serve two purposes. First, they mitigate multicollinearity by collapsing correlated predictors into a single, more stable signal. Second, they improve interpretability by allowing the model to reason in terms of high-level educational constructs rather than individual tags. This approach provides a structured summary of the tag space while retaining the core information needed for prediction.

Now let's look at correlations between predictors and target, and between predictors only.

In [ ]:
# correlations of predictors and target variables 
cor_8_y_feature = (
    x_train_8
    .corrwith(y_train_8)
    .sort_values(key=lambda x: x.abs(), ascending=False)
)

cor_8_y_feature

The correlation results summarize the marginal relationships between each predictor and the target variable, average rating. The strongest positive association is observed for the `teaching_quality_index`, indicating that instructional quality, engagement, and instructor support are the primary drivers of student evaluations. In contrast, the `workload_index` exhibits a strong negative correlation, suggesting that heavier perceived workload and assessment intensity are associated with lower ratings.

Among individual tags, variables such as `respected`, `caring`, `good_feedback`, `amazing_lectures`, and `inspirational` show moderate positive correlations, reinforcing the importance of clarity, engagement, and positive instructor - student interaction. Conversely, `tough_grader`, `lecture_heavy`, `lots_of_homework`, `test_heavy`, `lots_to_read`, and `so_many_papers` display moderate negative correlations, reflecting the consistent trade-off between academic rigor and student satisfaction.

Structure-related measures exhibit weaker marginal relationships. While `clear_grading` and `accessible` are positively correlated with ratings, their magnitudes are noticeably smaller than those of teaching quality indicators. Several structural and policy-related tags, including `participation_matters`, `group_projects`, `extra_credit`, and `pop_quizzes`, show correlations close to zero, indicating limited standalone association with overall ratings when considered individually.

Overall, the correlation analysis suggests that most explanatory power resides in teaching quality and workload intensity, while structural and assessment mechanics play a secondary role at the marginal level. Since many of these predictors are conceptually related and may capture overlapping information, the next step is to evaluate multicollinearity among predictors before proceeding to multivariate modeling.

In [ ]:
# correlations between predictors 
corr_8_x = x_train_8.corr()

# plot 
plt.figure(figsize=(12, 10)) # Adjust the figure size as needed

# plot itself
sns.heatmap(corr_8_x, annot=True, cmap='coolwarm', fmt=".2f")

# aesthetics 
# title 
plt.title('Correlation Matrix of Numerical Features', fontweight = "bold", fontsize = 18)

# ticks
plt.tick_params(labelsize = 16)

# show the plot
plt.show()


The correlation matrix shows clear and non-trivial multicollinearity, driven primarily by the constructed indices and their underlying tag variables.

The strongest dependencies appear between `teaching_quality_index` and several individual teaching-related tags. In particular, `teaching_quality_index` is highly correlated with `good_feedback` (≈ 0.48), `respected` (≈ 0.55), `caring` (≈ 0.56), `amazing_lectures` (≈ 0.40), and `hilarious` (≈ 0.38). These magnitudes indicate that the index is capturing much of the same signal as these individual variables, which is expected given its construction as an aggregate measure.

Similarly, `workload_index` exhibits strong correlations with workload-related tags, including `tough_grader` (≈ 0.73), `lots_of_homework` (≈ 0.51), `lots_to_read` (≈ 0.52), `test_heavy` (≈ 0.32), and `lecture_heavy` (≈ 0.45). This confirms that workload_index encodes a shared dimension of assessment intensity and course burden already present in the raw features.

Importantly, the two indices themselves are strongly negatively correlated (~ −0.82), indicating that courses perceived as high quality tend to be perceived as lower workload, and vice versa. Including both indices alongside their component tags therefore introduces overlapping predictors that encode highly similar information.

Outside of the indices, correlations among individual tags are generally moderate rather than extreme, typically in the 0.2-0.4 range in absolute value. This suggests partial redundancy but not exact duplication, meaning these variables contribute related but non-identical information.

The structure-related variables show weaker and more diffuse correlations. While `structure_index_positive` and `structure_index_negative` are correlated with each other and with a small subset of structural tags, their associations with teaching quality and workload variables are noticeably smaller. This supports earlier findings that structural characteristics play a secondary role relative to teaching quality and workload.

Overall, the correlation structure indicates that standard linear regression would suffer from unstable coefficient estimates due to groups of predictors encoding overlapping dimensions, particularly when indices and raw tags are included together. This motivates the use of regularized linear models. Lasso alone risks arbitrarily selecting a single variable from a correlated group, while Ridge retains all predictors without sparsity. Given the observed correlation patterns, `ElasticNet` is well-suited, as it allows correlated predictors to be shrunk together while still performing feature selection, leading to more stable and interpretable models under this dependency structure.

In [ ]:
# cv 
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

pipeline_q8_features = Pipeline([
        # scaler
        ("scaler", StandardScaler()),
        # model
        ("model", ElasticNetCV(
            cv=cv,
            random_state=n_number,
            n_alphas=100,
            max_iter=10000
        ))
    ])

# cross_validation
cv_q8_raw_features = cross_validate(
        pipeline_q8_features,
        x_train_8,
        y_train_8,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

# append to the results 
rows.append({
    "model": "ElaticNetCV | eng.features",
        "R2_mean": cv_q8_raw_features["test_r2"].mean(),
        "RMSE_mean": -cv_q8_raw_features["test_neg_root_mean_squared_error"].mean()
    })

# create and show dataframe 
compare_models_q8_df = ( 
pd.DataFrame(rows)
.reset_index(drop = True)
.sort_values(by = "RMSE_mean") )

# check the data 
compare_models_q8_df

In [ ]:
# LassoCV with top 11 features performance on the test data:
print("LassoCV with top 11 features:\n")
score_model(pipeline_q8_reduced,
            x_train_reduced_8, 
            x_test_reduced_8, 
            y_train_8, 
            y_test_8)

In [ ]:
# ElasticNetCV with engineered features on test data:
print("ElasticNetCV with engineered variables:\n")
score_model(pipeline_q8_features,
            x_train_8, 
            x_test_8, 
            y_train_8, 
            y_test_8)

After introducing the engineered index features, we observe a small but consistent performance improvement for models that retain the full predictor set. Under 5-fold cross-validation, ElasticNetCV with engineered features achieves an RMSE of approximately 0.500 and an R² of about 0.708, which is marginally better than the reduced Lasso specification and comparable to unregularized Linear Regression and Ridge. On the held-out test set, the same model attains a test RMSE of approximately 0.504 and a test R² of about 0.714, confirming that the gain is not limited to cross-validation noise.

By contrast, the Lasso model pruned to 11 predictors exhibits noticeably weaker generalization, with a test RMSE of approximately 0.516 and a test R² of about 0.700. This indicates that while aggressive feature reduction improves simplicity, it comes at a measurable cost in predictive accuracy.

However, the improved ElasticNet model is not sparse. It retains all original tag variables in addition to the engineered indices, which conflicts with the objective of parsimony. Moreover, ElasticNet is computationally more expensive than Lasso due to simultaneous tuning of L1 and L2 penalties. As a result, the observed performance gains appear to come from feature enrichment rather than efficient feature selection.

Therefore, while engineered indices clearly add useful signal when combined with the full feature set, the next step is to inspect coefficient magnitudes and prune weak predictors. The goal is to determine whether most of the performance improvement can be preserved under a reduced specification. Given the strong correlations between indices and their constituent tags, we will also re-evaluate multicollinearity in any reduced subset to decide whether Lasso remains sufficient or whether ElasticNet is still required.

In [ ]:
# model
model = pipeline_q8_features.named_steps["model"]

# extract feature names and coefficients
feature_names = x_train_8.columns
coef = model.coef_


# create the dataframe
weights_df_8 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort 
weights_df_8["abs_weight"] = weights_df_8["weight"].abs()

weights_df_8 = (
    weights_df_8
    .sort_values("abs_weight", ascending=False)
    .reset_index(drop=True)
)

# check results
weights_df_8


In [ ]:
# cv
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# define threshold
initial_threshold = [0.05]
elasticnet_q8_comparisons = []

# filter-out variables 
keep_features_q8_index = weights_df_8[
    weights_df_8["abs_weight"] >= 0.05
].sort_values("abs_weight", ascending=False)["tag"].tolist()

# create train and test samples 
x_train_reduced_8_index = x_train_8[keep_features_q8_index]

x_test_reduced_8_index = x_test_8[keep_features_q8_index]

# loop to drop each variable 1 by one 
for k in range(len(keep_features_q8_index), 0, -1):
    keep_pol_q8_index = keep_features_q8_index[:k]

    x_train_reduced_8_index = x_train_reduced_8_index[keep_pol_q8_index]

    # pipeline to test
    pipeline = Pipeline([
    
    # data standardization (scaler)
    ("scaler", StandardScaler()), 

    # model 
    ("model", ElasticNetCV(
        cv = cv, 
        alphas = 100, 
        max_iter= 10000, 
        random_state = n_number
    ))
])
    
    results_reduced_8_index = cross_validate(
        pipeline,
        x_train_reduced_8_index, 
        y_train_8, 
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

    elasticnet_q8_comparisons.append({
    "model": f"ElasticNetCV | top-{k} indexes",
    "R2_mean": results_reduced_8_index["test_r2"].mean(),
    "RMSE_mean": (-results_reduced_8_index["test_neg_root_mean_squared_error"]).mean()}
)
    
# dataframe 
drop_df_8_comparisons_index = (
            pd.DataFrame(elasticnet_q8_comparisons)
            .sort_values("RMSE_mean", ascending=True)
            .reset_index(drop=True))
drop_df_8_comparisons_index

In [ ]:
# best results dataframe
compare_models_q8_df

Progressively pruning the engineered ElasticNet model shows that modest sparsification is possible, but only up to a point. Reducing the model to 8 or 7 predictors yields cross-validated RMSE values of approximately 0.506-0.507 and R² around 0.699, which is noticeably worse than the full engineered ElasticNet model (RMSE ~ 0.500, R² ~ 0.708), but still materially better than the aggressively pruned Lasso model with 11 features (RMSE ~ 0.510, R² ~ 0.696).

Once the model is reduced below 7 predictors, performance deteriorates rapidly. RMSE increases beyond 0.51 and R² falls below 0.69, indicating that further pruning removes variables that collectively carry meaningful signal. This pattern suggests that the engineered indices and a small subset of original tags jointly encode complementary information, and overly aggressive sparsification breaks that balance.

From a practical standpoint, the 7-feature ElasticNet model represents the maximum reduction achievable without severe loss of predictive accuracy. However, it still underperforms the full engineered model by a non-trivial margin and remains worse than simpler linear baselines such as Ridge and unregularized Linear Regression in cross-validation.

Now we need to check and verify the multicollinearity of our selected variables.

In [ ]:
# sort features by absolute weight
weights_df_8_sorted_index = weights_df_8.copy()
weights_df_8_sorted_index = weights_df_8_sorted_index.sort_values("abs_weight", ascending=False)

# keep top 7 features
selected = weights_df_8_sorted_index.head(7)["tag"].tolist()

# reduce data
x_train_reduced_8_index = x_train_8[selected]
x_test_reduced_8_index  = x_test_8[selected]

# correlations between predictors 
corr_8_x_index = x_train_reduced_8_index.corr()

# plot 
plt.figure(figsize=(12, 10)) # Adjust the figure size as needed

# plot itself
sns.heatmap(corr_8_x_index, annot=True, cmap='coolwarm', fmt=".2f")

# aesthetics 
# title 
plt.title('Correlation Matrix of Numerical Features', fontweight = "bold", fontsize = 18)

# ticks
plt.tick_params(labelsize = 16)

# show the plot
plt.show()


The correlation matrix for the reduced feature set shows no severe multicollinearity. Using a conservative threshold of |ρ| ≥ 0.70 to indicate problematic dependence, none of the retained predictors exceed this level. The strongest association is between `teaching_quality_index` and `tough_grader` (~ −0.62), which is substantial but still below levels typically associated with unstable coefficient estimates. Other correlations involving `teaching_quality_index`, such as with `good_feedback` (~ 0.48), `amazing_lectures` (~ 0.40), and `accessible` (~ 0.40), are moderate and expected given the conceptual overlap between these variables.

Correlations among the remaining predictors are generally weak to moderate, with most falling below |ρ| ≈ 0.30. In particular, `structure_index_positive` shows low correlations with all other variables, and the remaining tag variables do not exhibit strong pairwise dependence.

Given this correlation structure, multicollinearity is unlikely to materially distort coefficient estimates in a linear model. As a result, `ElasticNet` regularization is no longer necessary for stability. We can safely fit a LassoCV model on this reduced feature set to retain sparsity and interpretability without sacrificing numerical stability or generalization performance.

This simplifies the modeling pipeline while preserving the key predictive signal, making Lasso the preferred regularized linear specification for the reduced model.

In [ ]:
# pipeline
pipeline_q8_features_reduced_index = Pipeline([
    # scaler
    ("scaler", StandardScaler()),

    # model 
    ("model", LassoCV(
        cv=cv, 
        random_state=n_number, 
        alphas = 100, 
        max_iter=10000))
    ])

# cross_validation
cv_raw_features_8 = cross_validate(
        pipeline_q8_features_reduced_index,
        x_train_reduced_8_index,
        y_train_8,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

# append to the results 
rows.append({
        "model": f"LassoCV | top-{len(x_train_reduced_8_index.columns)} by |w",
        "R2_mean": cv_raw_features_8["test_r2"].mean(),
        "RMSE_mean": -cv_raw_features_8["test_neg_root_mean_squared_error"].mean()
    })

# return results as dataframe 
compare_models_q8_df = pd.DataFrame(rows).reset_index(drop = True).sort_values(by = "RMSE_mean")

# check 
compare_models_q8_df

Compared to the previously selected sparse linear model, the LassoCV with top-11 features, the engineered-feature Lasso allows for a modest additional reduction in dimensionality without materially affecting predictive performance. The earlier `11-feature model` achieved a cross-validated RMSE of approximately 0.509 and an R² of about 0.696, while the `engineered Lasso with 7 features` retained predictors achieves a CV RMSE of approximately 0.507 and an R² close to `0.699`.

The improvement is small but consistent, and the difference in RMSE relative to the earlier sparse model is within cross-validation noise. Importantly, this result indicates that feature engineering enables a better `sparsity–performance trade-off`: we are able to remove several additional predictors while maintaining essentially the same explanatory power.

However, the table also shows that further reduction beyond this point leads to clear degradation. Models restricted to fewer than seven predictors exhibit noticeable drops in R² and increases in RMSE, indicating that the remaining variables no longer capture enough independent signal.

From a cross-validation perspective, this establishes the engineered-feature Lasso with roughly `7 predictors as the best sparse linear specification`. It improves interpretability relative to the earlier `11-feature model` while preserving nearly identical predictive accuracy.

In [ ]:
# LassoCV with interactions on test data 
print("LassoCV with top 7 engineered features:\n")
score_model(pipeline_q8_features_reduced_index,
            x_train_reduced_8_index, 
            x_test_reduced_8_index, 
            y_train_8, 
            y_test_8)

In [ ]:
# LassoCV with 11 variables 
print("LassoCV with top 11 variables:\n")
score_model(pipeline_q8_reduced,
            x_train_reduced_8, 
            x_test_reduced_8, 
            y_train_8, 
            y_test_8)

On held-out test data, the `LassoCV` model with 7 engineered features slightly outperforms the earlier `11-feature Lasso model`. The 7-feature specification achieves a test RMSE of approximately 0.514 and a test R² of about 0.702, compared to a test RMSE of approximately 0.516 and a test R² of about 0.700 for the 11-feature model.

Although the absolute performance difference is small, it is directionally consistent with cross-validation results and does not reflect overfitting. The train-test gaps remain comparable across both models, indicating stable generalization.

Crucially, this improvement is achieved with four fewer predictors, meaning the engineered-feature Lasso delivers a better `sparsity–performance trade-off`. The engineered indices allow the model to retain core signal while eliminating redundant variables that were previously required in the raw-feature formulation.

In summary, on test data as well as in cross-validation, the 7-feature engineered Lasso dominates the 11-feature raw Lasso, providing equal or slightly better predictive accuracy with a substantially simpler and more interpretable model. This makes the 7-feature engineered Lasso the preferred sparse linear specification.

In [ ]:
# model
model = pipeline_q8_features_reduced_index.named_steps["model"]

# extract feature names and coefficients
feature_names = x_train_reduced_8_index.columns
coef = model.coef_


# create the dataframe
weights_df_8 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort 
weights_df_8["abs_weight"] = weights_df_8["weight"].abs()

weights_df_8 = (
    weights_df_8
    .sort_values("abs_weight", ascending=False)
    .reset_index(drop=True)
)

# check results
weights_df_8


These coefficients represent standardized effects from the final LassoCV model with engineered features and therefore allow direct comparison of relative importance across predictors.

The dominant predictor is `teaching_quality_index`, which has by far the largest positive coefficient. This indicates that overall instructional quality, capturing clarity, engagement, respect, and feedback, is the primary driver of average professor ratings. Its magnitude substantially exceeds that of all other variables, confirming that teaching quality explains most of the systematic variation in ratings.

The strongest negative effect comes from `tough_grader`, reflecting a clear trade-off between grading strictness and student satisfaction. Even after accounting for teaching quality, stricter grading remains associated with lower average ratings.

`structure_index_positive` enters with a moderate positive coefficient, suggesting that well-organized courses with clear structure and policies contribute positively to evaluations, though their impact is secondary to instructional quality itself.

Several individual tags retain smaller but non-negligible effects. `accessible` has a negative coefficient, indicating that once teaching quality and structure are controlled for, accessibility-related perceptions add limited incremental signal and may partially overlap with broader quality measures. `dont_skip_class_or_you_will_not_pass`, `amazing_lectures`, and `good_feedback` have modest positive coefficients, suggesting localized contributions beyond what is already captured by the teaching quality index.

Overall, this final model shows that student ratings are primarily driven by broad teaching quality, with grading strictness acting as the main negative factor and course structure and select engagement indicators providing additional but smaller effects. The model achieves strong predictive performance while remaining highly parsimonious, relying on a compact and interpretable set of predictors that align closely with domain intuition.

#### Contrast Index


In [ ]:
# look at correlation again
corr_8_x_y = (x_train_8.corrwith(y_train_8)
              .sort_values(key=lambda x: x.abs(), ascending=False))
corr_8_x_y

In [ ]:
# contrast indexes between teaching quality and workload indexes 
x_train_8["quality_minus_workload"] = (
    x_train_8["teaching_quality_index"] - x_train_8["workload_index"]
)
x_test_8["quality_minus_workload"] = (
    x_test_8["teaching_quality_index"] - x_test_8["workload_index"]
)

# contrast indexes between teaching quality and structure indexes
x_train_8["quality_minus_structure_negative"] = (
    x_train_8["teaching_quality_index"] - x_train_8["structure_index_negative"]
)
x_test_8["quality_minus_structure_negative"] = (
    x_test_8["teaching_quality_index"] - x_test_8["structure_index_negative"]
)

# contrast indexes between workload and positive structure index 
x_train_8["workload_minus_structure_positive"] = (
    x_train_8["workload_index"] - x_train_8["structure_index_positive"]
)

x_test_8["workload_minus_structure_positive"] = (
    x_test_8["workload_index"] - x_test_8["structure_index_positive"]
)

# contrast indexes between structure indexes 
x_train_8["structure_index_difference"] = (
    x_train_8["structure_index_positive"] - x_train_8["structure_index_negative"]
)

x_test_8["structure_index_difference"] = (
    x_test_8["structure_index_positive"] - x_test_8["structure_index_negative"]
)


In [ ]:
# correlations of predictors and target variables 
cor_8_y_features_contrast = (
    x_train_8
    .corrwith(y_train_8)
    .sort_values(key=lambda x: x.abs(), ascending=False)
)

cor_8_y_features_contrast

The newly constructed contrast features capture relative trade-offs that are central to how students evaluate courses. Rather than measuring absolute levels of quality or workload, these variables encode balance, which is more aligned with how ratings are formed in practice.

The strongest correlation is observed for `quality_minus_workload` (~ 0.81). This indicates that ratings increase most when indicators of teaching quality substantially outweigh indicators of workload intensity. In other words, even demanding courses receive high ratings when strong instructional quality compensates for the workload, while heavy workload without sufficient teaching quality is penalized. This pattern directly reflects the intuitive evaluation process students use and explains why this contrast dominates both raw indices.

The contrast `quality_minus_structure_negative` (~ 0.67) captures a related mechanism. Poor structure harms ratings primarily when it is not compensated by strong teaching. When instructional quality is high, negative structural elements become less salient in student evaluations.

Other contrast variables, such as `workload_minus_structure_positive` and `structure_index_difference`, show more moderate correlations, indicating secondary trade-offs. These features still contribute information, but their effects are weaker than those involving teaching quality versus workload or grading rigor.

Notably, the contrast features exhibit stronger correlations than many individual tags and even the raw indices themselves, confirming that relative comparisons encode more signal than absolute measures. This is expected, as students rarely evaluate workload, grading, or structure in isolation. Instead, they judge whether these aspects feel justified by instructional quality.

Overall, the correlation structure validates the feature engineering strategy. The contrast indices behave exactly as intended, amplifying meaningful trade-offs while compressing overlapping information from multiple tags. This makes them particularly effective for sparsification, as they replace pairs of correlated variables with a single, more informative predictor.

In [ ]:
# cv 
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline
pipeline_q8_features_contrast = Pipeline([
        # scaler
        ("scaler", StandardScaler()),
        # model
        ("model", ElasticNetCV(
            cv=cv,
            random_state=n_number,
            alphas = 100,
            max_iter=10000
        ))
    ])

# cross_validation
cv_q8_raw_features_contrast = cross_validate(
        pipeline_q8_features_contrast,
        x_train_8,
        y_train_8,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

# append to the results 
rows.append({
    "model": "ElaticNetCV | eng.contrast indexes",
    "n_features": len(x_train_8.columns),
        "R2_mean": cv_q8_raw_features_contrast["test_r2"].mean(),
        "RMSE_mean": -cv_q8_raw_features_contrast["test_neg_root_mean_squared_error"].mean()
    })

# create and show dataframe 
compare_models_q8_df = ( 
pd.DataFrame(rows)
.reset_index(drop = True)
.sort_values(by = "RMSE_mean") )

# check the data 
compare_models_q8_df

Looking at cross-validation results, the models that incorporate engineered indices and contrast features achieve nearly identical performance to the strongest linear baselines. `The ElasticNetCV` model with contrast indices attains an RMSE of approximately 0.500 and an R² around 0.708, which is statistically indistinguishable from Linear Regression, RidgeCV, and the earlier ElasticNet and Lasso specifications. The difference in RMSE across these top models is on the order of 10⁻⁴, well within cross-validation noise.

Importantly, while the `contrast-index ElasticNet` performs as expected and slightly compresses signal relative to using separate quality, workload, and structure indices, it does not provide a meaningful performance advantage. Moreover, this specification still relies on a large feature set, retaining many predictors with small coefficients. As a result, it does not satisfy the objective of sparsity and interpretability, despite its strong predictive performance.

In contrast, the `sparse Lasso models` demonstrate the expected trade-off. Reducing the model to 11 or 7 predictors leads to higher RMSE values (~ 0.510 - 507), indicating a modest but real loss in explanatory power. However, these models offer substantial gains in simplicity and interpretability, which are often more valuable in applied settings.

Overall, the cross-validation results indicate that feature engineering and contrast construction can successfully preserve performance, but they do not materially raise the linear performance ceiling. The key benefit of these engineered features lies in enabling feature compression, not in improving predictive accuracy beyond existing linear baselines. This motivates using the engineered features primarily as a tool for further sparsification, rather than as an end in themselves, and sets up the next step of selecting a final sparse specification that balances performance, interpretability, and computational efficiency.

In [ ]:
# check contrast model on test data
print("ElasticNetCV with contrast index:\n")
score_model(pipeline_q8_features_contrast, 
            x_train_8, 
            x_test_8, 
            y_train_8, 
            y_test_8)

In [ ]:
# compare our contrast model to top 9 features engineered model 
print("LassoCV with top 7 engineered features:\n")
score_model(pipeline_q8_features_reduced_index, 
            x_train_reduced_8_index, 
            x_test_reduced_8_index, 
            y_train_8, 
            y_test_8)

On the held-out test set, the ElasticNetCV model with the contrast index also performs marginally better than the sparse Lasso alternative. It achieves a lower test RMSE and a slightly higher test R² than the LassoCV model with the top 7 engineered features, indicating improved generalization rather than overfitting. This consistency between cross-validation and test performance reinforces the value of the contrast feature as a meaningful representation of the underlying signal.

At the same time, this ElasticNet specification still relies on the full predictor set and therefore does not satisfy our goal of sparsity and parsimony. To address this, the next step is to examine the standardized coefficients of the ElasticNet model and prune weak predictors. By removing variables with negligible weights, we aim to recover a more interpretable and computationally efficient model while retaining most of the predictive gains introduced by the contrast index.

In [ ]:
# model
model = pipeline_q8_features_contrast.named_steps["model"]

# extract feature names and coefficients
feature_names = x_train_8.columns
coef = model.coef_


# create the dataframe
weights_df_8 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort 
weights_df_8["abs_weight"] = weights_df_8["weight"].abs()

weights_df_8 = (
    weights_df_8
    .sort_values("abs_weight", ascending=False)
    .reset_index(drop=True)
)

# check results
weights_df_8

Inspection of the final `ElasticNet coefficient estimates` shows clear convergence toward the same small subset of predictors identified in the previous feature-engineering step. The largest standardized coefficient remains `teaching_quality_index`, which dominates all other variables and captures the primary source of variation in ratings. The next strongest effects consistently correspond to `tough_grader`, `structure_index_positive`, and core teaching quality tags such as `amazing_lectures`, `good_feedback`, `accessible`, and `dont_skip_class_or_you_will_not_pass`. These variables form the same core group that defined the earlier `7-feature engineered Lasso model`.

All remaining predictors either have substantially smaller coefficients or are shrunk exactly to zero. Notably, the engineered contrast variables that were introduced to compress signal, such as `quality_minus_workload` and `quality_minus_structure_negative`, are fully eliminated once the model is regularized. This indicates that their informational content is already absorbed by the retained indices and tags and that they do not provide additional incremental signal in the multivariate setting.

This repeated selection of the same small group of variables across different specifications and regularization schemes provides strong evidence that the model has stabilized. Further rounds of feature engineering or pruning are unlikely to yield meaningful gains in either performance or interpretability. At this point, additional complexity would primarily introduce noise rather than signal.

We therefore stop here.

The final sparse linear specification is the `7-feature engineered Lasso model`, which represents the best balance between predictive performance, interpretability, and computational efficiency. It preserves nearly all of the explanatory power of the full linear models while relying on a compact and stable set of predictors.

If the modeling objective were instead shifted from this balance toward pure `performance maximization`, then linear sparsity would no longer be the primary concern. In that case, more flexible nonlinear models such as gradient-boosted trees would be appropriate to evaluate, as they can automatically capture higher-order interactions and nonlinear effects that linear models intentionally ignore.

### Extra: Prediction Accuracy Optimization

#### LightGBM

In [ ]:
# fixed cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline for LightGBM
lgbm_pipe_q8 = Pipeline([
    # scaler
    ("scaler", StandardScaler()), 

    # model 
    ("model", LGBMRegressor(
        objective="regression",
        metric="rmse",
        random_state=n_number,
        n_jobs=1,
        verbosity=-1
    ))
])

# optuna objective
def objective_lgbm(trial):
    params = {
        # tree complexity
        "model__num_leaves": trial.suggest_int("model__num_leaves", 16, 512),
        "model__max_depth": trial.suggest_int("model__max_depth", -1, 20),
        "model__min_child_samples": trial.suggest_int("model__min_child_samples", 5, 300),
        "model__min_sum_hessian_in_leaf": trial.suggest_float("model__min_sum_hessian_in_leaf", 1e-3, 10.0, log=True),

        # learning
        "model__n_estimators": trial.suggest_int("model__n_estimators", 500, 8000),
        "model__learning_rate": trial.suggest_float("model__learning_rate", 0.003, 0.2, log=True),

        # sampling
        "model__subsample": trial.suggest_float("model__subsample", 0.5, 1.0),
        "model__subsample_freq": trial.suggest_int("model__subsample_freq", 1, 10),
        "model__colsample_bytree": trial.suggest_float("model__colsample_bytree", 0.5, 1.0),

        # regularization
        "model__reg_alpha": trial.suggest_float("model__reg_alpha", 1e-8, 50.0, log=True),
        "model__reg_lambda": trial.suggest_float("model__reg_lambda", 1e-8, 50.0, log=True),

        # split control
        "model__min_split_gain": trial.suggest_float("model__min_split_gain", 0.0, 1.0),

        # binning
        "model__max_bin": trial.suggest_int("model__max_bin", 63, 511),
}

    # clone parameters and model pipeline
    pipe = clone(lgbm_pipe_q8)
    pipe.set_params(**params)

    # cross-validation
    scores = cross_val_score(
        pipe,
        x_train_8,
        y_train_8,
        cv=cv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )

    return -scores.mean()

# start optuna study 
study_lgbm = start_study_optuna(
    objective=objective_lgbm,
    n_trials=50,
    sampler_seed=n_number,
    direction="minimize"
)

In [ ]:
# cross-validate the model with best parameters
# set the best parameters to the pipeline
lgbm_pipe_q8.set_params(**study_lgbm.best_params)

cv_results_raw_lgbm_q8 = cross_validate(
    lgbm_pipe_q8,
    x_train_8,
    y_train_8,
    cv=cv,
    scoring=("r2", "neg_root_mean_squared_error"),
    return_train_score=False, 
    return_estimator = True
)

# results
rows.append({
    "model": "Tuned LightGBM",
    "R2_mean": cv_results_raw_lgbm_q8["test_r2"].mean(),
    "RMSE_mean": (-cv_results_raw_lgbm_q8["test_neg_root_mean_squared_error"]).mean()
})

# show as dataframe 
compare_models_q8_df = (
    pd.DataFrame(rows)
    .reset_index(drop = True)
    .sort_values(by = "RMSE_mean"))

# check 
compare_models_q8_df

The performance gap between linear models and LightGBM is substantial rather than marginal. Under 5-fold cross-validation, the tuned LightGBM model achieves an RMSE of approximately 0.48 and an R² of about 0.74, compared to RMSE ~ 0.51 and R² ~ 0.70 for the best linear models. This represents a meaningful reduction in prediction error relative to our strongest linear baseline.

Unlike the linear models, LightGBM is able to capture nonlinear effects and higher-order interactions among tags that linear regression, Lasso, and ElasticNet are structurally unable to represent. The magnitude of the improvement suggests that a nontrivial portion of the signal in the data is inherently nonlinear, rather than simply a result of noise reduction or regularization.

At this stage, cross-validation alone strongly favors LightGBM on predictive grounds. We therefore proceed to evaluate its performance on the held-out test set to confirm that this improvement reflects genuine generalization rather than overfitting.

In [ ]:
# evaluate our baseline model 
# against lgbm model 
print("LightGBM model performance:\n")
score_model(
    lgbm_pipe_q8,
    x_train_8,
    x_test_8,
    y_train_8,
    y_test_8
)

In [ ]:
# evaluate our LassoCV with top 9 engineered features on test data 
# against lgbm model 
print("LassoCV top 7 engineered features on test data:\n")
score_model(
    pipeline_q8_features_reduced_index,
    x_train_reduced_8_index,
    x_test_reduced_8_index,
    y_train_8,
    y_test_8,
)

This gap in performance persists on the held-out test set, where LightGBM achieves test RMSE ~ 0.475 and test R² ~ 0.745, substantially outperforming the best Lasso-based model with engineered features (test RMSE ~ 0.514, test R² ~ 0.705). The consistency between cross-validation and hold-out performance indicates that this gain reflects genuine improvements in generalization rather than overfitting.

This result provides an important conclusion. The linear models, even with careful regularization, feature selection, and domain-driven feature engineering, appear to have fully exhausted the linear signal in the data. Further gains require modeling nonlinearities and interactions among tags, which tree-based gradient boosting methods are explicitly designed to capture. The magnitude of the performance jump suggests that student evaluations are influenced by nonlinear trade-offs and conditional effects, such as workload penalties depending on teaching quality, that cannot be represented within a linear framework.

While LightGBM delivers the strongest predictive performance, it sacrifices direct coefficient-based interpretability. To understand which tags and engineered features drive its predictions, and how they interact, we therefore turn to SHAP values. This allows us to decompose LightGBM predictions into feature-level contributions, providing a transparent explanation of the nonlinear structure responsible for the performance gains.

In [ ]:
# shapley values
explainer = shap.TreeExplainer(lgbm_pipe_q8.named_steps["model"])
shap_values = explainer.shap_values(x_train_8)   

# plot 
shap.summary_plot(shap_values, x_train_8)

The SHAP results show clear convergence in the signal structure. `Teaching quality` is the dominant driver of predicted ratings, with higher values of the `teaching_quality_index` consistently pushing predictions upward and lower values pulling them down. The contrast features behave as intended: `quality_minus_workload` and `quality_minus_structure_negative` have strong positive effects, indicating that when perceived teaching quality outweighs workload intensity or negative course structure, overall ratings increase. 

Grading strictness remains a robust negative factor, as `tough_grader` exerts a sizable downward impact even after accounting for teaching quality. Individual teaching-related tags such as `amazing_lectures` and `respected` add refinement but contribute less than the aggregated index. Workload-related variables like `lecture_heavy`, `lots_to_read`, `lots_of_homework`, and `test_heavy` reduce predictions, but their effects are smaller and more localized. The slightly negative contribution of accessible is likely due to redundancy with the `teaching_quality_index` rather than a true adverse effect. Overall, the model has learned a stable and intuitive structure: ratings are primarily driven by perceived teaching quality and its tradeoff with workload, while additional features provide only marginal incremental information.

LightGBM with tuning does not materially improve RMSE beyond ~0.48. SHAP shows the same structure the linear models learned. Teaching quality dominates. Workload and grading strictness subtract. Contrast features encode the main signal. This means the problem is signal-limited, not model-limited.

Trying XGBoost now would do three things:
- Give almost identical performance to LightGBM
- Learn the same interactions we already exposed explicitly
- Add training cost and explanation burden without payoff

In industry terms, this is diminishing returns. When two different model families - Linear with engineered structure and boosted trees with automatic interactions converge to the same performance and same feature importance, that is a stopping signal.

From a balance perspective, the best model is the `7-feature engineered Lasso`, which preserves nearly all predictive power while remaining simple, stable, and fully interpretable. It achieves strong generalization with minimal computational cost and clear coefficient-based explanations, making it the preferred choice when interpretability and robustness are prioritized.

From a pure performance perspective, LightGBM remains the strongest model. It achieves the lowest RMSE and captures nonlinear interactions automatically, while SHAP analysis confirms that it relies on the same underlying structure identified by the linear models. Importantly, `LightGBM` retains a reasonable degree of interpretability through feature attributions, making it suitable when performance is the primary objective.

In a production setting, `LightGBM` could be further optimized by pruning weak features and tightening tree constraints to reduce computation time and improve efficiency. However, this additional optimization is not necessary here, as the current `LightGBM` model already demonstrates the attainable performance ceiling for this feature space.

Overall, the modeling results show clear convergence. The choice between models depends on the objective: the engineered Lasso offers the best balance of simplicity and accuracy, while LightGBM provides marginally better performance at the cost of increased complexity.

## Q9. Build a regression model predicting average difficulty from all tags (the ones in the rmpCapstoneTags.csv) file. Make sure to include the R2 and RMSE of this model. Which of these tags is most strongly predictive of average difficulty? 

### Linear Regression

In [ ]:
# first, make a subset of all variables that are needed. 
df_9 = df_filtered_final[["average_difficulty"] + df_tags_column_names].copy()

# check 
df_9.info()



In [ ]:
# separate y (dependent variable) and x(independent variables)
y = df_9["average_difficulty"]
x = df_9.drop(columns = "average_difficulty")

# check 
x.info()

In [ ]:
# split the data
x_train_9, x_test_9, y_train_9, y_test_9 = train_test_split(x, y, test_size=0.2, random_state=n_number)

# check distributions of the data 
print(f'Number of rows for train data:{y_train_9.shape[0]}\n')
print(f'Number of rows for test data:{y_test_9.shape[0]}\n')

Quite good amount of rows both in training and test data, we can proceed with the analysis. 

In [ ]:
# correlations between y and x predictors 
# based on training data
corr_yx_q9= x_train_9.corrwith(y_train_9, 
                            method = 'pearson').sort_values(ascending= False)
# check 
corr_yx_q9

In [ ]:
# plot correlation of y and x 
plt.figure(figsize= (12, 10))

# plot itself 
sns.barplot(corr_yx_q9, 
            orient= 'h', 
            color = 'skyblue', 
            edgecolor = 'black')

# title 
plt.title("Correlation of Predictors with Average Difficulty", 
          fontsize = 18, 
          fontweight = "bold")

# xlabel
plt.xlabel("Correlation coefficient", 
           fontsize = 16)
# ylabel 
plt.ylabel("Predictor", 
           fontsize = 16)
# ticks 
plt.tick_params(size = 14)

These correlations describe how students associate course characteristics with perceived difficulty.

Difficulty is driven primarily by grading strictness and workload.
The strongest positive correlation is tough grader (0.66). This dominates all other predictors and clearly anchors students’ notion of difficulty. Additional workload signals reinforce this perception. Don’t skip class or you will not pass (0.29), lots of homework (0.27), lots to read (0.25), test heavy (0.24), and lecture heavy (0.18) all move in the same direction. These tags describe time pressure, assessment intensity, and rigid structure, which students consistently interpret as higher difficulty.

Administrative and minor course features matter little.
Tags such as accessible (0.01), graded by few things (-0.00), and group projects (-0.02) show near-zero correlation. They do not meaningfully influence perceived difficulty.

Teaching quality works in the opposite direction.
All strong negative correlations reflect instructor quality and classroom experience. Caring (-0.35), respected (-0.34), hilarious (-0.28), inspirational (-0.27), clear grading (-0.27), good feedback (-0.25), and amazing lectures (-0.21) are associated with lower perceived difficulty. Students interpret clarity, support, and engagement as making courses feel easier, even when content may still be demanding.

In [ ]:
# correlation matrix among tags
corr_xx_q9 = x_train_9.corr()

In [ ]:
# plot correlation between predictors 
plt.figure(figsize= (12, 10))

# plot itself
sns.heatmap(
    corr_xx_q9,
    cmap="coolwarm",
    center=0,
    square=True,
    cbar=True,
    annot=True,
    fmt=".1f"
)

# title 
plt.title("Correlation Matrix of Tag Proportions", 
          fontweight = 'bold', 
          fontsize = 18)

# tickts 
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tick_params(size = 16)

plt.show()

This correlation matrix confirms the absence of multicollinearity among the tag proportions.

All pairwise correlations remain well below 0.7. Most values lie between -0.4 and 0.4, with no tightly coupled clusters. Even conceptually related tags, such as `caring`, `respected`, `amazing` lectures, and `inspirational`, show only moderate correlations. Workload-related tags also move together weakly rather than forming a single block.

This means the tags capture distinct dimensions of course experience rather than redundant information. As a result, linear models do not suffer from coefficient instability driven by multicollinearity, and regularization is not required for numerical reasons.

We therefore fit and compare three linear specifications: ordinary least squares, Ridge, Lasso, and ElasticNet using the same cross-validation setup.

The purpose here is not to resolve multicollinearity, since the correlation analysis shows no problematic dependence among predictors, but to verify whether regularization improves stability or predictive performance. Ordinary least squares serves as the baseline. Ridge tests whether mild shrinkage improves generalization when predictors are moderately correlated. Lasso tests whether automatic feature selection yields a more parsimonious model without sacrificing accuracy.

Comparing these three models allows us to confirm whether regularization provides any practical benefit in this setting and to justify the final linear specification on empirical grounds rather than assumption.

In [ ]:
# fixed CV
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipelines for linear model comparison (no imputation)
pipelines_linear_q9 = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),

    "RidgeCV": Pipeline([
        ("scaler", StandardScaler()),
        ("model", RidgeCV(
            alphas=np.logspace(-4, 4, 50),
            cv=cv
        ))
    ]),

    "LassoCV": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LassoCV(
            alphas=100,
            cv=cv,
            random_state=n_number,
            max_iter=10000
        ))
    ]),

    "ElasticNetCV": Pipeline([
        ("scaler", StandardScaler()),
        ("model", ElasticNetCV(
            l1_ratio=[0.1, 0.5, 0.9],
            alphas=100,
            cv=cv,
            random_state=n_number,
            max_iter=10000
        ))
    ])
}

# compare models
rows = []
for name, pipe in pipelines_linear_q9.items():
    results = cross_validate(
        pipe,
        x_train_9,
        y_train_9,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

    rows.append({
        "model": name,
        "R2_mean": results["test_r2"].mean(),
        "RMSE_mean": (-results["test_neg_root_mean_squared_error"]).mean()
    })

compare_models_q9_df = (
    pd.DataFrame(rows)
      .sort_values("RMSE_mean", ascending=True)
      .reset_index(drop=True)
)

# check results 
compare_models_q9_df

In [ ]:
# check models on test data
for name, pipe in pipelines_linear_q9.items():
    print(f"\nModel: {name}")
    print(score_model(
        pipe,
        x_train_9,
        x_test_9,
        y_train_9,
        y_test_9
    ))

All four linear models perform almost identically. Cross-validated RMSE differs by less than 0.001, and test-set RMSE and R² are essentially the same across models. These differences are far below any practical significance threshold.

Given this, model choice should be driven by structure and interpretability rather than marginal performance.

Lasso is preferred for three reasons.

First, it delivers comparable predictive accuracy to OLS, Ridge, and Elastic Net. There is no meaningful loss in fit.

Second, it enforces sparsity. Coefficients for weak or redundant tags are shrunk exactly to zero, yielding a simpler model that is easier to explain and reason about.

Third, it aligns with the goal of identifying the most influential teaching attributes rather than stabilizing all coefficients. Ridge and Elastic Net improve numerical stability, but earlier analysis showed no serious multicollinearity. That benefit is therefore unnecessary here.

#### Lasso Regression

In [ ]:
# fixed CV 
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline for Lasso Regression model 
pipeline_q9 = Pipeline([
    # scaler 
    ("scaler", StandardScaler()), 

    # model 
    ("model", LassoCV(
        cv=cv,
        random_state=n_number,
        alphas=100,

        max_iter=10000
    ))
])

# fit 
pipeline_q9.fit(x_train_9, 
                y_train_9)

# extract feature weights from the model
model_q9 = pipeline_q9.named_steps["model"]

# feature names
feature_names_q9 = x_train_9.columns.to_list()
# coefficients
coef_q9 = model_q9.coef_

# build dataframe
weights_df_q9 = pd.DataFrame({
    "tag": feature_names_q9,
    "weight": coef_q9
})

# sort by absolute weight
weights_df_q9["abs_weight"] = weights_df_q9["weight"].abs()
weights_df_q9 = weights_df_q9.sort_values("abs_weight", ascending=False).reset_index(drop = True)

# check 
weights_df_q9

In [ ]:
# Plot weights (coefficients)
# figure size 
plt.figure(figsize=(12, 10))

# plot itself 
sns.barplot(
    data=weights_df_q9,
    x="weight",
    y="tag",
    color="yellow",
    edgecolor="black", 
    orient= "h"
)

# title 
plt.title("Linear Regression Weights Sorted by Magnitude for Predicting Average Difficulty", 
          fontsize = 18, 
          fontweight = "bold")

# xlabel 
plt.xlabel("Tag", fontsize = 16)
# ylabel
plt.ylabel("Standardized regression coefficient", 
           fontsize = 16)

# ticks
plt.xticks(rotation=45, ha="right")
plt.tick_params(size = 14)

# show the plot 
plt.show()

We can see that the `average difficulty` of professors is primarily explained by a small subset of the 13 predictors. In particular, `tough_grader` shows by far the largest standardized weight, indicating that it is the strongest contributor to perceived difficulty. A few additional variables (such as `clear_grading`, `hilarious`, `caring`, `extra_credit`, and `test_heavy`) have moderate effects, while most of the remaining predictors have standardized coefficients that are very small or close to zero, suggesting minimal influence on the response variable.

Therefore, to make our model more parsimonious and sparse - without sacrificing predictive performance - we remove predictors whose absolute standardized weight is below 0.05 and then iteratively reduce number of features, choosing the most optimal model possible. This feature selection step simplifies the model while retaining the most informative variables for predicting average professor difficulty.

In [ ]:
# cv
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# define threshold
initial_threshold = [0.05]
lassocv_q9_comparison = []

# filter-out variables 
keep_features_q9 = weights_df_q9[
    weights_df_q9["abs_weight"] >= 0.05
].sort_values("abs_weight", ascending=False)["tag"].tolist()

# create train and test samples 
x_train_reduced_9 = x_train_9[keep_features_q9]

x_test_reduced_9 = x_test_9[keep_features_q9]

# loop to drop each variable 1 by one 
for k in range(len(keep_features_q9), 0, -1):
    keep_pol_q9 = keep_features_q9[:k]

    x_train_reduced_9 = x_train_reduced_9[keep_pol_q9]

    # pipeline to test
    pipeline = Pipeline([
    
    # data standardization (scaler)
    ("scaler", StandardScaler()), 

    # model 
    ("model", LassoCV(
        cv = cv, 
        alphas = 100, 
        max_iter= 10000, 
        random_state = n_number
    ))
])
    
    results_reduced_9 = cross_validate(
        pipeline,
        x_train_reduced_9, 
        y_train_9, 
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

    lassocv_q9_comparison.append({
    "model": f"LassoCV | top-{k} by |w|",
    "R2_mean": results_reduced_9["test_r2"].mean(),
    "RMSE_mean": (-results_reduced_9["test_neg_root_mean_squared_error"]).mean()}
)
    
# dataframe 
drop_df_9_comparisons = (
            pd.DataFrame(lassocv_q9_comparison)
            .sort_values("RMSE_mean", ascending=True)
            .reset_index(drop=True))
drop_df_9_comparisons

In [ ]:
# best models dataframe
compare_models_q9_df

The progressive pruning results show that meaningful sparsification is very limited in this setting. Performance declines monotonically as predictors are removed, with no flat region beyond the very top of the ranking. The best sparse Lasso specification retains `13 predictors`, achieving a cross-validated RMSE of approximately 0.543, which is close to the linear benchmark RMSE of ~0.541 obtained by Linear Regression, Ridge, ElasticNet, and the full Lasso.

Once the model is reduced below 13 predictors, both RMSE and R² degrade steadily. By the time the model is restricted to 10 or fewer variables, the loss in predictive accuracy becomes clearly non-trivial, and aggressive pruning to fewer than 7 predictors leads to substantial deterioration. This indicates that while individual coefficients may be small, many predictors contribute non-negligible signal collectively.

In [ ]:
# sort features by absolute weight
weights_df_9_sorted = weights_df_q9.copy()
weights_df_9_sorted = weights_df_9_sorted.sort_values("abs_weight", ascending=False)

# keep top 13 features
selected = weights_df_9_sorted.head(13)["tag"].tolist()

# reduce data
x_train_9_reduced = x_train_9[selected]
x_test_9_reduced  = x_test_9[selected]

In [ ]:
# pipeline
pipeline_q9_reduced = Pipeline([
    # scaler
    ("scaler", StandardScaler()),

    # model 
    ("model", LassoCV(
        cv=cv, 
        random_state=n_number, 
        alphas = 100, 
        max_iter=10000))
    ])

results_reduced_9 = cross_validate(
        pipeline,
        x_train_9_reduced, 
        y_train_9, 
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

rows.append({
    "model": f"LassoCV | top-{len(x_train_9_reduced.columns)} by |w|",
    "R2_mean": results_reduced_9["test_r2"].mean(),
    "RMSE_mean": (-results_reduced_9["test_neg_root_mean_squared_error"]).mean()}
)
    
# dataframe 
compare_models_q9_df = (
            pd.DataFrame(rows)
            .sort_values("RMSE_mean", ascending=True)
            .reset_index(drop=True))
compare_models_q9_df

In [ ]:
# score the model on test data 
print("Model performance with all features:\n")
score_model(pipeline_q9, 
            x_train_9, 
            x_test_9, 
            y_train_9, 
            y_test_9)

In [ ]:
# score the model on test data 
print("Model performance with 13 features:\n")
score_model(pipeline_q9_reduced, 
            x_train_9_reduced, 
            x_test_9_reduced, 
            y_train_9, 
            y_test_9)

In [ ]:
# check final model weights 
# extract feature weights from the model
model_q9 = pipeline_q9_reduced.named_steps["model"]

# feature names
feature_names_q9 = x_train_9_reduced.columns.to_list()
# coefficients
coef_q9 = model_q9.coef_

# build dataframe
weights_df_q9 = pd.DataFrame({
    "tag": feature_names_q9,
    "weight": coef_q9
})

# sort by absolute weight
weights_df_q9["abs_weight"] = weights_df_q9["weight"].abs()
weights_df_q9 = weights_df_q9.sort_values("abs_weight", ascending=False)

# cleanup
weights_df_q9 = weights_df_q9.drop(columns="abs_weight").reset_index(drop=True)
# check 
weights_df_q9

### Extra: Feature Engineering

#### Index Features 

In [ ]:
x_train_9.info()

In [ ]:
# check correlation again 
corr_yx_q9

In [ ]:
# teaching quality group
teaching_quality = [
    "good_feedback", "respected", "inspirational", "hilarious", 
    "amazing_lectures", "caring", "accessible", "clear_grading", 
    "accessible"]

# structure group
structure_positive = ["dont_skip_class_or_you_will_not_pass", 
                      "pop_quizzes"]

structure_negative = ["group_projects", 
                      "graded_by_few_things", 
                      "participation_matters", 
                      "extra_credit"]
    
# workload group
workload = ["so_many_papers", 
            "test_heavy",
            "lots_to_read", 
            "lots_of_homework", 
            "lecture_heavy", 
            "tough_grader"]



# create new index features 
# train data 
x_train_9["teaching_quality_index"] = x_train_9[teaching_quality].mean(axis=1)
x_train_9["workload_index"] = x_train_9[workload].mean(axis=1)
x_train_9["structure_index_positive"] = x_train_9[structure_positive].mean(axis=1)
x_train_9["structure_index_negative"] = x_train_9[structure_negative].mean(axis=1)

# test data 
x_test_9["teaching_quality_index"] = x_test_9[teaching_quality].mean(axis=1)
x_test_9["workload_index"] = x_test_9[workload].mean(axis=1)
x_test_9["structure_index_positive"] = x_test_9[structure_positive].mean(axis=1)
x_test_9["structure_index_negative"] = x_test_9[structure_negative].mean(axis=1)


In [ ]:
# correlations of predictors and target variables 
cor_9_y_features = (
    x_train_9
    .corrwith(y_train_9)
    .sort_values(key=lambda x: x.abs(), ascending=False)
)

cor_9_y_features

To capture a larger share of the underlying signal and potentially reduce RMSE, we construct the same set of engineered indices as in Question 8, aggregating related tags into broader dimensions such as teaching quality, workload, and course structure. The correlation results above show that these indices are strongly associated with their component variables, confirming that they successfully compress meaningful variation. However, because the indices and individual tags encode overlapping information, this transformation also introduces the risk of multicollinearity. We therefore proceed by explicitly examining the correlation structure among predictors to assess the extent of multicollinearity before fitting multivariate models.

In [ ]:
# correlations between predictors 
corr_9_x = x_train_9.corr()

# plot 
plt.figure(figsize=(12, 10)) # Adjust the figure size as needed

# plot itself
sns.heatmap(corr_9_x, annot=True, cmap='coolwarm', fmt=".2f")

# aesthetics 
# title 
plt.title('Correlation Matrix of Numerical Features', fontweight = "bold", fontsize = 18)

# ticks
plt.tick_params(labelsize = 16)

# show the plot
plt.show()


As expected from the correlation matrix, several predictors exhibit strong dependence, especially between the engineered indices and their underlying tag variables. Teaching quality, workload, and structure indices are all highly correlated with multiple individual features, creating clear multicollinearity in the full specification. In this setting, fitting Lasso directly would be unstable, since it would arbitrarily select one variable from a correlated group and discard the rest. Therefore, the correct first step is to fit ElasticNet, which can handle correlated predictors by shrinking them jointly. Only after identifying a stable set of important variables through ElasticNet does it make sense to perform feature reduction and then refit a Lasso model for sparsity and interpretability.

In [ ]:
# cv 
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline
pipeline_q9_features = Pipeline([
        # scaler
        ("scaler", StandardScaler()),
        # model
        ("model", ElasticNetCV(
            cv=cv,
            random_state=n_number,
            n_alphas=100,
            max_iter=10000
        ))
    ])

# cross_validation
cv_q9_raw_features = cross_validate(
        pipeline_q9_features,
        x_train_9,
        y_train_9,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

# append to the results 
rows.append({
    "model": "ElaticNetCV | eng.features",
        "R2_mean": cv_q9_raw_features["test_r2"].mean(),
        "RMSE_mean": -cv_q9_raw_features["test_neg_root_mean_squared_error"].mean()
    })

# create and show dataframe 
compare_models_q9_df = ( 
pd.DataFrame(rows)
.reset_index(drop = True)
.sort_values(by = "RMSE_mean") )

# check the data 
compare_models_q9_df

In [ ]:
# LassoCV with top 13 features performance on the test data:
print("LassoCV with top 13 features:\n")
score_model(pipeline_q9_reduced,
            x_train_9_reduced, 
            x_test_9_reduced, 
            y_train_9, 
            y_test_9)

In [ ]:
# ElasticNetCV with engineered features on test data:
print("ElasticNetCV with engineered variables:\n")
score_model(pipeline_q9_features,
            x_train_9, 
            x_test_9, 
            y_train_9, 
            y_test_9)

Cross-validation shows that the `ElasticNet` model with engineered features achieves the best linear performance, with RMSE ~ 0.541 and R² ~ 0.551. This slightly but consistently outperforms all baseline linear specifications, including plain Linear Regression, Ridge, and Lasso. The improvement is small in absolute terms, but it is systematic across folds.

The test-set results confirm the same ordering. The engineered `ElasticNet` model achieves a test RMSE of approximately 0.541 and a test R² of about 0.548, compared to the `13-feature Lasso model`, which reaches a higher test RMSE of roughly 0.544 and a lower test R² of about 0.543. This indicates a genuine generalization gain rather than cross-validation noise.

However, this improvement comes at the cost of complexity. The engineered ElasticNet model retains the full feature set, including correlated indices and individual tags. As a result, the model is not sparse and is harder to interpret, even though its predictive performance is the best among linear models.

Given this tradeoff, the next step is clear. We now fix the engineered ElasticNet as the performance upper bound for linear models and attempt to reduce the number of predictors. The goal is to remove weak coefficients while preserving most of the performance gains. If the reduced feature set does not exhibit problematic multicollinearity, we can refit a Lasso model to recover sparsity and interpretability with minimal loss in accuracy.

In [ ]:
# model
model = pipeline_q9_features.named_steps["model"]

# extract feature names and coefficients
feature_names = x_train_9.columns
coef = model.coef_


# create the dataframe
weights_df_9 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort 
weights_df_9["abs_weight"] = weights_df_9["weight"].abs()

weights_df_9 = (
    weights_df_9
    .sort_values("abs_weight", ascending=False)
    .reset_index(drop=True)
)

# check results
weights_df_9


In [ ]:
# cv
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# define threshold
initial_threshold = [0.05]
elastic_q9_comparison_index = []

# filter-out variables 
keep_features_q9_index = weights_df_9[
    weights_df_9["abs_weight"] >= 0.05
].sort_values("abs_weight", ascending=False)["tag"].tolist()

# create train and test samples 
x_train_9_reduced_index = x_train_9[keep_features_q9_index]

x_test_9_reduced_index = x_test_9[keep_features_q9_index]

# loop to drop each variable 1 by one 
for k in range(len(keep_features_q9_index), 0, -1):
    keep_pol_q9_index = keep_features_q9_index[:k]

    x_train_9_reduced_index = x_train_9_reduced_index[keep_pol_q9_index]

    # pipeline to test
    pipeline = Pipeline([
    
    # data standardization (scaler)
    ("scaler", StandardScaler()), 

    # model 
    ("model", ElasticNetCV(
        cv = cv, 
        alphas = 100, 
        max_iter= 10000, 
        random_state = n_number
    ))
])
    
    results_reduced_9_index = cross_validate(
        pipeline,
        x_train_9_reduced_index, 
        y_train_9, 
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

    elastic_q9_comparison_index.append({
    "model": f"ElasticNetCV | top-{k} by |w|",
    "R2_mean": results_reduced_9_index["test_r2"].mean(),
    "RMSE_mean": (-results_reduced_9_index["test_neg_root_mean_squared_error"]).mean()}
)
    
# dataframe 
drop_df_9_comparisons_index = (
            pd.DataFrame(elastic_q9_comparison_index)
            .sort_values("RMSE_mean", ascending=True)
            .reset_index(drop=True))
drop_df_9_comparisons_index

In [ ]:
# best models dataframe
compare_models_q9_df 

Weselect the 8-feature `ElasticNet specification` as the final reduced linear model. Relative to the full engineered ElasticNet, this model incurs only a very small loss in cross-validated performance, with RMSE increasing from approximately 0.541 to about 0.544, which is well within expected cross-validation noise. At the same time, it substantially improves parsimony by removing a large number of weak predictors whose contribution to predictive accuracy is negligible.

Further reduction beyond eight variables is not justified. Once the model is constrained to seven or fewer predictors, RMSE increases more sharply and R² declines meaningfully, indicating that relevant signal is being discarded rather than redundant noise. Thus, eight predictors represent the practical lower bound at which the engineered linear model retains nearly all of the explanatory power of the full specification.

In summary, the 8-feature ElasticNet model provides the best balance between predictive performance, interpretability, and stability. It captures the dominant structure in the data while avoiding unnecessary complexity, making it the preferred linear specification under a parsimony-aware modeling objective.


Before finalizing the reduced model, we next examine multicollinearity among the selected eight predictors. Feature reduction alone does not guarantee coefficient stability, especially when engineered indices and their component variables may still encode overlapping information.

We therefore inspect pairwise correlations within the 8-feature subset. If correlations remain moderate and below commonly used thresholds for concern, the remaining predictors no longer pose a serious multicollinearity risk. In that case, the benefits of ElasticNet diminish, since its L2 component is primarily needed to stabilize estimates under strong dependence.

If no severe multicollinearity is present, we can refit the model using LassoCV on these eight variables. This yields a simpler and more efficient estimator, preserves sparsity, and improves interpretability by relying purely on L1 regularization. ElasticNet is only retained if meaningful collinearity persists; otherwise, Lasso becomes the appropriate final linear model for this reduced feature set.

In [ ]:
# sort features by absolute weight
weights_df_9_sorted_index = weights_df_9.copy()
weights_df_9_sorted_index = weights_df_9_sorted_index.sort_values("abs_weight", ascending=False)

# keep top 8 features
selected = weights_df_9_sorted_index.head(8)["tag"].tolist()

# reduce data
x_train_9_reduced_index = x_train_9[selected]
x_test_9_reduced_index = x_test_9[selected]

In [ ]:
# correlations between predictors 
corr_9_x_index = x_train_9_reduced_index.corr()

# plot 
plt.figure(figsize=(12, 10)) # Adjust the figure size as needed

# plot itself
sns.heatmap(corr_9_x_index, annot=True, cmap='coolwarm', fmt=".2f")

# aesthetics 
# title 
plt.title('Correlation Matrix of Numerical Features', fontweight = "bold", fontsize = 18)

# ticks
plt.tick_params(labelsize = 16)

# show the plot
plt.show()

The correlation matrix for the reduced eight-feature set shows no problematic multicollinearity. All pairwise correlations remain well below the usual concern threshold of 0.70. The strongest relationship is between `teaching_quality_index` and caring at about 0.56, which is moderate and expected, since caring reflects one aspect of perceived teaching quality. The negative correlation between `teaching_quality_index` and `tough_grader` at roughly -0.62 reflects a meaningful tradeoff rather than redundancy and does not indicate instability. All other correlations are small to moderate, often close to zero, especially among `structure_index_negative`, `clear_grading`, `hilarious`, and `test_heavy`.

Given this structure, the predictors do not encode the same information in a way that would distort coefficient estimates. The stabilization benefit of ElasticNet is therefore no longer necessary. We are free to refit this reduced specification using LassoCV, achieving a sparse, interpretable model without sacrificing numerical stability.

In [ ]:
# pipeline
pipeline_q9_features_reduced = Pipeline([
    # scaler
    ("scaler", StandardScaler()),

    # model 
    ("model", LassoCV(
        cv=cv, 
        random_state=n_number, 
        alphas = 100, 
        max_iter=10000))
    ])

# cross_validation
cv_raw_features_9 = cross_validate(
        pipeline_q9_features_reduced,
        x_train_9_reduced_index,
        y_train_9,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

# append to the results 
rows.append({
        "model": f"Lasso CV | top-{len(x_train_9_reduced_index.columns)} with indexes",
        "R2_mean": cv_raw_features_9["test_r2"].mean(),
        "RMSE_mean": -cv_raw_features_9["test_neg_root_mean_squared_error"].mean()
    })

# return results as dataframe 
compare_models_q9_df = pd.DataFrame(rows).reset_index(drop = True).sort_values(by = "RMSE_mean")

# check 
compare_models_q9_df

In [ ]:
# LassoCV with interactions on test data 
print("LassoCV with top 8 engineered features:\n")
score_model(pipeline_q9_features_reduced,
            x_train_9_reduced_index, 
            x_test_9_reduced_index, 
            y_train_9, 
            y_test_9)

In [ ]:
# LassoCV with 13 variables 
print("LassoCV with top 13 variables:\n")
score_model(pipeline_q9_reduced,
            x_train_9_reduced, 
            x_test_9_reduced, 
            y_train_9, 
            y_test_9)

In [ ]:
# model
model = pipeline_q9_features_reduced.named_steps["model"]

# extract feature names and coefficients
feature_names = x_train_9_reduced_index.columns
coef = model.coef_


# create the dataframe
weights_df_9 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort 
weights_df_9["abs_weight"] = weights_df_9["weight"].abs()

weights_df_9 = (
    weights_df_9
    .sort_values("abs_weight", ascending=False)
    .drop(columns="abs_weight")
    .reset_index(drop=True)
)

# check results
weights_df_9


The dominant driver of higher average difficulty is `tough_grader`. A larger value strongly increases perceived difficulty, which is consistent with students associating strict grading with harder courses. This effect is the largest in the model and clearly dominates all others.

`teaching_quality_index` has a negative coefficient, indicating that higher teaching quality is associated with lower perceived difficulty, holding grading strictness constant. This suggests that good instruction, clarity, and support make courses feel easier even when the material itself may be challenging.

`structure_index_negative` also has a negative coefficient. This index aggregates features such as group projects, fewer grading components, participation-based evaluation, and extra credit. Higher values therefore reflect more flexible or forgiving course structures, which are associated with lower perceived difficulty.

The remaining variables play secondary roles. `accessible` slightly increases perceived difficulty, likely capturing that accessibility often co-occurs with demanding courses that require frequent interaction. `clear_grading` and `hilarious` both reduce difficulty modestly, suggesting that transparency and classroom atmosphere soften how demanding a course feels. `test_heavy` increases difficulty, but its effect is small once grading strictness and structure are accounted for. `caring` has a minor negative effect, reinforcing that instructor support marginally reduces perceived difficulty.

Overall, this final model is coherent and interpretable. Perceived difficulty is primarily driven by grading strictness, then moderated downward by teaching quality and course structure. The model achieves this with a small set of features and minimal redundancy, preserving interpretability while retaining most of the predictive signal.

#### Contrast Index


In [ ]:
# check correlations again 
corr_yx_q9_indexes = ( x_train_9.corrwith(y_train_9).
                      sort_values(key = lambda x: x.abs(), ascending = False))
corr_yx_q9_indexes

In [ ]:
# main tradeoff: quality vs workload
x_train_9["quality_minus_workload"] = (
    x_train_9["teaching_quality_index"] - x_train_9["workload_index"]
)
x_test_9["quality_minus_workload"] = (
    x_test_9["teaching_quality_index"] - x_test_9["workload_index"]
)

# quality vs structure domain 
x_train_9["quality_minus_structure_positive"] = (
    x_train_9["teaching_quality_index"] - x_train_9["structure_index_positive"]
)

x_test_9["quality_minus_structure_positive"] = (
    x_test_9["teaching_quality_index"] - x_test_9["structure_index_positive"]
)


# workload vs structure domain
x_train_9["workload_minus_structure_negative"] = (
    x_train_9["workload_index"] - x_train_9["structure_index_negative"]
)
x_test_9["workload_minus_structure_negative"] = (
    x_test_9["workload_index"] - x_test_9["structure_index_negative"]
)

# optional: a single net-structure score (strictness-ish vs leniency-ish)
x_train_9["structure_net"] = (
    x_train_9["structure_index_positive"] - x_train_9["structure_index_negative"]
)

x_test_9["structure_net"] = (
    x_test_9["structure_index_positive"] - x_test_9["structure_index_negative"]
)

In [ ]:
# correlations of predictors and target variables 
cor_9_y_features_contrast = (
    x_train_9
    .corrwith(y_train_9)
    .sort_values(key=lambda x: x.abs(), ascending=False)
)

cor_9_y_features_contrast

We engineered index and contrast features to capture relative pressure, not just isolated course characteristics. The correlation results justify this design choice.

The strongest correlations are not individual tags but aggregated and contrast features. `tough_grader` and `workload_index` have the highest positive correlations, confirming that perceived difficulty is driven by grading strictness and overall workload intensity. More importantly, contrast features such as `quality_minus_workload` and `workload_minus_structure_negative` appear among the top signals with large magnitudes. This shows that difficulty is determined by imbalance, specifically when workload dominates teaching quality or when workload overwhelms flexible structure. These effects are stronger than many raw tags, which validates encoding tradeoffs explicitly rather than relying on the model to infer them indirectly.

Teaching quality variables consistently load in the opposite direction. `teaching_quality_index` and `quality_minus_structure_positive` are strongly negatively correlated, indicating that strong instruction and positive structure offset difficulty even in demanding courses. Individual quality tags such as `caring`, `respected`, `good_feedback`, `amazing_lectures`, `hilarious`, and `inspirational` all reinforce this pattern with moderate negative correlations. This confirms that these tags capture the same underlying dimension, which motivated collapsing them into a single index in the first place.

Structural features behave asymmetrically, which explains why we separated them into positive and negative components. Enforcement-based structure indicators like `dont_skip_class_or_you_will_not_pass`, `test_heavy`, and `lecture_heavy` correlate positively with difficulty, while flexibility-oriented features such as `extra_credit`, `participation_matters`, and `group_projects` have weak or negative correlations. The derived structure indices reflect this split and show smaller magnitudes than workload or grading, indicating that structure matters, but mainly through how it interacts with workload rather than as a standalone driver.

Overall, the correlation table confirms the core modeling hypothesis same as from Q8. Difficulty is not driven by single tags in isolation. It is driven by relative dominance of workload and grading over teaching quality and flexible structure. The engineered indices and contrast features capture this mechanism directly, reduce redundancy among correlated tags, and concentrate signal into a smaller number of meaningful predictors. 

In [ ]:
# cv 
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline
pipeline_q9_features_contrast = Pipeline([
        # scaler
        ("scaler", StandardScaler()),
        # model
        ("model", ElasticNetCV(
            cv=cv,
            random_state=n_number,
            alphas = 100,
            max_iter=10000
        ))
    ])

# cross_validation
cv_q9_raw_features_contrast = cross_validate(
        pipeline_q9_features_contrast,
        x_train_9,
        y_train_9,
        cv=cv,
        scoring=("r2", "neg_root_mean_squared_error"),
        n_jobs=-1,
        return_train_score=False
    )

# append to the results 
rows.append({
    "model": "ElaticNetCV | with contrast indexes",
        "R2_mean": cv_q9_raw_features_contrast["test_r2"].mean(),
        "RMSE_mean": -cv_q9_raw_features_contrast["test_neg_root_mean_squared_error"].mean()
    })

# create and show dataframe 
compare_models_q9_df = ( 
pd.DataFrame(rows)
.reset_index(drop = True)
.sort_values(by = "RMSE_mean") )

# check the data 
compare_models_q9_df

In [ ]:
# check contrast model on test data
print("ElasticNetCV with contrast index:\n")
score_model(pipeline_q9_features_contrast, 
            x_train_9, 
            x_test_9, 
            y_train_9, 
            y_test_9)

In [ ]:
# LassoCV with interactions on test data 
print("LassoCV with top 8 engineered features:\n")
score_model(pipeline_q9_features_reduced,
            x_train_9_reduced_index, 
            x_test_9_reduced_index, 
            y_train_9, 
            y_test_9)

Again, the `contrast-index` specification outperforms the earlier model with eight engineered features in both cross-validation and test performance. This indicates that explicitly encoding tradeoffs, such as quality versus workload and workload versus structure, captures the underlying signal more efficiently than modeling these components separately.

However, we do not treat this as a final improvement yet. The contrast model still relies on the full predictor set, which inflates complexity and weakens interpretability. The observed gain may partly reflect additional degrees of freedom rather than a genuinely better structural representation. As before, performance alone is not sufficient. The next step is to inspect coefficients and apply reduction to determine whether the contrast features retain their advantage once sparsity is enforced. Only if the improvement survives pruning will we accept the contrast specification as a superior model.

In [ ]:
# model
model = pipeline_q9_features_contrast.named_steps["model"]

# extract feature names and coefficients
feature_names = x_train_9.columns
coef = model.coef_


# create the dataframe
weights_df_9 = pd.DataFrame({
    "tag": feature_names,
    "weight": coef
})

# sort 
weights_df_9["abs_weight"] = weights_df_9["weight"].abs()

weights_df_9 = (
    weights_df_9
    .sort_values("abs_weight", ascending=False)
    .reset_index(drop=True)
)

# check results
weights_df_9





The coefficient pattern confirms convergence. After introducing contrast indices and refitting with regularization, the model again concentrates its mass on the same small subset of predictors that was found before. The largest absolute coefficients are assigned to `tough_grader`, `teaching_quality_index`, `structure_index_negative`, `accessible`, `clear_grading`, `hilarious`, `caring`, and `test_heavy`. All engineered contrast features are shrunk to zero, along with most secondary tags. This means the additional feature engineering does not introduce new independent signal once regularization is applied.

This convergence is the key stopping criterion. Multiple modeling paths - raw tags, aggregated indices, contrast features, Elastic Net followed by pruning - consistently recover the same core drivers. At this point, further linear feature engineering only reshuffles representations of the same information and does not change what the model learns.

We therefore stop. Under the objective of balancing interpretability, stability, and performance, the final model is the sparse linear specification with these eight predictors. If the objective were changed from balance to pure performance maximization, we would instead select to train `LightGBM`, which achieves slightly lower RMSE but at the cost of higher computational complexity and weaker interpretability.

### Extra: Prediction Accuracy Optimization

#### LightGBM

In [ ]:
# fixed cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=n_number)

# pipeline for LightGBM
lgbm_pipe_q9 = Pipeline([
    # scaler
    ("scaler", StandardScaler()), 

    # model 
    ("model", LGBMRegressor(
        objective="regression",
        metric="rmse",
        random_state=n_number,
        n_jobs=1,
        verbosity=-1
    ))
])

# optuna objective
def objective_lgbm(trial):
    params = {
        # tree complexity
        "model__num_leaves": trial.suggest_int("model__num_leaves", 16, 512),
        "model__max_depth": trial.suggest_int("model__max_depth", -1, 20),
        "model__min_child_samples": trial.suggest_int("model__min_child_samples", 5, 300),
        "model__min_sum_hessian_in_leaf": trial.suggest_float("model__min_sum_hessian_in_leaf", 1e-3, 10.0, log=True),

        # learning
        "model__n_estimators": trial.suggest_int("model__n_estimators", 500, 8000),
        "model__learning_rate": trial.suggest_float("model__learning_rate", 0.003, 0.2, log=True),

        # sampling
        "model__subsample": trial.suggest_float("model__subsample", 0.5, 1.0),
        "model__subsample_freq": trial.suggest_int("model__subsample_freq", 1, 10),
        "model__colsample_bytree": trial.suggest_float("model__colsample_bytree", 0.5, 1.0),

        # regularization
        "model__reg_alpha": trial.suggest_float("model__reg_alpha", 1e-8, 50.0, log=True),
        "model__reg_lambda": trial.suggest_float("model__reg_lambda", 1e-8, 50.0, log=True),

        # split control
        "model__min_split_gain": trial.suggest_float("model__min_split_gain", 0.0, 1.0),

        # binning
        "model__max_bin": trial.suggest_int("model__max_bin", 63, 511),
}
    # clone parameters and model pipeline
    pipe = clone(lgbm_pipe_q9)
    pipe.set_params(**params)

    # cross-validation
    scores = cross_val_score(
        pipe,
        x_train_9,
        y_train_9,
        cv=cv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )

    return -scores.mean()

# start optuna study 
study_lgbm = start_study_optuna(
    objective=objective_lgbm,
    n_trials=50,
    sampler_seed=n_number,
    direction="minimize"
)




In [ ]:
# cross-validate the model with best parameters
# set the best parameters to the pipeline
lgbm_pipe_q9.set_params(**study_lgbm.best_params)

cv_results_raw_lgbm_q9 = cross_validate(
    lgbm_pipe_q9,
    x_train_9,
    y_train_9,
    cv=cv,
    scoring=("r2", "neg_root_mean_squared_error"),
    return_train_score=False, 
    return_estimator = True
)

# results
rows.append({
    "model": "Tuned LightGBM",
    "R2_mean": cv_results_raw_lgbm_q9["test_r2"].mean(),
    "RMSE_mean": (-cv_results_raw_lgbm_q9["test_neg_root_mean_squared_error"]).mean()
})

# show as dataframe 
cv_results_raw_lgbm_q9 = (
    pd.DataFrame(rows)
    .reset_index(drop = True)
    .sort_values(by = "RMSE_mean"))

# check 
cv_results_raw_lgbm_q9

In [ ]:
# evaluate our baseline model 
# against lgbm model 
print("LightGBM model performance:\n")
score_model(
    lgbm_pipe_q9,
    x_train_9,
    x_test_9,
    y_train_9,
    y_test_9
)

In [ ]:
# evaluate our LassoCV with top 8 engineered features on test data 
# against lgbm model 
print("LassoCV top 8 engineered features on test data:\n")
score_model(
    pipeline_q9_features_reduced,
    x_train_9_reduced_index,
    x_test_9_reduced_index,
    y_train_9,
    y_test_9
)

The results are fully consistent with expectations. Once the objective shifts from parsimony to pure predictive performance, the tuned LightGBM clearly dominates all linear specifications. In cross-validation, `LightGBM` improves RMSE from roughly 0.54 for the best linear and `LassoCV` models down to about 0.526, with a corresponding increase in R². This advantage persists on the test set, where `LightGBM` achieves a testing RMSE of approximately 0.529 and R² of about 0.567, compared to the best sparse Lasso model's RMSE of roughly 0.545 and R² near 0.541.

The gap is economically meaningful and well outside cross-validation noise. This confirms that nonlinear tree-based models are able to extract additional signal beyond what linear models capture, even after extensive feature engineering. Under a performance-maximization objective, `LightGBM` is therefore the preferred final model.

In [ ]:
# shapley values
explainer = shap.TreeExplainer(lgbm_pipe_q9.named_steps["model"])
shap_values = explainer.shap_values(x_train_9)   

# plot 
shap.summary_plot(shap_values, x_train_9)

This SHAP summary confirms and sharpens everything learned earlier. The model is driven first by grading strictness and workload. High values of `tough_grader` and `workload_index` push predictions strongly toward higher difficulty, with large, asymmetric impacts. The contrast feature `quality_minus_workload` works exactly as intended. When teaching quality outweighs workload, predicted difficulty drops, and when workload dominates, difficulty rises. 

`Test_heavy` reinforces this pattern, adding positive difficulty pressure independently of general workload. `Teaching_quality_index`, `hilarious`, `accessible`, `clear_grading`, and `extra_credit` all reduce predicted difficulty, but with smaller and more localized effects. These features act as moderators rather than primary drivers. 

Structural and participation-related variables cluster tightly around zero, indicating negligible marginal contribution once workload and grading severity are known. Importantly, the nonlinear model assigns importance in the same order and direction as the linear and ElasticNet models, but exploits threshold effects and interaction strength more efficiently. This convergence across model families indicates the problem is signal-limited rather than model-limited. At this point, testing `XGBoost` would almost certainly reproduce the same structure and deliver marginal or no improvement while increasing training cost and explanation burden. We are already squeezing the available signal. The `LightGBM` model represents the performance ceiling, and further modeling effort would have diminishing returns.



If the goal is interpretability, `the linear model with engineered indices` and a small, stable feature set is the clear choice. It captures the dominant mechanisms, workload and grading severity versus teaching quality, with transparent coefficients and minimal complexity.

If the goal is pure predictive performance with no concern for interpretability, `LightGBM` is the top performer. It consistently achieves lower RMSE by exploiting nonlinearities and interaction strength automatically.

That said, the improvement is small, roughly a 2 percent gain in RMSE. Given the added training cost, tuning overhead, and explanation burden, this gain is not compelling enough to justify replacing the linear specification. In practical terms, the models converge on the same signal, and the marginal performance advantage of `LightGBM` does not outweigh the loss in simplicity and clarity.

## Q10. Build a classification model that predicts whether a professor receives a “pepper” from all available factors (both tags and numerical). Make sure to include model quality metrics such as AU(RO)C and also address class imbalance concerns.

In [ ]:
# first, make a subset of all variables that are needed. 
df_10 = df_filtered_final.copy()

# check 
df_10.info()

In [ ]:
# drop female_gender as it is redundant 
# drop major, university, and state 
df_10 = df_10.drop(columns = ['female_gender', 
                            'major', 
                            'university', 
                            'state'])
# check 
df_10.info()

We drop female_gender because gender is already fully captured by the male_gender indicator. Including both would introduce perfect redundancy without adding information.

We also remove major, university, and state. These variables are high-cardinality identifiers and do not have a clear behavioral relationship with whether a professor receives a pepper. The pepper reflects student perception of teaching style and engagement, which is already directly captured by the tag variables and numerical ratings. Once these behavioral signals are included, institutional or geographic identifiers are unlikely to add meaningful, generalizable information.

From a modeling perspective, keeping major, university, or state would substantially increase dimensionality and the risk of overfitting while primarily capturing dataset-specific patterns rather than transferable structure. Since the goal is to build a behavior-driven classification model that generalizes beyond specific institutions, these identifiers are excluded.

In [ ]:
# split the data to x and y 
x_10 = df_10.drop(columns = "received_a_pepper")
y_10 = df_10["received_a_pepper"]

# split between train and test data 
x_train_10, x_test_10, y_train_10, y_test_10 = train_test_split(x_10, y_10, test_size=0.2, random_state=n_number)

# correlations between y and x predictors 
corr_yx_10 = x_train_10.corrwith(y_train_10)

# check 
corr_yx_10.sort_values(ascending=False)

In [ ]:
# plot correlation of y and x 
plt.figure(figsize= (12, 10))

# plot itself 
sns.barplot(corr_yx_10, 
            orient= 'h', 
            color = 'purple', 
            edgecolor = 'black')

# title 
plt.title("Correlation of Predictors with Received a pepper", 
          fontsize = 18, 
          fontweight = "bold")

# xlabel
plt.xlabel("Correlation coefficient", 
           fontsize = 16)
# ylabel 
plt.ylabel("Predictor", 
           fontsize = 16)
# ticks 
plt.tick_params(size = 14)

The correlation analysis provides a clear first look at which factors are associated with receiving a pepper. The strongest positive relationships are driven by overall student sentiment. Average rating shows the highest correlation at about 0.45, followed closely by would take again at about 0.43. This is expected. Peppers are primarily awarded to professors students genuinely like.

Teaching quality tags also matter. Inspirational, amazing lectures, respected, good feedback, caring, and hilarious all show moderate positive correlations in the 0.18-0.25 range. These variables capture engagement, clarity, and emotional connection, which aligns well with the informal nature of the pepper signal.

Course popularity plays a smaller role. Number of ratings is positively correlated at around 0.11, suggesting that more visible professors are slightly more likely to receive a pepper, but exposure alone is not a dominant factor.

On the negative side, difficulty and workload clearly work against receiving a pepper. Tough grader shows the strongest negative correlation at about -0.28. Average difficulty, lecture heavy, lots of homework, lots to read, and test heavy all fall between roughly -0.13 and -0.24. This indicates that stricter and heavier courses reduce the likelihood of a pepper, even if teaching quality is strong.

Administrative or structural factors have little impact. Male gender, number of online ratings, participation matters, accessible, and extra credit all sit close to zero, suggesting minimal direct association with receiving a pepper.

In [ ]:
# correlation matrix among tags
corr_xx = x_train_10.corr()

In [ ]:
# plot correlation between predictors 
plt.figure(figsize= (12, 10))

# plot itself
sns.heatmap(
    corr_xx,
    cmap="coolwarm",
    center=0,
    square=True,
    cbar=True,
    annot=True,
    fmt=".1f"
)

# title 
plt.title("Correlation Matrix of Independent variables", 
          fontweight = 'bold', 
          fontsize = 18)

# tickts 
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tick_params(size = 16)

plt.show()

The correlation matrix shows clear multicollinearity among several predictors, with absolute correlations reaching and exceeding 0.7.
Examples include strong relationships between `average rating`, `would take again`, and multiple teaching quality tags, as well as between difficulty-related variables.

This level of correlation violates the independence assumption required for stable coefficient estimation in standard logistic regression and makes coefficient interpretation unreliable.

Since we want to implement both feature selection and coefficient interpretability, `Elastic Net` is the appropriate choice at this stage.

Elastic Net is preferred because:
- L1 regularization promotes sparsity and feature selection
- L2 regularization stabilizes coefficient estimates under multicollinearity

Using Elastic Net during feature selection allows us to reduce redundancy while maintaining stable, interpretable coefficients, which would not be achievable with pure Lasso under strong multicollinearity.

In [ ]:
# check distributions of the data 
print(f'Number of rows for train data:{y_train_10.shape[0]}\n')
print(f'Number of rows for test data:{y_test_10.shape[0]}\n')

In [ ]:
# check classes in train data
y_train_10.value_counts()

Both classes are reasonably well represented. The training set contains 8,504 observations without a pepper and 6,233 with a pepper, which corresponds to roughly a 58-42 split. This is not a severe imbalance, so extreme remedies such as aggressive resampling are unnecessary. However, the classes are still not perfectly balanced, so evaluation will rely on threshold-independent metrics such as ROC-AUC rather than accuracy.

Given this setup, we proceed without class resampling and instead focus on model specification and preprocessing choices. As a first step, we tune KNN imputation within a regular logistic regression framework to fill null values in `would_take_again` feature. This allows us to assess whether a more sophisticated imputation strategy meaningfully improves predictive performance compared to simpler alternatives such as mean or median imputation. Once the best imputation approach is identified, we will carry it forward to the regularized logistic models.

### Logistic Regression

#### KNN Imputation 

In [ ]:
# fixed cross-validation (classification needs stratification)
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=n_number
)
# pipeline for tuning 
knn_pipe_q10 = Pipeline([
    # imputer 
    ("imputer", KNNImputer()),

    # model 
    ("model", LogisticRegression(
        max_iter=10000,
        solver="lbfgs"
    ))
])

# define objective function for optuna
def objective(trial):
    # parameters to tune (numeric imputer only)
    params = {
        "imputer__n_neighbors": trial.suggest_int("imputer__n_neighbors", 2, 25),
        "imputer__weights": trial.suggest_categorical("imputer__weights", ["uniform", "distance"]),
        "imputer__add_indicator": False
    }

    # clone our model and set parameters
    pipe = clone(knn_pipe_q10)
    pipe.set_params(**params)

    # cross-validation score with tuned parameters (classification metric)
    scores = cross_val_score(
        pipe,
        x_train_10,
        y_train_10,
        cv=cv,
        scoring="roc_auc",
        n_jobs=-1
    )

    # maximize ROC-AUC
    return scores.mean()

# start the study for knn imputer tuning
study_knn_q10 = start_study_optuna(
    objective=objective,
    n_trials=15,
    sampler_seed=n_number,
    direction="maximize"
)

# set the best parameters to the pipeline
knn_pipe_q10.set_params(**study_knn_q10.best_params)

# assess 
score_classifier(
    knn_pipe_q10,
    x_train_10,
    x_test_10,
    y_train_10,
    y_test_10
)

The logistic regression with tuned KNN imputation performs well and generalizes cleanly. ROC-AUC is about 0.793 on both training and test data, indicating strong discriminative ability with no signs of overfitting. The near-identical train and test AUC confirms stable generalization. The use of Stratified K-Fold cross-validation ensures that class proportions are preserved across folds, so these results are not driven by sampling artifacts.

Overall accuracy is roughly 0.72, which is reasonable given the subjective and noisy nature of pepper assignments. Test precision is about 0.66, meaning roughly two thirds of professors predicted to receive a pepper actually do. Test recall is about 0.69, so the model identifies close to 70 percent of true pepper cases. Specificity is about 0.74, indicating slightly better performance at identifying non-pepper cases. The F1 score of approximately 0.68 reflects a balanced tradeoff between precision and recall.

The confusion matrix confirms this balance. Errors are distributed across both classes, and there is no evidence that the model systematically favors one class over the other. Class imbalance is therefore not a limiting factor in this setting, and no reweighting or resampling is required at this stage.

With this strong and stable baseline in place, the next step is to compare different imputation strategies. We will evaluate whether simpler imputers, such as mean or median imputation, perform comparably to the tuned KNN imputer, or whether the added complexity of KNN imputation provides a measurable advantage in classification performance.

In [ ]:
# fixed cross-validation (classification needs stratification)
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=n_number
)

# pipelines for comparison
pipelines_q10 = {
    "KNNImputer (tuned)": Pipeline([
        # imputer
        ("imputer", KNNImputer(
            n_neighbors=22,
            weights="uniform",
            add_indicator=False
        )),

        # scaler
        ("scaler", StandardScaler()),

        # model
        ("model", LogisticRegression(
            max_iter=10000,
            solver="lbfgs",
            random_state=n_number
        ))
    ]),

    "SimpleImputer (mean)": Pipeline([
        ("imputer", SimpleImputer(
            strategy="mean",
            add_indicator=False
        )),
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(
            max_iter=10000,
            solver="lbfgs",
            random_state=n_number
        ))
    ]),

    "SimpleImputer (median)": Pipeline([
        ("imputer", SimpleImputer(
            strategy="median",
            add_indicator=False
        )),
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(
            max_iter=10000,
            solver="lbfgs",
            random_state=n_number
        ))
    ]),
}

# compare the pipelines
rows = []
for name, pipe in pipelines_q10.items():
    results = cross_validate(
        pipe,
        x_train_10,
        y_train_10,
        cv=cv,
        scoring={
            "roc_auc": "roc_auc",
            "accuracy": "accuracy",
            "precision": "precision",
            "recall": "recall",
            "f1": "f1"
        },
        return_train_score=False,
        n_jobs=-1
    )

    rows.append({
        "model": name,
        "ROC_AUC_mean": results["test_roc_auc"].mean(),
        "Accuracy_mean": results["test_accuracy"].mean(),
        "Precision_mean": results["test_precision"].mean(),
        "Recall_mean": results["test_recall"].mean(),
        "F1_mean": results["test_f1"].mean()
    })

# create a dataframe for comparison
compare_imputers_df_q10 = (
    pd.DataFrame(rows)
      .sort_values("ROC_AUC_mean", ascending=False)
      .reset_index(drop=True)
)

# check the results
compare_imputers_df_q10

All three imputers produce nearly identical performance across all metrics.
`ROC AUC` differences are below 0.001 and precision, recall, and F1 are effectively the same.

Since KNN and median imputation are more computationally expensive and do not provide any measurable performance benefit, mean imputation is the most reasonable choice.

Mean imputation is:
- Simpler
- Faster
- Deterministic
- Easier to maintain in production

Because there is no accuracy or ranking advantage from more complex imputers, using mean imputation improves operational simplicity without sacrificing model performance.

#### Logistic Regression tuning

In [ ]:
# cv 
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=n_number
)

# pipelines for logistic models
pipelines_q10_logistic = Pipeline([
    # imputer 
    ("imputer", SimpleImputer(strategy="mean")),

    # scaler
    ("scaler", StandardScaler()),

    # model itself
    ("model", LogisticRegressionCV(
        cv=5,
        l1_ratios=[0.1, 0.5, 0.9],
        Cs = np.logspace(-3, 2, 6),
        penalty='elasticnet',  
        solver='saga',
        scoring='roc_auc',
        max_iter=10000,
        random_state=n_number,
        n_jobs= 1
    ))
])


# cross-validation
results_logistic = cross_validate(
    pipelines_q10_logistic,
    x_train_10,
    y_train_10,
    cv=cv,
    scoring=['roc_auc', 
             'accuracy', 
             'precision', 
             'recall', 
             'f1'],
    n_jobs=-1,  
    return_train_score=False
)

# copy rows with results 
models_comparison = rows.copy()

models_comparison.append({
        "model": "Logistic Regression with ElasticNet penalty",
        "ROC_AUC_mean": results_logistic["test_roc_auc"].mean(),
        "Accuracy_mean": results_logistic["test_accuracy"].mean(),
        "Precision_mean": results_logistic["test_precision"].mean(),
        "Recall_mean": results_logistic["test_recall"].mean(),
        "F1_mean": results_logistic["test_f1"].mean()
    })

# save to the dataset 
compare_models_q10_logistic_df = (
    pd.DataFrame(models_comparison)
      .sort_values("ROC_AUC_mean", ascending=False)
      .reset_index(drop=True)
)

# check 
compare_models_q10_logistic_df

After introducing L1 and L2 penalties through Elastic Net, we observe a marginal decrease across all evaluation metrics in relation to SimpleImputer (mean) logistic regression model. This behavior is expected, as regularization deliberately introduces bias in exchange for reduced variance, improved generalization, and better handling of multicollinearity.

Despite the slight metric drop, cross-validation results remain stable, indicating that the model is not overfitting and that performance differences are within noise.

While the regularized model performs comparably to the unregularized baseline, fitting `Elastic Net` with the full set of predictors is computationally more expensive and adds unnecessary complexity. Since the goal includes interpretability and deployment robustness, retaining all variables is not justified.

As a result, after looking at test data sample performance, we proceed by pruning predictors based on standardized Elastic Net coefficients, retaining only features with meaningful contribution to the decision boundary.

In [ ]:
# assess on test data
print("Elastic-Net Logistic Regression performance on test data:")
score_classifier(
    pipelines_q10_logistic,
    x_train_10,
    x_test_10,
    y_train_10,
    y_test_10)

Held-out test performance closely matches cross-validation results, indicating strong generalization and model stability.

In [ ]:
# extract weights from the fitted model
logistic_model = pipelines_q10_logistic.named_steps["model"]

# feature names after preprocessing
feature_names = pipelines_q10_logistic.named_steps["imputer"].get_feature_names_out(
    x_train_10.columns
)

# coefficients (binary classification -> 1D)
coef = logistic_model.coef_.ravel()

# build dataframe of weights
weights_df_q10_elastic = pd.DataFrame({
    "feature": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_q10_elastic["abs_weight"] = weights_df_q10_elastic["weight"].abs()
weights_df_q10_elastic = (
    weights_df_q10_elastic
      .sort_values("abs_weight", ascending=False)
      .reset_index(drop=True)
)

# check
weights_df_q10_elastic

The standardized Elastic Net coefficients do not exhibit a clear elbow or sharp cutoff point. Instead, weights decay gradually, indicating that several predictors contribute weakly and may be redundant under multicollinearity.

Given the absence of a natural breakpoint, we adopt a pragmatic threshold of |weight| < 0.05 as an initial pruning criterion. Coefficients below this level are small relative to the dominant predictors and likely contribute marginal signal.

This threshold is not treated as a hard rule, but as a starting point.

In [ ]:
# cv
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=n_number)

# define threshold
initial_threshold = [0.05]
elasticnetcv_q10_comparisons = []

# filter-out variables 
keep_features = weights_df_q10_elastic[
    weights_df_q10_elastic["abs_weight"] >= 0.05
].sort_values("abs_weight", ascending=False)["feature"].tolist()

# create train and test samples 
x_train_reduced_10 = x_train_10[keep_features]
x_test_reduced_10 = x_test_10[keep_features]

# loop to drop each variable 1 by one 
for k in range(len(keep_features), 0, -1):
    keep_pool = keep_features[:k]

    x_train_reduced_10 = x_train_10[keep_pool]

    pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
        ("model", LogisticRegressionCV(
            cv=5,
            l1_ratios=[0.1, 0.5, 0.9],
            Cs=np.logspace(-3, 2, 6),
            penalty="elasticnet",
            solver="saga",
            scoring="roc_auc",
            max_iter=10000,
            random_state=n_number,
            n_jobs=1
        ))
    ])

    # save results 
    results_reduced_10 = cross_validate(
        pipeline,
        x_train_reduced_10,
        y_train_10,
        cv=cv,
        scoring=["roc_auc", "accuracy", "precision", "recall", "f1"],
        n_jobs=-1,
        return_train_score=False
    )
    # append results 
    elasticnetcv_q10_comparisons.append({
        "model": f"ElasticNetCV | top-{k} by |w|",
        "ROC_AUC_mean": results_reduced_10["test_roc_auc"].mean(),
        "Accuracy_mean": results_reduced_10["test_accuracy"].mean(),
        "Precision_mean": results_reduced_10["test_precision"].mean(),
        "Recall_mean": results_reduced_10["test_recall"].mean(),
        "F1_mean": results_reduced_10["test_f1"].mean(),
        "num_features": k
    })

# dataframe 
drop1_df = (pd.DataFrame(elasticnetcv_q10_comparisons)
            .sort_values("ROC_AUC_mean", ascending=False)
            .reset_index(drop=True))
drop1_df


As features are progressively removed based on `standardized Elastic Net weights`, model performance degrades smoothly rather than abruptly. This indicates that weaker predictors contribute marginal signal but are partially redundant.

The `6-feature model` represents the best trade-off between performance and parsimony:
- ROC AUC remains high at ~0.787
- F1 score is near its maximum
- Recall slightly improves relative to larger models
- Accuracy remains stable
- Model complexity is reduced by ~70%

Below 6 features, performance begins to degrade more noticeably, particularly in `ROC AUC` and stability. Above 6 features, additional predictors provide diminishing returns while increasing model complexity and multicollinearity risk.

We therefore select the top 6 features by absolute standardized Elastic Net weight as the final feature set.

This choice preserves predictive performance while improving interpretability, robustness, and deployment simplicity.

In [ ]:
# keep features of 6-variable model 
keep_features = weights_df_q10_elastic.head(6)["feature"].tolist()

# reduce data
x_train_reduced_10_final = x_train_10[keep_features]
x_train_reduced_10_final = x_test_10[keep_features]



In [ ]:
# reduced pipeline
pipelines_q10_logistic_reduced = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler()),
    ("model", LogisticRegressionCV(
        cv=5,
        l1_ratios=[0.1, 0.5, 0.9],
        Cs=np.logspace(-3, 2, 6),
        penalty="elasticnet",
        solver="saga",
        scoring="roc_auc",
        max_iter=10000,
        random_state=n_number,
        n_jobs=1
    ))
])

# cross-validation 
results_reduced = cross_validate(
    pipelines_q10_logistic_reduced,
    x_train_reduced_10_final,
    y_train_10,
    cv=cv,
    scoring=['roc_auc', 'accuracy', 'precision', 'recall', 'f1'],
    n_jobs=-1,
    return_train_score=False
)

# appends results to our dictionary 
models_comparison.append({
    "model": f"ElasticNetCV | top-{len(x_train_reduced_10_final.columns)} features by |w|",
    "ROC_AUC_mean": results_reduced["test_roc_auc"].mean(),
    "Accuracy_mean": results_reduced["test_accuracy"].mean(),
    "Precision_mean": results_reduced["test_precision"].mean(),
    "Recall_mean": results_reduced["test_recall"].mean(),
    "F1_mean": results_reduced["test_f1"].mean()
})

# check 
compare_models_q10_logistic_df = (
    pd.DataFrame(models_comparison)
      .sort_values("ROC_AUC_mean", ascending=False)
      .reset_index(drop=True)
)
compare_models_q10_logistic_df 

Reducing the feature set from the full model to the top 6 Elastic Net - selected predictors resulted in only a marginal decrease in ROC AUC (~0.004), while precision, recall, and F1 remained stable, indicating that the pruned model retains most of the predictive signal.

In [ ]:
# assess 6 variable model on test data
print("ElasticNet CV with 6 features:\n")
score_classifier(
    pipelines_q10_logistic_reduced,
    x_train_reduced_10_final,
    x_test_reduced_10_final,
    y_train_10,
    y_test_10)

In [ ]:
# assess all variables model on test data
print("ElasticNet CV with all features:\n")
score_classifier(
    pipelines_q10_logistic,
    x_train_10,
    x_test_10,
    y_train_10,
    y_test_10)

The 6 - feature `Elastic Net` model shows only a modest reduction in ROC AUC compared to the full model (≈ 0.006 - 0.008), while maintaining nearly identical F1 score and slightly improving recall on the test set.

Key observations:
- ROC AUC drops from ~0.794 to ~0.786. This is small and expected after aggressive feature pruning.
- Recall increases from ~0.68 to ~0.69, aligning better with a recall-oriented objective.
- Precision decreases slightly, consistent with the recall gain.
- F1 score remains effectively unchanged.
- Specificity decreases modestly, reflecting the same trade-off.

Overall, predictive behavior remains stable and consistent between train and test, indicating no overfitting and good generalization.
The reduced model uses only 6 predictors and achieves significantly faster inference and lower computational overhead compared to the full feature set. This makes the pruned model more suitable for deployment, especially in settings where latency or scalability matters.


- Given the minimal performance loss, stable generalization, and substantial gains in simplicity and speed, the 6 - feature Elastic Net model represents the better deployment choice.

In [ ]:
# correlation matrix among final predictors 
corr_xx_10_reduced = x_train_reduced_10_final.corr()

# plot correlation between predictors 
plt.figure(figsize= (12, 10))

# plot itself
sns.heatmap(
    corr_xx_10_reduced,
    cmap="coolwarm",
    center=0,
    square=True,
    cbar=True,
    annot=True,
    fmt=".1f"
)

# title 
plt.title("Correlation Matrix of Independent variables", 
          fontweight = 'bold', 
          fontsize = 18)

# tickts 
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tick_params(size = 16)

plt.show()

Since we still see high multicollinearity between predictors (>=|0.7|) after reducing the subset, we proceed with ElasticNet logistic regression.

In [ ]:
# confusion matrix 
# predict 
y_test_10_pred = pipelines_q10_logistic_reduced.predict(x_test_reduced_10_final)
y_test_prob_10 = pipelines_q10_logistic_reduced.predict_proba(x_test_reduced_10_final)[:, 1]

cm = confusion_matrix(y_test_10, y_test_10_pred)

# figure size 
plt.figure(figsize=(12, 10))

# confusion matrix parameters 
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues", values_format="d")

# title 
plt.title("Confusion Matrix", fontsize=18, fontweight="bold")

# xlabel
plt.xlabel("Predicted", fontsize=16)
#ylabel
plt.ylabel("Actual", fontsize=16)

# ticks
plt.tick_params(labelsize=14)

plt.show()

The confusion matrix shows balanced and stable behavior on the held-out test set. The model correctly identifies a large portion of positive cases, with 1,087 true positives, while missing 489 positives, resulting in recall of approximately 0.69. This confirms that the model is operating in a recall-oriented regime.

On the negative class, the model correctly classifies 1,535 instances, while producing 574 false positives, yielding a specificity of about 0.73. This level of false positives is expected given the emphasis on recall.

Overall, the distribution of errors is consistent with cross-validation results and indicates no pathological behavior or overfitting. These results are obtained using the default classification threshold, without any additional threshold tuning.

In [ ]:
# create the figure once
fig, ax = plt.subplots(figsize=(12, 10))

# diagonal baseline once
ax.plot([0, 1], [0, 1], linewidth=2)

# add models using our function 
add_roc(ax, y_test_10, y_test_prob_10, "ElasticNetCV | top-{6} features by |w|")

# aesthetics 
# title
ax.set_title("AUROC Curve", fontsize=18, fontweight="bold")

# xlabel
ax.set_xlabel("False Positive Rate", fontsize=16)
# ylabel
ax.set_ylabel("True Positive Rate", fontsize=16)

# tick_params
ax.tick_params(labelsize=14)

# legend
ax.legend(frameon=True, loc="lower right")

# show the plot
plt.show()


In [ ]:
# predicted probabilities on train set
y_train_prob_10 = pipelines_q10_logistic_reduced.predict_proba(x_train_reduced_10_final)[:, 1]

# thresholds
thresholds = np.linspace(0.01, 0.99, 50)
rows_thresholds = []

# loop over many thresholds
for t in thresholds:
    y_pred = (y_train_prob_10 >= t).astype(int)
    rows_thresholds.append({
        "threshold": t,
        "AUC": roc_auc_score(y_train_10, y_train_prob_10),
        "recall": recall_score(y_train_10, y_pred),
        "precision": precision_score(y_train_10, y_pred),
        "f1": f1_score(y_train_10, y_pred)
    })

# check 
probability_threshold_10_reduced = pd.DataFrame(rows_thresholds)
probability_threshold_10_reduced


The table shows the expected precision - recall trade-off as the classification threshold increases. Lower thresholds capture nearly all positives but generate many false positives, while higher thresholds sharply reduce recall in exchange for higher precision.

Since we think that the better objective is to identify as many professors who receive a pepper as possible, recall is the primary metric. Precision is secondary, as false positives are less costly than missed positives in this setting.

At a threshold of 0.27:
- Recall is ~0.90
- Precision is ~0.56
- F1 score is near its maximum
- Performance degradation relative to neighboring thresholds is minimal

Beyond this point, recall drops more rapidly without providing a proportional gain in precision or overall model quality. Below this point, recall gains are marginal while precision deteriorates noticeably.

We therefore select 0.27 as the operating threshold. This threshold maximizes recall while maintaining reasonable precision and overall balance, and aligns with the goal of identifying as many qualifying professors as possible without introducing excessive noise.

This choice reflects an explicit, cost-aware decision rather than arbitrary thresholding.

In [ ]:
# apply chosen threshold
threshold = 0.27
y_test_10_prob_threshold = pipelines_q10_logistic_reduced.predict_proba(x_test_reduced_10_final)[:, 1]
y_test_10_pred_threshold = (y_test_10_prob_threshold >= threshold).astype(int)

In [ ]:
# confusion matrix 
cm = confusion_matrix(y_test_10, y_test_10_pred_threshold)

# figure size 
plt.figure(figsize=(12, 10))

# confusion matrix parameters 
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues", values_format="d")

# title 
plt.title("Confusion Matrix", fontsize=18, fontweight="bold")

# xlabel
plt.xlabel("Predicted", fontsize=16)
#ylabel
plt.ylabel("Actual", fontsize=16)

# ticks
plt.tick_params(labelsize=14)

plt.show()

At the selected threshold of 0.27, the model strongly prioritizes recall. The number of true positives increases substantially to 1,431, while false negatives drop to 145, confirming that most professors who receive a pepper are correctly identified.

This improvement in recall comes at the cost of additional false positives, with 1,087 negative cases incorrectly flagged as positive. As a result, specificity decreases, which is expected given the recall-first objective.

Overall, this behavior is consistent with the threshold analysis. The model successfully shifts toward identifying as many positive cases as possible, aligning with the stated goal, without introducing instability or unexpected error patterns.

Next step

With the operating point fixed, we now examine the six retained predictors to understand which variables drive the model’s decisions and how they contribute to identifying professors who receive a pepper.

In [ ]:
# extract weights from the fitted model
logistic_model_final = pipelines_q10_logistic_reduced.named_steps["model"]

# feature names after preprocessing
feature_names = pipelines_q10_logistic_reduced.named_steps["imputer"].get_feature_names_out(
    x_train_reduced_10_final.columns
)

# coefficients (binary classification -> 1D)
coef = logistic_model_final.coef_.ravel()

# build dataframe of weights
weights_df_q10_elastic_final = pd.DataFrame({
    "feature": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_q10_elastic_final["abs_weight"] = weights_df_q10_elastic_final["weight"].abs()
weights_df_q10_elastic_final = (
    weights_df_q10_elastic_final
      .sort_values("abs_weight", ascending=False)
      .reset_index(drop=True)
)

# check
weights_df_q10_elastic_final

Final model features and interpretation: 

After feature selection and threshold tuning, the final model retains six predictors. All coefficients are standardized, so magnitudes are directly comparable.

- `average_rating`
This is the strongest predictor by a wide margin. Higher average ratings substantially increase the likelihood of receiving a pepper, indicating that overall student satisfaction is the dominant signal.

- `number_of_ratings`
A positive coefficient suggests that professors with more ratings are more likely to receive a pepper. This likely reflects visibility and confidence in the estimate rather than pure quality.

- `inspirational`
Professors described as inspirational are more likely to receive a pepper, capturing a qualitative teaching attribute beyond raw difficulty or grading.

- `average_difficulty`
Higher difficulty surprisingly slightly increases the probability of receiving a pepper. This suggests that challenge, when paired with quality, is not penalized and may even be valued.

- `would_take_again`
A positive effect indicates that students' willingness to retake a professor strongly aligns with receiving a pepper, reinforcing this variable as a proxy for overall teaching approval.

- `lots_of_homework`
This is the only negative coefficient. Professors perceived as assigning excessive homework are less likely to receive a pepper, even after controlling for overall rating and difficulty.

The final model captures a coherent narrative: peppers are associated with highly rated, inspiring professors who are challenging but not overwhelming, widely reviewed, and perceived as worth retaking. The retained features are interpretable, stable, and align with intuitive expectations.

This confirms that aggressive pruning preserved the core signal while removing redundant noise.

### Extra: Feature Engineering 

In [ ]:
# look at train_data
x_train_10.info()

In [ ]:
# look at correlations again 
x_train_10.corrwith(y_train_10).sort_values(ascending= False)
# check

#### Polynomials

In [ ]:
# polynomials 
# copy x_train and x_test for safety 
# since the original dataset is pretty large by number of variables, we need to copy 
x_train_10_pol = x_train_10.copy()
x_test_10_pol = x_test_10.copy()

# average rating
x_train_10_pol["average_rating^2"] = np.square(x_train_10_pol["average_rating"])
x_test_10_pol["average_rating^2"] = np.square(x_test_10_pol["average_rating"])

# average difficulty 
x_train_10_pol["average_difficulty^2"] = np.square(x_train_10_pol["average_difficulty"])
x_test_10_pol["average_difficulty^2"] = np.square(x_test_10_pol["average_difficulty"])

In [ ]:
# look at correlations again 
x_train_10_pol.corrwith(y_train_10).sort_values(ascending= False)
# check

In [ ]:
# let's try to drop redundant average_rating now 
# average rating 
x_train_10_pol.drop("average_rating", 
                    axis = 1, 
                    inplace = True)

x_test_10_pol.drop("average_rating", 
                    axis = 1, 
                    inplace = True)

# average difficulty 
x_train_10_pol.drop("average_difficulty", 
                    axis = 1, 
                    inplace = True)

x_test_10_pol.drop("average_difficulty", 
                    axis = 1, 
                    inplace = True)

In [ ]:
# cv 
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=n_number
)

# pipelines for logistic models
pipelines_q10_logistic_pol = Pipeline([
    # imputer 
    ("imputer", SimpleImputer(strategy="mean")),

    # scaler
    ("scaler", StandardScaler()),

    # model itself
    ("model", LogisticRegressionCV(
        cv=5,
        l1_ratios=[0.1, 0.5, 0.9],
        Cs = np.logspace(-3, 2, 6),
        penalty='elasticnet',  
        solver='saga',
        scoring='roc_auc',
        max_iter=10000,
        random_state=n_number,
        n_jobs= 1
    ))
])


# cross-validation
results_features_pol = cross_validate(
    pipelines_q10_logistic_pol,
    x_train_10_pol,
    y_train_10,
    cv=cv,
    scoring=['roc_auc', 
             'accuracy', 
             'precision', 
             'recall', 
             'f1'],
    n_jobs=-1,  
    return_train_score=False
)

# results 
models_comparison.append({
        "model": "Logistic Regression with ElasticNet penalty (polynomial features)",
        "ROC_AUC_mean": results_features_pol["test_roc_auc"].mean(),
        "Accuracy_mean": results_features_pol["test_accuracy"].mean(),
        "Precision_mean": results_features_pol["test_precision"].mean(),
        "Recall_mean": results_features_pol["test_recall"].mean(),
        "F1_mean": results_features_pol["test_f1"].mean()
    })

# save to the dataset 
compare_models_q10_features_polynomials = (
    pd.DataFrame(models_comparison)
      .sort_values("ROC_AUC_mean", ascending=False)
      .reset_index(drop=True)
)

# check 
compare_models_q10_features_polynomials

In [ ]:
# assess polynomials on test data
print("Elastic-Net Logistic Regression with polynomial features performance on test data:")
score_classifier(
    pipelines_q10_logistic_pol,
    x_train_10_pol,
    x_test_10_pol,
    y_train_10,
    y_test_10)

In [ ]:
# compare first baseline model to polynomial model 
print("Baseline Elastic-Net Logistic Regression performance on test data:")
score_classifier(
    pipelines_q10_logistic,
    x_train_10,
    x_test_10,
    y_train_10,
    y_test_10)

In [ ]:
# extract weights from the fitted model
logistic_model_pol = pipelines_q10_logistic_pol.named_steps["model"]

# feature names after preprocessing
feature_names = pipelines_q10_logistic_pol.named_steps["imputer"].get_feature_names_out(
    x_train_10_pol.columns
)

# coefficients (binary classification -> 1D)
coef = logistic_model_pol.coef_.ravel()

# build dataframe of weights
weights_df_q10_elastic_pol = pd.DataFrame({
    "feature": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_q10_elastic_pol["abs_weight"] = weights_df_q10_elastic_pol["weight"].abs()
weights_df_q10_elastic_pol = (
    weights_df_q10_elastic_pol
      .sort_values("abs_weight", ascending=False)
      .reset_index(drop=True)
)

# check
weights_df_q10_elastic_pol

In [ ]:
# cv
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=n_number)

# define threshold
initial_threshold = [0.05]
elasticnetcv_q10_comparisons_pol = []

# filter-out variables 
keep_features_pol = weights_df_q10_elastic_pol[
    weights_df_q10_elastic_pol["abs_weight"] >= 0.05
].sort_values("abs_weight", ascending=False)["feature"].tolist()

# create train and test samples 
x_train_reduced_10_pol = x_train_10_pol[keep_features_pol]
x_test_reduced_10_pol = x_test_10_pol[keep_features_pol]

# loop to drop each variable 1 by one 
for k in range(len(keep_features_pol), 0, -1):
    keep_pool_pol = keep_features_pol[:k]

    x_train_reduced_10_pol = x_train_10_pol[keep_pool_pol]

    pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
        ("model", LogisticRegressionCV(
            cv=5,
            l1_ratios=[0.1, 0.5, 0.9],
            Cs=np.logspace(-3, 2, 6),
            penalty="elasticnet",
            solver="saga",
            scoring="roc_auc",
            max_iter=10000,
            random_state=n_number,
            n_jobs=1
        ))
    ])


    results_reduced_10_pol = cross_validate(
        pipeline,
        x_train_reduced_10_pol,
        y_train_10,
        cv=cv,
        scoring=["roc_auc", "accuracy", "precision", "recall", "f1"],
        n_jobs=-1,
        return_train_score=False
    )

    elasticnetcv_q10_comparisons_pol.append({
        "model": f"ElasticNetCV | top-{k} by |w|",
        "ROC_AUC_mean": results_reduced_10_pol["test_roc_auc"].mean(),
        "Accuracy_mean": results_reduced_10_pol["test_accuracy"].mean(),
        "Precision_mean": results_reduced_10_pol["test_precision"].mean(),
        "Recall_mean": results_reduced_10_pol["test_recall"].mean(),
        "F1_mean": results_reduced_10_pol["test_f1"].mean(),
        "num_features": k
    })

# dataframe 
drop2_df = (pd.DataFrame(elasticnetcv_q10_comparisons_pol)
            .sort_values("ROC_AUC_mean", ascending=False)
            .reset_index(drop=True))
drop2_df


In [ ]:
# check dataframe with best models 
compare_models_q10_logistic_df

The ElasticNet logistic regression with polynomial features performs worse than the baseline ElasticNet model across all cross-validated metrics and on the held-out test set. This consistent degradation indicates that polynomial expansion does not add useful signal and instead increases noise and variance. As a result, we do not proceed with further feature selection or trimming on this specification and retain our best ElasticNetCV top-6 features model.

#### Index Features 

In [ ]:
# copy x_train_10 and x_test_10 again for safety 
# since we will be doing a lot of feature engineering, we want to keep the original data safe for reference and for any future use
x_train_10_indexes = x_train_10.copy()
x_test_10_indexes = x_test_10.copy()

# check correlation first 
x_y_index_corr = x_train_10_indexes.corrwith(y_train_10).sort_values(ascending= False)
x_y_index_corr

In [ ]:
# indexes
# teaching quality group
teaching_quality = [
    "inspirational", 
    "amazing_lectures",
    "respected", 
    "good_feedback", 
    "caring",
    "hilarious", 
    "accessible", 
    "tough_grader"
]

# structure group 
structure_positive = [
    "clear_grading", 
    "extra_credit"
]

structure_negative = [
    "pop_quizzes", 
    "group_projects", 
    "graded_by_few_things"
]

# workload group
workload = [
    "participation_matters", 
    "so_many_papers", 
    "test_heavy", 
    "lots_to_read", 
    "dont_skip_class_or_you_will_not_pass", 
    "lots_of_homework", 
    "lecture_heavy"
]

# train data 
x_train_10_indexes["teaching_quality_index"] = x_train_10_indexes[teaching_quality].mean(axis=1)
x_train_10_indexes["workload_index"] = x_train_10_indexes[workload].mean(axis=1)
x_train_10_indexes["structure_index_positive"] = x_train_10_indexes[structure_positive].mean(axis=1)
x_train_10_indexes["structure_index_negative"] = x_train_10_indexes[structure_negative].mean(axis=1)

# test data 
x_test_10_indexes["teaching_quality_index"] = x_test_10_indexes[teaching_quality].mean(axis=1)
x_test_10_indexes["workload_index"] = x_test_10_indexes[workload].mean(axis=1)
x_test_10_indexes["structure_index_positive"] = x_test_10_indexes[structure_positive].mean(axis=1)
x_test_10_indexes["structure_index_negative"] = x_test_10_indexes[structure_negative].mean(axis=1)

In [ ]:
# check correlation first 
x_y_index_corr = x_train_10_indexes.corrwith(y_train_10).sort_values(ascending= False)
x_y_index_corr

In [ ]:
# cv 
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=n_number
)

# pipelines for logistic models
pipelines_q10_logistic_index = Pipeline([
    # imputer 
    ("imputer", SimpleImputer(strategy="mean")),

    # scaler
    ("scaler", StandardScaler()),

    # model itself
    ("model", LogisticRegressionCV(
        cv=5,
        l1_ratios=[0.1, 0.5, 0.9],
        Cs = np.logspace(-3, 2, 6),
        penalty='elasticnet',  
        solver='saga',
        scoring='roc_auc',
        max_iter=10000,
        random_state=n_number,
        n_jobs= 1
    ))
])


# cross-validation
results_index = cross_validate(
    pipelines_q10_logistic_index,
    x_train_10_indexes,
    y_train_10,
    cv=cv,
    scoring=['roc_auc', 
             'accuracy', 
             'precision', 
             'recall', 
             'f1'],
    n_jobs=-1,  
    return_train_score=False
)

models_comparison.append({
        "model": "Logistic Regression ElasticNet (indexes)",
        "ROC_AUC_mean": results_index["test_roc_auc"].mean(),
        "Accuracy_mean": results_index["test_accuracy"].mean(),
        "Precision_mean": results_index["test_precision"].mean(),
        "Recall_mean": results_index["test_recall"].mean(),
        "F1_mean": results_index["test_f1"].mean()
    })

# save to the dataset 
compare_models_q10_logistic_df = (
    pd.DataFrame(models_comparison)
      .sort_values("ROC_AUC_mean", ascending=False)
      .reset_index(drop=True)
)

# check 
compare_models_q10_logistic_df

In [ ]:
# assess model with indexes on test data
print("ElasticNetCV with indexes:")
score_classifier(
    pipelines_q10_logistic_index,
    x_train_10_indexes,
    x_test_10_indexes,
    y_train_10,
    y_test_10)

In [ ]:
# assess model with 6 features on test data
print("ElasticNetCV with 6 features:")
score_classifier(
    pipelines_q10_logistic_reduced,
    x_train_reduced_10_final,
    x_test_reduced_10_final,
    y_train_10,
    y_test_10)

In [ ]:
# extract weights from the fitted model
logistic_model_indexes = pipelines_q10_logistic_index.named_steps["model"]

# feature names after preprocessing
feature_names = pipelines_q10_logistic_index.named_steps["imputer"].get_feature_names_out(
    x_train_10_indexes.columns
)

# coefficients (binary classification -> 1D)
coef = logistic_model_indexes.coef_.ravel()

# build dataframe of weights
weights_df_q10_elastic_indexes = pd.DataFrame({
    "feature": feature_names,
    "weight": coef
})

# sort by absolute weight
weights_df_q10_elastic_indexes["abs_weight"] = weights_df_q10_elastic_indexes["weight"].abs()
weights_df_q10_elastic_indexes = (
    weights_df_q10_elastic_indexes
      .sort_values("abs_weight", ascending=False)
      .reset_index(drop=True)
)

# check
weights_df_q10_elastic_indexes

In [ ]:
# cv
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=n_number)

# define threshold
initial_threshold = [0.05]
elasticnetcv_q10_comparisons_indexes = []

# filter-out variables 
keep_features_indexes = weights_df_q10_elastic_indexes[
    weights_df_q10_elastic_indexes["abs_weight"] >= 0.05
].sort_values("abs_weight", ascending=False)["feature"].tolist()

# create train and test samples 
x_train_reduced_10_indexes = x_train_10_indexes[keep_features_indexes]
x_test_reduced_10_indexes = x_test_10_indexes[keep_features_indexes]

# loop to drop each variable 1 by one 
for k in range(len(keep_features_indexes), 0, -1):
    keep_pool_indexes = keep_features_indexes[:k]

    x_train_reduced_10_indexes= x_train_10_indexes[keep_pool_indexes]

    pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
        ("model", LogisticRegressionCV(
            cv=5,
            l1_ratios=[0.1, 0.5, 0.9],
            Cs=np.logspace(-3, 2, 6),
            penalty="elasticnet",
            solver="saga",
            scoring="roc_auc",
            max_iter=10000,
            random_state=n_number,
            n_jobs=1
        ))
    ])


    results_reduced_10_indexes = cross_validate(
        pipeline,
        x_train_reduced_10_indexes,
        y_train_10,
        cv=cv,
        scoring=["roc_auc", "accuracy", "precision", "recall", "f1"],
        n_jobs=-1,
        return_train_score=False
    )

    elasticnetcv_q10_comparisons_indexes.append({
        "model": f"ElasticNetCV | top-{k} by |w|",
        "ROC_AUC_mean": results_reduced_10_indexes["test_roc_auc"].mean(),
        "Accuracy_mean": results_reduced_10_indexes["test_accuracy"].mean(),
        "Precision_mean": results_reduced_10_indexes["test_precision"].mean(),
        "Recall_mean": results_reduced_10_indexes["test_recall"].mean(),
        "F1_mean": results_reduced_10_indexes["test_f1"].mean(),
        "num_features": k
    })

# dataframe 
drop3_df = (pd.DataFrame(elasticnetcv_q10_comparisons_indexes)
            .sort_values("ROC_AUC_mean", ascending=False)
            .reset_index(drop=True))
drop3_df


In [ ]:
# check dataframe with best models 
compare_models_q10_logistic_df

This experiment evaluates whether replacing raw features with index-based representations improves model performance. The results show that index-based features do not provide a meaningful advantage over the original feature formulation.

Performance across all index-based Elastic Net variants remains nearly identical to the baseline model, with ROC AUC around 0.79 and no consistent improvement in accuracy, precision, recall, or F1. Any observed differences are small and fall within cross-validation noise.

Importantly, index-based models introduce additional abstraction without increasing predictive power. They neither improve separation nor enable a simpler model with better performance.

Index-based feature representations do not add new signal in this setting. Since they increase complexity without improving performance or interpretability, they are rejected. The original feature formulation with Elastic Net regularization remains the preferred approach.

Up to this point, model selection prioritized balance, interpretability, and deployment stability. In the final experiment, we change the objective from balanced performance to pure predictive performance maximization.

Under this setting, interpretability and coefficient stability are no longer primary constraints. Instead, the goal is to assess whether more flexible, non-linear models can extract additional signal beyond what linear Elastic Net models can capture.

To evaluate this upper bound on performance, we employ LightGBM and XGBoost, which are well-suited for handling non-linear interactions, feature hierarchies, and complex decision boundaries. These models are optimized directly for predictive performance and serve as a benchmark for the maximum achievable performance given the available features.

This comparison allows us to determine whether the observed performance plateau is a limitation of linear modeling or a fundamental constraint imposed by the data itself.

#### Extra: Prediction Accuracy Optimization

#### LightGBM

In [ ]:
# cv 
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=n_number
)

# pipelines for logistic models
# pipeline for LightGBM
lgbm_pipe_q10 = Pipeline([
    # scaler
    ("scaler", StandardScaler()), 

    # model 
    ("model", LGBMClassifier(
        objective = "binary",
        random_state = n_number, 
        n_jobs = 1, 
        verbosity = -1
    ))
])

# objective for tuning 
def objective_lgbm_q10(trial):
    params = {
    "model__n_estimators": trial.suggest_int("model__n_estimators", 300, 3000),
    "model__learning_rate": trial.suggest_float("model__learning_rate", 0.005, 0.15, log=True),

    "model__num_leaves": trial.suggest_int("model__num_leaves", 15, 255),
    "model__max_depth": trial.suggest_int("model__max_depth", -1, 16),

    "model__min_child_samples": trial.suggest_int("model__min_child_samples", 5, 200),
    "model__min_child_weight": trial.suggest_float("model__min_child_weight", 1e-3, 10.0, log=True),
    "model__min_split_gain": trial.suggest_float("model__min_split_gain", 0.0, 1.0),

    "model__subsample": trial.suggest_float("model__subsample", 0.6, 1.0),
    "model__colsample_bytree": trial.suggest_float("model__colsample_bytree", 0.6, 1.0),

    "model__reg_alpha": trial.suggest_float("model__reg_alpha", 1e-8, 10.0, log=True),
    "model__reg_lambda": trial.suggest_float("model__reg_lambda", 1e-8, 10.0, log=True),

    "model__max_bin": trial.suggest_int("model__max_bin", 63, 511),

    "model__scale_pos_weight": trial.suggest_float("model__scale_pos_weight", 0.5, 5.0),
}

    # copy the pipeline
    pipe = clone(lgbm_pipe_q10)
    pipe.set_params(**params)

    # tune the score
    scores_lgbm_q10 = cross_val_score(
        pipe,
        x_train_10,
        y_train_10,
        cv=cv,
        scoring="roc_auc",
        n_jobs=-1
    )
    return scores_lgbm_q10.mean()

# start optuna study 
study_lgbm_q10 = start_study_optuna(
    objective=objective_lgbm_q10,
    n_trials=50,
    sampler_seed=n_number,
    direction="maximize"
)

# set the best parameters to the pipeline
lgbm_pipe_q10.set_params(**study_lgbm_q10.best_params)

cv_results_raw_lgbm_q10 = cross_validate(
    lgbm_pipe_q10,
    x_train_10,
    y_train_10,
    cv=cv,  # StratifiedKFold
    scoring=("roc_auc", "accuracy", "precision", "recall", "f1"),
    return_train_score=False,
    return_estimator=True
)


# results
models_comparison.append({
        "model": "LightGBM (tuned)",
        "ROC_AUC_mean": cv_results_raw_lgbm_q10["test_roc_auc"].mean(),
        "Accuracy_mean": cv_results_raw_lgbm_q10["test_accuracy"].mean(),
        "Precision_mean": cv_results_raw_lgbm_q10["test_precision"].mean(),
        "Recall_mean": cv_results_raw_lgbm_q10["test_recall"].mean(),
        "F1_mean": cv_results_raw_lgbm_q10["test_f1"].mean()
    })

# show as dataframe 
compare_models_q10_logistic_df = (
    pd.DataFrame(models_comparison)
    .reset_index(drop = True)
    .sort_values(by = "ROC_AUC_mean", ascending = False))

# check 
compare_models_q10_logistic_df


Looking at cross-validation first, the key result is that all models achieve almost identical ROC AUC around 0.79, which means they rank observations equally well and extract essentially the same signal. The main difference is not discrimination but decision behavior. Tuned LightGBM reaches extremely high recall ( ~ 91%) but with much lower precision (~ 55%), indicating an aggressive positive classification strategy that trades many false positives for sensitivity. In contrast, ElasticNet and logistic models operate at a more balanced point, with recall and precision both around 0.66-0.68 without any thresholding yet. Since AUC is the same across models, LightGBM is not learning a better representation of the problem, it is simply choosing a different operating point. 

In [ ]:
# assess lightgbm on test data
print("LightGBM (tuned) model performance on test data:")
score_classifier(lgbm_pipe_q10, 
                 x_train_10, 
                 x_test_10, 
                 y_train_10, 
                y_test_10)

In [ ]:
# assess 6 feature model on test data
print("ElasticNetCV with 6 features model performance on test data:")
score_classifier(pipelines_q10_logistic_reduced, 
                 x_train_reduced_10_final, 
                 x_test_reduced_10_final, 
                 y_train_10, 
                y_test_10, 
                threshold = 0.27)

In [ ]:
# confusion matrix 
# predict 
y_test_10_pred_lgbm = lgbm_pipe_q10.predict(x_test_10)
y_test_prob_10_lgbm = lgbm_pipe_q10.predict_proba(x_test_10)[:, 1]

cm_lgbm = confusion_matrix(y_test_10, y_test_10_pred_lgbm)

# confusion matrixes or both models 
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# ElasticNet CM
ConfusionMatrixDisplay(cm).plot(
    ax=axes[0],
    cmap="Blues",
    values_format="d"
)
# aesthetics 
axes[0].set_title("ElasticNet (6 features, threshold = 0.27)", 
                  fontweight = "bold",
                  fontsize = 18)
axes[0].set_xlabel("Predicted", 
                   fontsize = 16)
axes[0].set_ylabel("Actual", 
                   fontsize = 16)

# LightGBM CM
ConfusionMatrixDisplay(cm_lgbm).plot(
    ax=axes[1],
    cmap="Blues",
    values_format="d"
)
# aesthetics 
axes[1].set_title("LightGBM (default threshold = 0.5)", 
                  fontweight = "bold", 
                  fontsize = 18)
axes[1].set_xlabel("Predicted", 
                   fontsize = 16)
axes[1].set_ylabel("Actual", 
                   fontsize = 16)

# show the plot
plt.show()


On test data, both models converge to almost the same operating point. ROC AUC differs by about 0.01, which is within noise and does not indicate a meaningful ranking advantage for LightGBM. Accuracy, precision, recall, and F1 are nearly identical. LightGBM achieves slightly higher recall, but this comes with lower specificity and more false positives, as seen in the confusion matrix. ElasticNet trades a small amount of recall for better specificity, while matching LightGBM on F1 almost exactly. This confirms that both models are extracting the same signal and making similar errors. The boosted model is not discovering new structure, only shifting the decision boundary. In practical terms, this is convergence across model families, which signals that additional complexity is not buying real performance gains.

In [ ]:
# create the figure once
fig, ax = plt.subplots(figsize=(12, 10))

# diagonal baseline once
ax.plot([0, 1], [0, 1], linewidth=2)

# predictions on probability 
y_test_prob_10_lgbm = lgbm_pipe_q10.predict_proba(x_test_10)[:, 1]
y_test_prob_10_elastic = pipelines_q10_logistic_reduced.predict_proba(x_test_reduced_10_final)[:, 1]

# add models using our function 
add_roc(ax, y_test_10, y_test_prob_10_lgbm, "LightGBM (tuned)")
add_roc(ax, y_test_10, y_test_prob_10_elastic, "ElasticNetCV | top-{6} features by |w|")
# aesthetics 
# title
ax.set_title("AUROC Curve", fontsize=18, fontweight="bold")

# xlabel
ax.set_xlabel("False Positive Rate", fontsize=16)
# ylabel
ax.set_ylabel("True Positive Rate", fontsize=16)

# tick_params
ax.tick_params(labelsize=14)

# legend
ax.legend(frameon=True, loc="lower right")

# show the plot
plt.show()


We evaluated whether switching from a parsimonious, interpretable Elastic Net model to a fully tuned LightGBM model yields a meaningful performance improvement.

LightGBM achieves a slightly higher ROC AUC (≈ 0.796 vs ≈ 0.786). However, the improvement is modest and does not translate into a qualitatively different error profile. The ROC curves largely overlap, indicating that both models rank observations similarly across most operating points.

From the confusion matrices, LightGBM identifies slightly more true positives and slightly fewer false negatives at its default threshold, but this comes with a comparable increase in false positives. When thresholds are adjusted to match recall levels, the practical difference between the two models further narrows.

While LightGBM marginally improves ranking performance, it introduces:
- Substantially higher model complexity
- Reduced interpretability
- Additional tuning and maintenance overhead

Given that the performance gain is small and does not change downstream decisions, the added complexity is not justified for this task.

The data appears to be signal-limited rather than model-limited. Even a highly flexible, tuned LightGBM model cannot extract substantially more information than the 6-feature Elastic Net model.

As a result, we do not proceed to XGBoost. The expected gains would be similarly marginal while further increasing complexity. The 6-feature Elastic Net model remains the preferred final model due to its strong performance, interpretability, and operational simplicity.